# .----------------------------------------------------------------


---
# 🗲 Seattle Buildings Power Consumption / GHG emissions Predictions 🗲
---
<img src='https://i.snipboard.io/MGbyzf.jpg'/>

---

- This notebook begins with an Eda on building energy consumption ( electric, gaz, steam, miscellaneous ) and GHG emissions as an excuse to explore and compare different regression models ( excluding complex neural networks )
- Then we'll remove unique data which might lead to bad learning an remove closely correlated data in order to match our requirements : predict data when we don't have all the measurements for a building : aka : predict the consumption for a newly built building ( remove anything wich is energy related ), otherwise that would be too simple .. 
- Please note 1Kwh equals 3.412Kbtu, Logistic Regression works better as all data is normalized and some composite data might help us a lot in that process
---
- uses Alpow framework "alpow.py" mainly for save/load functions for resuming program state / training / exports

# .----------------------------------------------------------------
# Benchmarks > Determination coefficient r² and Nb random keys
- Model : Best Score, Seed > Todo > Dict and Table {{}}
- Attention, car optimisation sur jeu de test, donc mesurer résultats sur test !!
---
Enough with 60 keys ?
- PassiveAggressiveRegressorr2:0.88	rd:23
- LinearSVR :r2:0.87:rd:56
- XGBRegressor:r2:0.17:rd:1
- MLPRegressor	r2:0.82	rd:7
---
- Ridge	r2:0.29	rd:1
- ExtraTreesRegressor	r2:0.2	rd:2
- RandomForestRegressor	r2:0.17	rd:59
- AdaBoostRegressor	r2:-0.39	rd:43
- GradientBoostingRegressor	r2:0.18	rd:26
- HistGradientBoostingRegressor	r2:0.21	rd:1
---
Enough avec 30 clés ? Moins
- LinearSVR	r2:0.55	rd:27 < 0.87
- XGBRegressor	r2:0.17	rd:1
- PassiveAggressiveRegressor	r2:0.88	rd:23 (30)
- MLPRegressor	r2:0.82	rd:7
---
Enough 200 clés ? Aucun Intérêt dégradation passiveAgressive
- LinearSVR	r2:0.88	rd:61 =
- XGBRegressor	r2:0.17	rd:1 <
- PassiveAggressiveRegressor	r2:0.62	rd:187 <<
- MLPRegressor	r2:0.82	rd:7 <<<
---
Enough 10 clés ?
- LinearSVR	r2:0.55	rd:1 <
- XGBRegressor	r2:0.17	rd:1
- PassiveAggressiveRegressor	r2:0.89	rd:4
- MLPRegressor	r2:0.83	rd:2







---
#I) Library
- Including some helpfull functions
---

In [2]:
import os;
os.system('rm -f gv.py alpow.py;wget https://alpow.fr/alpow.py;wget https://alpow.fr/gv.py');
if False:#freeze version at first run please !
  os.system('pip freeze > versions.txt')
  !sed -e ':a' -e 'N' -e '$!ba' -e 's/\n/ ; /g' versions.txt
requiredModules='Pillow==7.0.0 absl-py==0.10.0 alabaster==0.7.12 albumentations==0.1.12 altair==4.1.0 argon2-cffi==20.1.0 asgiref==3.3.0 astor==0.8.1 astropy==4.1 astunparse==1.6.3 async-generator==1.10 atari-py==0.2.6 atomicwrites==1.4.0 attrs==20.2.0 audioread==2.1.9 autograd==1.3 Babel==2.8.0 backcall==0.2.0 bcrypt==3.2.0 beautifulsoup4==4.6.3 bleach==3.2.1 blis==0.4.1 bokeh==2.1.1 Bottleneck==1.3.2 branca==0.4.1 bs4==0.0.1 CacheControl==0.12.6 cachetools==4.1.1 catalogue==1.0.0 certifi==2020.6.20 cffi==1.14.3 chainer==7.4.0 chardet==3.0.4 click==7.1.2 cloudpickle==1.3.0 cmake==3.12.0 cmdstanpy==0.9.5 colorlover==0.3.0 community==1.0.0b1 contextlib2==0.5.5 convertdate==2.2.2 coverage==3.7.1 coveralls==0.5 crcmod==1.7 cryptography==3.2.1 cufflinks==0.17.3 cvxopt==1.2.5 cvxpy==1.0.31 cycler==0.10.0 cymem==2.0.4 Cython==0.29.21 daft==0.0.4 dask==2.12.0 dataclasses==0.7 datascience==0.10.6 debugpy==1.0.0 decorator==4.4.2 defusedxml==0.6.0 descartes==1.1.0 dill==0.3.3 distributed==1.25.3 Django==3.1.3 dlib==19.18.0 dm-tree==0.1.5 docopt==0.6.2 docutils==0.16 dopamine-rl==1.0.5 earthengine-api==0.1.238 easydict==1.9 ecos==2.0.7.post1 editdistance==0.5.3 en-core-web-sm==2.2.5 entrypoints==0.3 ephem==3.7.7.1 et-xmlfile==1.0.1 fa2==0.3.5 fancyimpute==0.4.3 fastai==1.0.61 fastdtw==0.3.4 fastprogress==1.0.0 fastrlock==0.5 fbprophet==0.7.1 feather-format==0.4.1 filelock==3.0.12 firebase-admin==4.4.0 fix-yahoo-finance==0.0.22 Flask==1.1.2 folium==0.8.3 future==0.16.0 gast==0.3.3 GDAL==2.2.2 gdown==3.6.4 gensim==3.6.0 geographiclib==1.50 geopy==1.17.0 gin-config==0.3.0 glob2==0.7 google==2.0.3 google-api-core==1.16.0 google-api-python-client==1.7.12 google-auth==1.17.2 google-auth-httplib2==0.0.4 google-auth-oauthlib==0.4.2 google-cloud-bigquery==1.21.0 google-cloud-bigquery-storage==1.1.0 google-cloud-core==1.0.3 google-cloud-datastore==1.8.0 google-cloud-firestore==1.7.0 google-cloud-language==1.2.0 google-cloud-storage==1.18.1 google-cloud-translate==1.5.0 google-colab==1.0.0 google-pasta==0.2.0 google-resumable-media==0.4.1 googleapis-common-protos==1.52.0 googledrivedownloader==0.4 graphviz==0.10.1 grpcio==1.33.2 gspread==3.0.1 gspread-dataframe==3.0.8 gym==0.17.3 h5py==2.10.0 HeapDict==1.0.1 holidays==0.10.3 holoviews==1.13.5 html5lib==1.0.1 httpimport==0.5.18 httplib2==0.17.4 httplib2shim==0.0.3 humanize==0.5.1 hyperopt==0.1.2 ideep4py==2.0.0.post3 idna==2.10 image==1.5.33 imageio==2.4.1 imagesize==1.2.0 imbalanced-learn==0.4.3 imblearn==0.0 imgaug==0.2.9 importlib-metadata==2.0.0 importlib-resources==3.3.0 imutils==0.5.3 inflect==2.1.0 iniconfig==1.1.1 intel-openmp==2020.0.133 intervaltree==2.1.0 ipykernel==4.10.1 ipython==5.5.0 ipython-genutils==0.2.0 ipython-sql==0.3.9 ipywidgets==7.5.1 itsdangerous==1.1.0 jax==0.2.4 jaxlib==0.1.56+cuda101 jdcal==1.4.1 jedi==0.17.2 jieba==0.42.1 Jinja2==2.11.2 joblib==0.17.0 jpeg4py==0.1.4 jsonschema==2.6.0 jupyter==1.0.0 jupyter-client==5.3.5 jupyter-console==5.2.0 jupyter-core==4.6.3 jupyterlab-pygments==0.1.2 kaggle==1.5.9 kapre==0.1.3.1 Keras==2.4.3 Keras-Preprocessing==1.1.2 keras-vis==0.4.1 kiwisolver==1.3.1 knnimpute==0.1.0 korean-lunar-calendar==0.2.1 librosa==0.6.3 lightgbm==2.2.3 llvmlite==0.31.0 lmdb==0.99 lucid==0.3.8 LunarCalendar==0.0.9 lxml==4.2.6 Markdown==3.3.3 MarkupSafe==1.1.1 matplotlib==3.2.2 matplotlib-venn==0.11.6 missingno==0.4.2 mistune==0.8.4 mizani==0.6.0 mkl==2019.0 mlxtend==0.14.0 more-itertools==8.6.0 moviepy==0.2.3.5 mpmath==1.1.0 msgpack==1.0.0 multiprocess==0.70.10 multitasking==0.0.9 murmurhash==1.0.3 music21==5.5.0 natsort==5.5.0 nbclient==0.5.1 nbconvert==5.6.1 nbformat==5.0.8 nest-asyncio==1.4.2 networkx==2.5 nibabel==3.0.2 nltk==3.2.5 notebook==5.3.1 np-utils==0.5.12.1 numba==0.48.0 numexpr==2.7.1 numpy==1.18.5 nvidia-ml-py3==7.352.0 oauth2client==4.1.3 oauthlib==3.1.0 okgrade==0.4.3 opencv-contrib-python==4.1.2.30 opencv-python==4.1.2.30 openpyxl==2.5.9 opt-einsum==3.3.0 osqp==0.6.1 packaging==20.4 palettable==3.3.0 pandas==1.1.4 pandas-datareader==0.9.0 pandas-gbq==0.13.3 pandas-profiling==1.4.1 pandocfilters==1.4.3 panel==0.9.7 param==1.10.0 paramiko==2.7.2 parso==0.7.1 pathlib==1.0.1 patsy==0.5.1 pexpect==4.8.0 pickleshare==0.7.5 Pillow==7.0.0 pip-tools==4.5.1 plac==1.1.3 plotly==4.4.1 plotnine==0.6.0 pluggy==0.7.1 portpicker==1.3.1 prefetch-generator==1.0.1 preshed==3.0.2 prettytable==1.0.1 progressbar2==3.38.0 prometheus-client==0.8.0 promise==2.3 prompt-toolkit==1.0.18 protobuf==3.12.4 psutil==5.4.8 psycopg2==2.7.6.1 ptyprocess==0.6.0 py==1.9.0 pyarrow==0.14.1 pyasn1==0.4.8 pyasn1-modules==0.2.8 pycocotools==2.0.2 pycparser==2.20 pyct==0.4.8 pydata-google-auth==1.1.0 pydot==1.3.0 pydot-ng==2.0.0 pydotplus==2.0.2 PyDrive==1.3.1 pyemd==0.5.1 pyglet==1.5.0 Pygments==2.6.1 pygobject==3.26.1 pymc3==3.7 PyMeeus==0.3.7 pymongo==3.11.0 pymystem3==0.2.0 PyNaCl==1.4.0 PyOpenGL==3.1.5 pyparsing==2.4.7 pyrsistent==0.17.3 pysftp==0.2.9 pysndfile==1.3.8 PySocks==1.7.1 pystan==2.19.1.1 pytest==3.6.4 python-apt==1.6.5+ubuntu0.3 python-chess==0.23.11 python-dateutil==2.8.1 python-louvain==0.14 python-slugify==4.0.1 python-utils==2.4.0 pytz==2018.9 pyviz-comms==0.7.6 PyWavelets==1.1.1 PyYAML==3.13 pyzmq==19.0.2 qtconsole==4.7.7 QtPy==1.9.0 regex==2019.12.20 requests==2.23.0 requests-oauthlib==1.3.0 resampy==0.2.2 retrying==1.3.3 rpy2==3.2.7 rsa==4.6 scikit-image==0.16.2 scikit-learn==0.22.2.post1 scipy==1.4.1 screen-resolution-extra==0.0.0 scs==2.1.2 seaborn==0.11.0 Send2Trash==1.5.0 setuptools-git==1.2 Shapely==1.7.1 simplegeneric==0.8.1 six==1.15.0 sklearn==0.0 sklearn-pandas==1.8.0 slugify==0.0.1 smart-open==3.0.0 snowballstemmer==2.0.0 sortedcontainers==2.2.2 spacy==2.2.4 Sphinx==1.8.5 sphinxcontrib-serializinghtml==1.1.4 sphinxcontrib-websupport==1.2.4 SQLAlchemy==1.3.20 sqlparse==0.4.1 srsly==1.0.2 statsmodels==0.10.2 sympy==1.1.1 tables==3.4.4 tabulate==0.8.7 tblib==1.7.0 tensorboard==2.3.0 tensorboard-plugin-wit==1.7.0 tensorboardcolab==0.0.22 tensorflow==2.3.0 tensorflow-addons==0.8.3 tensorflow-datasets==4.0.1 tensorflow-estimator==2.3.0 tensorflow-gcs-config==2.3.0 tensorflow-hub==0.10.0 tensorflow-metadata==0.24.0 tensorflow-privacy==0.2.2 tensorflow-probability==0.11.0 termcolor==1.1.0 terminado==0.9.1 testpath==0.4.4 text-unidecode==1.3 textblob==0.15.3 textgenrnn==1.4.1 Theano==1.0.5 thinc==7.4.0 tifffile==2020.9.3 toml==0.10.2 toolz==0.11.1 torch==1.7.0+cu101 torchsummary==1.5.1 torchtext==0.3.1 torchvision==0.8.1+cu101 tornado==5.1.1 tqdm==4.41.1 traitlets==4.3.3 tweepy==3.6.0 typeguard==2.7.1 typing-extensions==3.7.4.3 tzlocal==1.5.1 umap-learn==0.4.6 uritemplate==3.0.1 urllib3==1.24.3 vega-datasets==0.8.0 wasabi==0.8.0 wcwidth==0.2.5 webencodings==0.5.1 webptools==0.0.3 Werkzeug==1.0.1 wget==3.2 widgetsnbextension==3.5.1 wordcloud==1.5.0 wrapt==1.12.1 xarray==0.15.1 xgboost==0.90 xkit==0.0.0 xlrd==1.1.0 xlwt==1.3.0 yellowbrick==0.9.1 zict==2.0.0 zipp==3.4.0'
import alpow;import alpow;from alpow import *

Pillow==7.0.0


---
## Other functions
- Variables and stuff

In [ ]:
#}IO{
noPG=0;#ignores parameters grid .. if it takes too much time
dpi=100  
my_cv=5;#20 nbFolds
nbRdKeys=2
scoring='r2';#neg_mean_squared_error
votingOnReel=1
lastVotingKey=''
nbBestModel4Voting=3
SG('webRepo','http://1.x24.fr/a/jupyter/')

npRandomKeys=randomKeys=[1,2,3,4,12,24,42,68,124,256]
nbRdKeysPerModel={'LinearSVR':56,'XGBRegressor':10,'PassiveAggressiveRegressor':20,'MLPRegressor':10}
nbRdKeysPerModel={'LinearSVR':3,'XGBRegressor':3,'PassiveAggressiveRegressor':3,'MLPRegressor':3}
energies='ENERGYSTARScore,Electricity(kBtu),SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),SteamUse(kBtu),NaturalGas(kBtu)'.split(',')
toPredictBase=toPredict='Electricity(kBtu),ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),SteamUse(kBtu),NaturalGas(kBtu),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu)'.split(',')
#}{


jumpto=0;resumed=0;
allVars={'jumpto':1}
votedm={};k1=0;bestNP=[];previousR2=0;res={};pred={};acy={};nfm={};fi={};ev={};bestparams={};models={};ignoreParameters=0;bss={}
repeatedGridSearchCVSameResults={};bestRandomStateValues={};bestRandom={}

#on exclut ainsi les indices non reporté au SQFT : #
pd.set_option('display.max_rows',900)
pd.set_option('display.max_columns',40)
pd.set_option('display.width',1200)#or evaluate js document frame innerWidth .
plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = (24,12)
plt.rcParams['figure.facecolor'] = 'white'
#pandas as pd,numpy as np,matplotlib.pyplot as plt,
import math,gc,ast,json,hashlib,seaborn as sns,keras,sklearn.metrics,sklearn.model_selection,sklearn.svm,xgboost,numpy,pandas,sklearn.linear_model,sklearn.neural_network,sklearn.dummy,sklearn.metrics,sklearn.model_selection,sklearn.preprocessing,matplotlib,sklearn.ensemble

from mpl_toolkits import mplot3d
from keras.models import Sequential
from keras.optimizers import Adam,Adadelta,Nadam,RMSprop,Adamax,SGD
from keras.layers import Dense, Activation, Dropout
from pandas.plotting import scatter_matrix
from keras.optimizers import SGD  

from sklearn import ensemble, tree, linear_model,metrics,utils,preprocessing,metrics,feature_selection,model_selection
from sklearn.preprocessing import LabelEncoder,MinMaxScaler #Imputer,
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
#!pip install 
#import sklearn.cross_decomposition.PLSRegression

import sklearn.gaussian_process,sklearn.kernel_ridge
from sklearn.experimental import enable_hist_gradient_boosting
#from sklearn.linear_model import Ridge, BayesianRidge, LinearRegression, Lasso, ElasticNet,MultiTaskLasso,MultiTaskElasticNet,Lars,LassoLars,OrthogonalMatchingPursuit,ARDRegression,TheilSenRegressor,HuberRegressor,SGDRegressor

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,cross_val_score,GridSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_error,mean_squared_log_error,accuracy_score,average_precision_score,f1_score,balanced_accuracy_score,r2_score

from sklearn.ensemble import RandomForestClassifier,VotingClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron,SGDClassifier
from sklearn.tree import DecisionTreeClassifier


from keras.utils import to_categorical
from sklearn.utils import shuffle
from time import time

# Set default for pandas displays
#x1,y1,x2,y2,y_mean,mean=getSplitted(energy,dfNoEnergies)
def getSplitted(energy,_df):    
  global splittedData
  Hashv=str(hash(_df.values.tobytes()))
  k='labEnc-'+energy+'-'+Hashv
  if(k in splittedData):
    print(energy+' dataframe split exists')
    x1, y1, x2, y2, mean = splittedData[k]
  else:
    _df2=labelEncoded.copy()   
    if energy not in labEncCols: 
      _df2.insert(2,energy,dfNewKeys[energy].values,True) 
    results=_df2[energy].fillna(0).astype('float')
    mean=results.mean()      
    x1, y1, x2, y2 = ShuffleOrNot(_df,results)#labelEncoded.drop([energy],axis=1)
    splittedData[k]=[x1, y1, x2, y2,mean];

  y_mean = [mean]*len(y2)
  return x1,y1,x2,y2,y_mean,mean

def save(_globals,exclusions=[],fn='allVars',include=False):
    exclusions+='models,allVars,sftp'.split(',')        
    _vars={};_gk=list(_globals.keys())
    if(include):
        _gk=include
    for i in _gk:
        if '_' in i:
            continue;
        if i in _globals.keys():
            if type(_globals[i]) in [str,dict,list,int,pd.DataFrame,pd.Series]:#yyuuuuuu !
                _vars[i]=_globals[i];  
    size={};
    #x=%who_ls str dict list int DataFrame Series
    a=time()
    for i in exclusions:
        if i in _vars.keys():
            del _vars[i]
        
    for i in _vars:#compact
        size[i]=sys.getsizeof(_vars[i])
    print(arsort(size))    
    #print(','.join(x)+':: saved')
    os.system('rm '+fn+'.pickle '+fn+'.pickle.zip')
    FPCP(fn,_vars); 
    print('saved in:'+str(time()-a))
    o=subprocess.check_output('zip '+fn+'.pickle.zip '+fn+'.pickle', shell=True);print(o)
    now=datetime.datetime.now()
    fn2=fn+'.'+now.strftime("%Y%m%d-%H%M")+'.pickle.zip'
    os.system('cp '+fn+'.pickle.zip '+fn2)  
    ftpput([fn+'.pickle.zip',fn2])  

#x1, y1, x2, y2 = sklearn.model_selection.train_test_split(dfNoEnergies,results,train_size=0.8, test_size=0.2, random_state=100 )
shuffleData=0
def ShuffleOrNot(_df,results,train_size=0.8,test_size=0.2,random_state=100):
  if shuffleData:
#TypeError: train_test_split train_size Singleton array array cannot be considered a valid collection    
    return sklearn.model_selection.train_test_split(_df,results,train_size=train_size,test_size=test_size,random_state=random_state)
  print('non shuffled')
  return sklearn.model_selection.train_test_split(_df,results,train_size=train_size,test_size=test_size,shuffle=False)#, stratify = None  
  return non_shuffling_train_test_split(_df,results)  

def loadData(f,sep=','):
  return pd.read_csv(f,sep=',')

def pltinit(df,i,j):
    fig,ax=plt.subplots()  
    fig.patch.set_facecolor('white')
    x=df[i];y=df[j];
    plt.title(i+' vs '+j);plt.xlabel(i);plt.ylabel(j);
    return [x,y,filename(i+'.'+j),fig,ax];
    
def plot(df,i,j):
    x,y,fn,fig,ax=pltinit(df,i,j)
    #bestCorrelationsKeys.keys():
    plt.plot(x,y)
    plt.savefig('plot'+fn+'.png', bbox_inches = 'tight',dpi=dpi)
    show()

def scatter1(df,i,j,ts=0):
    x,y,fn,fig,ax=pltinit(df,i,j)  
    if ts:
      plt.gca().set_yticklabels(backgroundcolor='white',labels=y,rotation=(0),fontsize=ts,linespacing=ts)     
      #ax.tick_params(axis='y',which='major',pad=ts)
      #ax.set_yticklabels(labels=y,rotation = (45), fontsize = 10, va='bottom', ha='left')     
    plt.scatter(x,y);
    plt.savefig('scatter'+fn+'.png', bbox_inches = 'tight',dpi=dpi);ftpput('scatter'+fn+'.png')
    show();

#type(sup0)==pandas.core.series.Series

def md5(x):  
  return hashlib.md5(bencode.bencode(x)).hexdigest()        


def NewModel1(x_size, y_size):
    t_model = Sequential()
    t_model.add(Dense(100, activation="tanh", input_shape=(x_size,)))
    t_model.add(Dense(50, activation="relu"))
    t_model.add(Dense(y_size, activation='linear'))
    return(t_model)

def NewModel2(x_size, y_size):
    t_model = Sequential()
    t_model.add(Dense(100, activation="tanh", input_shape=(x_size,)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(50, activation="relu"))
    t_model.add(Dense(20, activation="relu"))
    t_model.add(Dense(y_size, activation='linear'))
    return(t_model)    

def NewModel3(x_size, y_size):
    t_model = Sequential()
    t_model.add(Dense(80, activation="tanh", kernel_initializer='normal', input_shape=(x_size,)))
    t_model.add(Dropout(0.2))
    t_model.add(Dense(120, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1(0.01), bias_regularizer=regularizers.l1(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(20, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1_l2(0.01), bias_regularizer=regularizers.l1_l2(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(10, activation="relu", kernel_initializer='normal'))
    t_model.add(Dropout(0.0))
    t_model.add(Dense(y_size, activation='linear'))
    return(t_model)    

def nn4(x_size, y_size):
  model = Sequential([
  Dense(32, activation='relu', input_shape=(x_size,)),
  Dense(32, activation='relu'),
  Dense(y_size, activation='sigmoid')
  ])
  #model.compile(optimizer='sgd',
  return model  

def histogram(x,name,score):  
  #k=x.history.keys();print(k)#dict_keys(['val_loss', 'val_mean_absolute_error', 'loss', 'mean_absolute_error'])#"mean_squared_error" 
  #mean_squared_logarithmic_error
  first=x.history['mean_squared_error'][:1] 
  last=x.history['mean_squared_error'][-1:]#TypeError: list indices must be integers or slices, not tuple
  #print(first[0]);print(last[0])
  progress=first[0]-last[0];
  print('Progress:'+str(progress))
  #plusieurs champs k
  for i in x.history.keys():
    plt.plot(x.history[i],label=i);
  plt.title(str(score)+'/'+name)
  x=filename(str(name))+'.histogram.png';
  plt.savefig(x,bbox_inches='tight');ftpput(x);plt.close()
  return progress
  show();

#Fit and Predict at the same time !!!
def fit(x,fn=0,noload=0):
  global x1,x2,y1,y2,ep,bs,k,pred,acy,k,toGuess;
  res=0;history=0
  if(fn):#changmenet de shape de données en input ..
    if(noload==0 & os.path.isfile(fn)==True):
      getFile(fn)
      x=keras.models.load_model(fn) 
      print('load model:'+fn)
      res=1;

    if(res==0):
      print('generate model:'+fn)
      history=x.fit(x1,x2,validation_data=(y1,y2),epochs=ep,batch_size=bs,shuffle=True,verbose=0)
      print('_'*160)
      #history=x.fit(standardize(x1),standardize(x2),validation_data=(standardize(y1),standardize(y2)),epochs=ep,batch_size=bs,shuffle=True,verbose=2)
      x.save(fn);ftpput(fn);#zipped ?    

  nfm[k]=x;
  pred[toGuess][k]=x.predict(y1);#mean_squared_log_error
#!!!:ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.  
  acy[toGuess][k]=round(mean_squared_error(y2,pred[toGuess][k]) ** (1/2));
  say(acy[toGuess][k]);print(k+' => '+str(acy[toGuess][k]));
  if(history):
    histogram(history,k,acy[toGuess][k]);
  print('_'*120)

def r2(a,b):
  return stats.pearsonr(a,b)[0] ** 2

def standardize(df):
  mean = np.mean(df, axis=0)
  std = np.std(df, axis=0)#+0.000001
  return (df - mean) / std

def snsscat(df,a='',b='',ts=0,minx=0,maxx=0,miny=0,maxy=0,opacity=0.02,color='blue',size=2,axis=0,xscale=0,yscale=0,fn=0,kind='scatter'):
  if(fn==0):
    fn=filename(a+'.'+b+'.png'); 
  x=df[a];y=df[b];#fig,ax=plt.subplots(1);#ax=ax.flatten()
  slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
  corr=stats.pearsonr(x, y)[0] ** 2
  #x,y,fn2,fig,ax=pltinit(df,a,b,axis)
  #ax.scatter(x,y,alpha=opacity,c=color,s=size);
  if(maxx==0):
    maxx=x.max()  
  if(maxy==0): 
    maxy=y.max()
#https://stackoverflow.com/questions/36191906/rescale-axis-on-seaborn-jointgrid-kde-marginal-plots    'ax':ax, 'height':height,
#https://seaborn.pydata.org/generated/seaborn.JointGrid.html
  args={'dropna':True,'kind':kind,'x':x,'y':y, 'data':df, 'joint_kws': {'alpha':opacity,'color':color},'xlim':[minx,maxx],'ylim':[miny,maxy],'color':'red'} #
  ##regplot() got an unexpected keyword argument 'stat_func'
  #args['stat_func']=r2
  args['marginal_kws']=dict(bins=15, rug=True)
  #args['annot_kws']=dict(stat="r");      #joinplot annot_kws
#N'apparait pas ..

  if(kind in ['reg']):
    args['joint_kws']={'color':color}#reg no opacity
    args['line_kws']={'label':"y={0:.1f}x+{1:.1f}".format(slope,intercept)};#not hex,nor kde  
    args['scatter_kws']={'alpha':opacity}; #,nor kde
  if(kind in ['kde']):    
    args['joint_kws']={'color':color}
    args['marginal_kws']={}#no bins property
    pass;

  #print(args);#TypeError: regplot() got an unexpected keyword argument 'alpha',dir(ax)
  g=sns.jointplot(**args) #unpack 
  #g=sns.JointGrid(**args)  
  """
  g = sns.JointGrid(x="total_bill", y="tip", data=tips)
  g = g.plot_joint(plt.scatter,color="g", s=40, edgecolor="white")
  g = g.plot_marginals(sns.distplot, kde=False, color="g")
  g = g.annotate(stats.pearsonr)

  g = g.plot_joint(sns.kdeplot, cmap="Blues_d")
  g = g.plot_marginals(sns.kdeplot, shade=True)
  """
  if(xscale):
    g.set_xscale(xscale)
  if(yscale):
    g.set_yscale(yscale)
  #ax.set_xlabel(a);ax.set_ylabel(b);#déjà attribués
#AttributeError: 'JointGrid' object has no attribute 'set_xlim'     
  if((kind in ['reg','kde']) & False):      
    regline=g.ax_joint.get_lines()[0]
    regline.set_color('red')
    regline.set_zorder('5')
  #fig.text(1, 1, "An annotation", horizontalalignment='left', size='medium', color='black', weight='semibold')
  #dir(g)
  #rsquare = lambda a, b: 
  #rsquare = lambda a, b: 2;g.annotate(rsquare)
  #ax = g.axes[0,0]
  plt.title(str(int(slope))+'-'+str(int(intercept)))
  g.savefig(fn, bbox_inches='tight');
  webp(fn)
  #ftpput(fn)     
  show()
  #plt.show()
  return 1;  

#snsscat(_df,'Real','Guess',opacity=0.2,color=[1,0,0.3,0.01],kind='reg',fn=filename('reg-'+toGuess+'-'+k)+'.png')#Reg  

def saveEvaluation():
  print('\n\n\n');
  message('evaluations saved')
  save(globals(),[],'modelsEvalutation-'+'-'+filename(','.join(models.keys())),'energy,dfn,r2s,pred,resultsArray,y_means,bestParameters,ev,acy'.split(','))

SG('webpBroken',1)
def scatter(df,a='',b='',ts=0,minx=0,maxx=0,miny=0,maxy=0,opacity=0.02,color='blue',size=2,axis=0,xscale=0,yscale=0,reg=0,fn=0):
  if(fn==0):
    fn=filename(a+'.'+b+'.png'); 
  #recombiné avec son index  
  if(type(df)==pd.core.series.Series):#{
    p('serie')
    if(len(df)==0):
      print('empty.. skipping')
      return 0;   
    #x=df.index
    #print('serie')
    if(maxy==0):#maxx or de propos
      maxy=df.max()  
    x=range(df.shape[0])  

    if(axis):
      axis.scatter(x,df,s=size,alpha=opacity)
      #axis.set_title(fn)
      #  ax.set_xlim(minx,maxx);ax.set_ylim(miny,maxy)
      axis.set_xlabel(a)
      axis.set_ylabel(b)
      axis.set_facecolor('white')
      if(yscale):
        axis.set_yscale(yscale)
      if(xscale):
        axis.set_xscale(xscale)
      return 1;
    
    plt.figure().set_facecolor('white')
    plt.scatter(x,df,s=size,alpha=opacity)
    #plt.title(fn)
    if maxy:
      plt.ylim(miny,maxy)      
    plt.xlabel(a)
    plt.ylabel(b)
    if(yscale):
      plt.yscale(yscale)
    if(xscale):
      plt.xscale(xscale)
    plt.savefig(fn, bbox_inches='tight')
    #ftpput(fn) 
    webp(fn)#show()
    return fn;
#}end pandas serie    
### Sinon Dataframe ordinaire
  elif(axis==0):
    x=df[a];y=df[b]
    args={'x':x,'y':y, 'data':df, 'joint_kws': {'alpha':opacity,'color':color}}
    if(reg):
      args['kind']='reg';      
      args['joint_kws']={'color':color};     
      args['scatter_kws']={'alpha':opacity};     
      
    #print(args);#TypeError: regplot() got an unexpected keyword argument 'alpha'
    sns.jointplot(**args).savefig(fn, bbox_inches='tight');
    webp(fn)#show()
    return fn;
### Rendu sur un axe ..
  x,y,fn2,fig,ax=pltinit(df,a,b,axis)

  if(maxx==0):
    maxx=x.max()  
  if(maxy!=0): 
    maxy=y.max()

  if(axis!=0):#specifié non généré
    axis.set_xlabel(a);axis.set_ylabel(b);
    axis.scatter(x,y,alpha=opacity,c=color,s=size);
    ax=axis

  ax.set_facecolor('white')

  if(xscale):
    ax.set_xscale(xscale)
  if(yscale):
    ax.set_yscale(yscale)
  return 1
  
  if(ts & False):
    plt.gca().set_yticklabels(backgroundcolor='white',labels=y,rotation=(0),fontsize=ts,linespacing=ts)     
    #ax.tick_params(axis='y',which='major',pad=ts)
    #ax.set_yticklabels(labels=y,rotation = (45), fontsize = 10, va='bottom', ha='left')      
  if(axis==0):    
    if False:    
      plt.scatter(x,y,alpha=opacity,c=color,s=size);    
      #show();
      plt.savefig(fn, bbox_inches='tight')    
      webp(fn)  

#scatter(train,'longitude','latitude',fn='eliottBayLatLonPrint.png')#  
def load2(fn='allVars', onlyIfNotSet=1):
    fns = fn.split(',')
    for fn in fns:
        fn = fn.strip(', \n')
        ok = 1
        if(len(fn) == 0):
            continue
        if(onlyIfNotSet):
            if fn in globals().keys():
                # override empty lists, dict, dataframe and items
                if isinstance(globals()[fn], type):
                    continue
                elif isinstance(globals()[fn], pd.DataFrame):
                    if globals()[fn].shape[0] > 0:
                        continue
                elif(isinstance(globals()[fn], dict)):
                    if(len(globals()[fn]) > 0):
                        continue
                elif(isinstance(globals()[fn], list)):
                    if(len(globals()[fn]) > 0):
                        continue
                elif(isinstance(globals()[fn], scipy.sparse.csr.csr_matrix)):
                    if(globals()[fn].shape[0] > 0):
                        continue
                elif(isinstance(globals()[fn], np.ndarray)):
                    if(globals()[fn].shape[0] > 0):
                        continue
    # si déjà définie, passer au prochain
                elif(globals()[fn]):
                    continue
        globals().update(alpow.resume(fn))
    # endfor fn
    return

def save2(
  exc=[],
  fn='allVars',
  include=False,
  backup=False,
  ftp=True,
  cleanup=False,
  zip=True,
  authTypes=[
      str,
      dict,
      list,
      int,
      np.ndarray,
      pd.DataFrame,
      pd.Series]):
  if (not GG('useFTP')) | (GG('sftp')['h'] == '-'):
      p('ftp offline')
      return

  global ftplist
  if(isinstance(exc, str)):  # quicksave single var
      excs = exc.split(',')
      for exc in excs:
          exc = exc.strip(', \n')
          if(len(exc) == 0):
              continue
          fn = exc
          include = [exc]
          exc = []
          alpow._save(
              globals(),
              exclusions=exc,
              fn=fn,
              include=include,
              backup=backup,
              ftp=ftp,
              cleanup=cleanup,
              zip=zip,
              authTypes=False)
      # p(excs)
      return 1
  elif exc == []:
      exc = exclusions
  alpow._save(
      globals(),
      exclusions=exc,
      fn=fn,
      include=include,
      backup=backup,
      ftp=ftp,
      cleanup=cleanup,
      zip=zip,
      authTypes=authTypes)
exclusions=['dnotknow-allwaysExlucdeThisVarFromBeingSaved']

import collections
def asort(dict):
  if type(dict) == list:
    return dict  # allready
  return sorted(dict.items(), key=operator.itemgetter(1), reverse=False)
  #return collections.OrderedDict({k: v for k, v in sorted(dict.items(), key=lambda item: item[1])} )        

def arsort(dict):
  if type(dict) == list:
    return dict  # allready        
  return sorted(dict.items(), key=operator.itemgetter(1), reverse=True)
  #return collections.OrderedDict({k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)} )        

p(date())

2020-11-09 08:00:54.849173


---
###GridSearchCV
---


In [ ]:
#gridsearchcv
def modelsEvalutation(variableAPredireNonPresente=False,reset=False,multiplyPredicitions=False,matchAgainst=''):  
  global bss,grid_search,bestNP,votedm,dfNewKeys,dfNoEnergies,models,voting_grid,energy,dfn,ev,fi,pred,x1,x2,y1,y2,mean,bestParameters,voting,r2s,r2,k1,voted,splittedData,resultsArray,y_means,resultss;#ev,r2,msqrt,grid_search,k2,acy,pred;
  bestNP=[];bss={}
  #ev={};fi={};pred={};r2s={};#reset those indices  
  energiess=toPredict
  #noPG or _ignoreParameters
  if(variableAPredireNonPresente):
    energiess=[variableAPredireNonPresente]

  for energy in energiess:
    toGuess=index=energy;    
    print('_'*180)
    print('}'+energy+'{')
    #_df=labelEncoded.copy()
    
    if(variableAPredireNonPresente):
      results=dfNewKeys[variableAPredireNonPresente].fillna(0).astype('float')
    elif energy not in labEncCols:
      results=dfNewKeys[energy].fillna(0).astype('float')  
      #_df.insert(2,energy,dfNewKeys[energy].values,True) 
    else:
      results=labelEncoded[energy].fillna(0).astype('float')      

    print(results[:2])
    mean=results.mean()

    resultss[energy]=results
    y_means[energy]=mean;    
    
    dfs={'trimmed':dfNoEnergies}
    if(includeOriginalDf):      
      dfs['original']=labelEncoded.drop([energy],axis=1)
    #Ré-incorporation
    if True:
      if(includeEnergyStar):
        dfs['EnergyStar']=dfNoEnergies.copy(deep=True)
        dfs['EnergyStar']['ENERGYSTARScore']=labelEncoded['ENERGYSTARScore']          

    if 'voting' in models.keys():
      del models['voting'];
  #KeyError: 'Electricity(kBtu)-trimmed'SSSSS
    for dfn in dfs:  
      k1=energy+'-'+dfn;
      if(k1 not in r2s.keys()):
        r2s[k1]={}
      if(reset):
        r2s[k1]={}
        
      print('_'*190)
      print('}'+k1+'{')
      dfx=dfs[dfn]
      #history[dfn+'-'+energy]={}           
      if dfn+'-'+energy in splittedData.keys():
        print('Keeping previous splitted data')
        x1, y1, x2, y2, mean = splittedData[dfn+'-'+energy]#todo:laisser en aléatoire : répétant 20 fois, si 2 coups succeffis
      else:
        x1, y1, x2, y2 = ShuffleOrNot(dfx,results)#Stays the same !
        splittedData[dfn+'-'+energy]=[x1, y1, x2, y2,mean];
        resultsArray[dfn+'-'+energy]=y2
      #x1, y1, x2, y2 = sklearn.model_selection.train_test_split(dfx,results,train_size = 0.8, test_size = 0.2, random_state=100 )
      a=time();
      votingModels=[];  
      votedm=[];
      voting_grid={}    

      #}foreach model{    
      for model in models.keys():      
        k2=k1+'-'+model        
        if (voted==1) & (model != 'dummy'):#only voted dummy is cheating !
          votedm+=[model]  
          votingModels.append((model,models[model]))#later
          if model in param_grid.keys():
            for p in param_grid[model]:
              voting_grid[model+'__'+p]=param_grid[model][p]#pas génial ==> trop de candidates
  #'dummy__strategy':['mean','median'],          
  ##for i in param_grid['model']=voting_grid['model__param']= get All Parameters !          
          continue;
        #individual Metrics Here !
        print('\n\t}'+k2+'{\n')
        print('multiplyPredicitions',multiplyPredicitions)

        gsm(models,model,k1,multiplyPredicitions=multiplyPredicitions,matchAgainst=matchAgainst)

        if(len(grid_search.best_params_)):
          print('\tBest Parameters :: ',end='');
          print(len(grid_search.best_params_),end='')
          print(grid_search.best_params_)
          if ignoreParameters:
            print('multiplyPredicitions',multiplyPredicitions);
            gsm(models,model,k1,1,multiplyPredicitions=multiplyPredicitions,matchAgainst=matchAgainst)
        else:
          print('\t no parameters..')
          #.r2_score(y2,predictions)   #.predict(y1)
        #r22=sklearn.metrics.r2_score(y2,grid_search.best_estimator_.predict(y1))#revient à faire exactement la même chose que précédement..
        r22=grid_search.best_score_
        bss[model]=bs=r22;#grid_search.best_score_
        print('\tBS:'+str(bs))
      #}endformodel{

      if 'voting' in r2s[k1].keys():
        del r2s[k1]['voting'];#previous run
      
      print('\n\t\t ==> Best Reel Model r2:',end='')
      print(arsort(r2s[k1])[:1])
      print('\t\t ==> Best Reel Cv Score:',end='')
      print(arsort(bss)[:1])    
  #colect parameters for voting !      
      if voted==2:
        modelOk=0
  #voted=1 blind vote on cv results
        if(votingOnReel):
          topModels=arsort(r2s[k1])
        else:
          topModels=arsort(bss)
        for mdl,r2score in topModels:
          if(mdl in['dummy','voting']):#cheating, no recursion
            continue
          #from another run, subset
          if mdl not in models.keys():
            continue
          if modelOk>nbBestModel4Voting:
            break;
  #ne pas prendre les coeff négatifs ( moins que dummy )           
          if(votingOnReel & (r2score<-0.05)):
            continue
          votingModels.append((mdl,models[mdl]))#selected
          modelOk+=1
          k2=k1+'-'+mdl
          votedm+=[mdl]     
          if k2 in bestParameters.keys():
            for param in bestParameters[k2]:
              #print(param,bestParameters[k2][param])
              voting_grid[mdl+'__'+param]=[bestParameters[k2][param]]    

  #todo: get best 3 models, and their parameters as well .. bestParameters[k2] for voting ==> voting_grid
      #arsort(r2s[k1])[:3] 
      if voted==0:
        print('no voting')
        continue;

      if len(votingModels)==0:
        print('no suitable voting models')
        continue
  #Ensemble methods
      model='voting';k2=energy+'-'+dfn+'-'+model
      print('\n\nvoting between : '+','.join(votedm))
      print(voting_grid)
  #TypeError: VotingRegressor __init__() got an unexpected keyword argument 'scoring'  
      models[model]=sklearn.ensemble.VotingRegressor(votingModels)#,scoring=scoring,, voting='soft'
  #todo:bagging here      
      gsm(models,model,k1,multiplyPredicitions=multiplyPredicitions,matchAgainst=matchAgainst)
      #pour prochains passages
      del models['voting']
      print('\n')
      #print('voting classifier : best :');print(grid_search.best_estimator_);#is voting
      #grid_search.best_estimator_.estimators
      #score(self, X, y[, sample_weight]);Return the coefficient of determination R^2 of the prediction.        
      if(len(r2s[k1])):
        print('\t\t ==> best model r2 with voting is :',end='')
        print(arsort(r2s[k1])[:1])  
      if(len(bss)):
        print('Best CV score')
        best=arsort(bss)[:1][0];print(best)     
      #none#print('\t\t ==> best model cv score is :',end='');print(arsort(bss[k1])[:1])
      print('\t\tVoting Classifier final r2:'+str(r2));                 
      message('all models processed')
      save(globals(),[],'pendingModels-'+filename(energy),'energy,dfn,r2s,pred,resultsArray,y_means'.split(','));#
    #}endfor dataframe{
    if(stopAtOne):
      return;
  message(energy+' processed')
  save(globals(),[],'magicModelsRunnedWithVotingOnAllEnergiesShort-'+filename(energy),'r2s,pred,resultsArray,y_means'.split(','));#
  #}endfor energy{  
  return 1;

def gsm(__models,model,k1,_ignoreParameters=0,abort=False,multiplyPredicitions=False,matchAgainst=''):
  #globales pour écritures,autonome en lecture
  global lastVotingKey,mean,y2,grid_search,bestNP,previousR2,votedm,models,ev,fi,pred,_df,y_mean,per,x,voting,r2,voting_grid,cvResults,bestParameters,k2;#,x1,x2,y1,y2energy,dfn,

  #récupère sur scope global quand non spécifié mais écrit au niveau de la fonction quand non global !
  #global ev,r2,msqrt,grid_search,k2,acy,pred;
  #global energy,dfn,grid_search,mean,param_grid,x1,x2,y1,y1,pred,ev,toGuess,acy,t,a,nbFold,scoring,arretAnticipe,history,bp,bestParameters
  k=model;
  suffix=''
  k2=k1+'-'+model
  p('multiplyPredicitions',multiplyPredicitions)

  if multiplyPredicitions:
    k2+='-M:'+str(multiplyPredicitions)+'-A:'+matchAgainst

  if(model not in param_grid.keys()):
    param_grid[model]={}
    p(model,'no within',param_grid)

  _pg=param_grid[model]

  if model=='voting':
    _pg=voting_grid
    suffix+='-'+','.join(votedm)
    k2+='-'+','.join(votedm)
    lastVotingKey=k2    

  if((_ignoreParameters == 1) or (noPG)):
    suffix+='-np';
    k2+='-np'
    _pg={}

  if(k2 not in ev.keys()):
    ev[k2]={}
      
  #params = {'modelName__Param':[],'logisticregression__C': [1.0, 100.0],'randomforestclassifier__n_estimators': [20, 200],}  
  #ValueError: Parameter values for parameter (MLPRegressor) need to be a sequence(but not a string) or np.ndarray.    
  modelHasRandomState = model in hasRandomState
  if model.startswith('Ridge'):
    modelHasRandomState=True

  if modelHasRandomState & ('random_state' not in _pg):
    _randomKeys=nbRdKeys
    if(abort):
      pass
    elif(model in nbRdKeysPerModel.keys()):
      _randomKeys=nbRdKeysPerModel[model]
    #print(i+' hasRandomState')   
    _pg['random_state']=range(1,_randomKeys);#always produce similar results, mais en permettant 3 shuffle, une fois les meilleurs paramètres retenus pour voting :):)    
    print(_pg['random_state'])
  #signature=k2+str(hash(json.dumps(_pg)))  

  print('\n'+model+suffix+' :: param grid :: ',end='');print(_pg)
  
  if(False):# & (signature in repeatedGridSearchCVSameResults)
    print('grid_search allready exists')
    grid_search=repeatedGridSearchCVSameResults[signature]
  else:
    #print(signature);repeatedGridSearchCVSameResults[signature]=
    grid_search=sklearn.model_selection.GridSearchCV(models[model], _pg, cv=my_cv, n_jobs = -1, scoring=scoring, refit=True, verbose=1, return_train_score=True)  #
  
  a=time();
  #when model is a classifier .. fit Transform the Results !
  if(k=='LogisticRegression'):
    x3=sklearn.preprocessing.LabelEncoder().fit_transform(x2)
    grid_search.fit(x1,x3)
  else:
    #here ..
    grid_search.fit(x1,x2)
  #now its okay   
  if modelHasRandomState: 
    del _pg['random_state']
  #Report Best Parameters MSE for the Grid Search !!!
  #sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
  #CV = KFold(n_splits=5, random_state=None, shuffle=False)
  #cv_results_  
  cvResults[k2]=grid_search.cv_results_
  bestParameters[k2]=grid_search.best_params_
  if 'random_state' in grid_search.best_params_:
    rds=grid_search.best_params_['random_state']
    if rds not in bestRandomStateValues.keys():
      bestRandomStateValues[rds]=0
    bestRandomStateValues[rds]+=1

  if(multiplyPredicitions):
    print('\n\t\tpredictions multiplied by : '+multiplyPredicitions)
    pred[k2]=predictions=grid_search.predict(y1)*y1[multiplyPredicitions]
  else:
    pred[k2]=predictions=grid_search.predict(y1)

  predMean=predictions.mean()#0 = bof bof guassian  
  #KneighValueError: Expected 2D array, got 1D array instead:array=[ 36  72 134 ...  38 254  25].Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
  #score=grid_search.score(y2,predictions)#scoring selected#is not a serie ...  
  #print(ev)print(ev.keys())
  #assert(False)
  if True:
    bs=grid_search.best_score_
    if(matchAgainst):
      print('\t\tmatchedAgainst:'+matchAgainst)
      results=labelEncoded[matchAgainst].fillna(0).astype('float')    
      if dfn+'-'+matchAgainst in splittedData.keys():
        print('Keeping previous splitted data')
        _x1, _y1, _x2, y2, mean = splittedData[dfn+'-'+matchAgainst]
      else:
        print('Generating new split')
        _x1, _y1, _x2, y2 = ShuffleOrNot(dfx,results)#Stays the same !
    #r2=sklearn.metrics.r2_score(Kbtu_y2,predictions)#matchAgainst labenc='Electricity(kBtu)'   

    r2s[k1][model+suffix]=ev[k2]['r2']=r2=sklearn.metrics.r2_score(y2,predictions)   #.predict(y1) 
    if abort:
      print('Real rd:'+str(round(r2,2))+',bs:'+str(bs));
      return grid_search;

    nmse=sklearn.metrics.mean_squared_error(y2,predictions)
    sqrt=ev[k2]['sqrt']=round(nmse ** (1/2))#=math.sqrt(nmse) mean_squared_log_error
    ev[k2]['bs']=nmse
    ev[k2]['nmse']=nmse    
    ev[k2]['mean']=round(mean,2)
    ev[k2]['PredMean']=round(predMean,2)
    ev[k2]['time']=round(time()-a,1)
    ev[k2]['explained_variance_score']=sklearn.metrics.explained_variance_score(y2,predictions)    
    per=ev[k2]['per']=round(sqrt/mean,5)

  sep='+';
  if(r2<0):
    sep='';

  #bestparams[toGuess+k]=grid_search.best_params_
  #AttributeError: 'KNeighborsClassifier' object has no attribute 'feature_importances_'
  #grid_search.best_estimator_.feature_importances_
  if hasattr(grid_search,'best_estimator_'):#IS VOTING !
    if hasattr(grid_search.best_estimator_, 'feature_importances_'):
      fi[k2]=grid_search.best_estimator_.feature_importances_
      plot_feature_importances(grid_search.best_estimator_.feature_importances_,dfn+'-'+energy+'-'+model, x1.columns.values,fn='FeatImportance-'+filename(k2)+str(round(r2,2))+'.png')

  elif hasattr(grid_search,'feature_importances_'):      
    fi[k2]=grid_search.feature_importances_
    plot_feature_importances(grid_search.feature_importances_,dfn+'-'+energy+'-'+model, x1.columns.values,fn='FeatImportance-'+filename(k2)+str(round(r2,2))+'.png')

  plt.rcParams["figure.figsize"] = (24,12)
  #plt.rcParams["figure.figsize"] = (6,6);plt.scatter(y2,predictions,alpha=0.2)
  y_mean = [mean]*len(y2)
  _df=pd.DataFrame(list(zip(y2,predictions,y_mean)),columns=['Real','Guess','Mean'])

  #fig,ax = plt.subplots()
  fn=filename(k2)
  if(len(fn)>100):
    fn=fn[0:50]
  fn='sp-'+fn+'.r2:'+sep+str(round(r2,2))+'.p:'+str(round(per,2))+'.png'
  f=_df[['Real','Guess','Mean']].plot(alpha=0.4,style=['o','r^','g-'],title='Données Réelles vs prédictions '+dfn+'-'+energy+'-'+model+suffix+' r2:'+str(round(r2,2))+','+str(round(per,2))+'%');
  #f.getAxes;
  #f.plot(y2.index,y_mean, label='Mean', linestyle='--',color=[0,0.5,0,1],alpha=1)
  plt.yscale('log');f.get_figure().savefig(fn, bbox_inches = 'tight',dpi=dpi);webp(fn);#show();

  print('\n\t',end='');print(ev[k2])
  if _ignoreParameters:
    if r2>=previousR2:
      bestNP+=[model]
      print('\n\n\t\t==> Best results without parameters')

  previousR2=r2
  return grid_search;  
p(date())

2020-11-10 16:55:12.899721


---
# Cross-Validation & random_state
---
- Stake of cross-validation would be to return same determination coefficients "<b>R²</b>"  between two runs :: always pass a fixed random_state: 42 to grid_params in the function which will pass to grid_search_cv, however this can protect us from better results .. Meanwhile .. running several runs with various keys ( <u>please store them</u> ) might get you better results, not based on the grid, but based on the starting point randomness wich might lead you to a best "local" function loss
---
- Just imagine this map, wider, with more hole, across several dimensions, and you'll get your main picture, but somehow, just don't waste that much time with randomness, cause you'll train an overspecialized model .. and that's not the purpose here ..
<img src='https://snipboard.io/QpSUvP.jpg'/>
---
- In order to be able to compare the predictions, it is necessary to operate each separation of the data set in training, test, in a non-random manner, in order to be able to reproduce the results
- <u> Random_state </u> The same goes for the results of certain regression models, the cross validation taking into account a hyper-parameter matrix must in no case return a random state, which would distort all interpretation
- We cannot select a "best" model based on the scoring of the latter with respect to the data to be predicted, since we are not supposed to have this information, this data may vary in the future
- The goal of cross-validation is to lessen "over-fitting" a model so that it reacts better in a global way to the presentation of new input data

---
## Load data
---

In [ ]:
ftpls();#GG('ftplist')#list files from ftp, so they're available for downloading ..
getFile('2015-building-energy-benchmarking.csv')#gets the zip & unzip if exists
getFile('2016-building-energy-benchmarking.csv')

df2015=loadData('2015-building-energy-benchmarking.csv')#Total Rows in dataset :156980,Rows containing null values :106880
df2016=loadData('2016-building-energy-benchmarking.csv')#Total Rows in dataset :155296,Rows containing null values :87776

indf2015Only=set(df2015.columns) - set(df2016.columns)#{'2010 Census Tracts','City Council Districts','Comment','GHGEmissions(MetricTonsCO2e)','GHGEmissionsIntensity(kgCO2e/ft2)','Location','OtherFuelUse(kBtu)','SPD Beats','Seattle Police Department Micro Community Policing Plan Areas','Zip Codes'}
indf2016Only=set(df2016.columns) - set(df2015.columns)#{'Address','City','Comments','GHGEmissionsIntensity','Latitude','Longitude','State','TotalGHGEmissions','ZipCode'}
#df2015['Year']=2015;df2016['Year']=2016;#DataYear exists
print('nbcol 2015:'+str(len(df2015.columns))+',nbrows:'+str(df2015.shape[0]))#47
print('nbcol 2016:'+str(df2016.shape[1])+',nbrows:'+str(df2016.shape[0]))#46

print('Common columns within both files :'+str(len(frozenset(df2015.columns).intersection(df2016.columns))))#37 en commun
print('Data in 2015 only:\n\t\t'+','.join(indf2015Only))
print('Data in 2016 only:\n\t\t'+','.join(indf2016Only))
p(date())

2020-11-09 07:06:41.243656


#.-------------------------------------------------------------

---
# II) Different dataframes
- Some of these columns contain the same data, but do not have the same names ..
- {'2010 Census Tracts','City Council Districts','Comment','GHGEmissions(MetricTonsCO2e)','GHGEmissionsIntensity(kgCO2e/ft2)','Location','OtherFuelUse(kBtu)','SPD Beats','Seattle Police Department Micro Community Policing Plan Areas','Zip Codes'} vs {'Address','City','Comments','GHGEmissionsIntensity','Latitude','Longitude','State','TotalGHGEmissions','ZipCode'}
----

In [ ]:
print(',\t'.join(df2015.columns))
print(',\t'.join(df2016.columns))
print(df2015['Location'][:2][0])
print(df2016['Address'][:2])
p(date())
#df2016['TotalGHGEmissions']=df2015['GHGEmissions(MetricTonsCO2e)']#?

OSEBuildingID,	DataYear,	BuildingType,	PrimaryPropertyType,	PropertyName,	TaxParcelIdentificationNumber,	Location,	CouncilDistrictCode,	Neighborhood,	YearBuilt,	NumberofBuildings,	NumberofFloors,	PropertyGFATotal,	PropertyGFAParking,	PropertyGFABuilding(s),	ListOfAllPropertyUseTypes,	LargestPropertyUseType,	LargestPropertyUseTypeGFA,	SecondLargestPropertyUseType,	SecondLargestPropertyUseTypeGFA,	ThirdLargestPropertyUseType,	ThirdLargestPropertyUseTypeGFA,	YearsENERGYSTARCertified,	ENERGYSTARScore,	SiteEUI(kBtu/sf),	SiteEUIWN(kBtu/sf),	SourceEUI(kBtu/sf),	SourceEUIWN(kBtu/sf),	SiteEnergyUse(kBtu),	SiteEnergyUseWN(kBtu),	SteamUse(kBtu),	Electricity(kWh),	Electricity(kBtu),	NaturalGas(therms),	NaturalGas(kBtu),	OtherFuelUse(kBtu),	GHGEmissions(MetricTonsCO2e),	GHGEmissionsIntensity(kgCO2e/ft2),	DefaultData,	Comment,	ComplianceStatus,	Outlier,	2010 Census Tracts,	Seattle Police Department Micro Community Policing Plan Areas,	City Council Districts,	SPD Beats,	Zip Codes
OSEBuildingID,	DataY

---
## Location df2015 -> json{Lat,Lon}
- Location est un champ Json réparti sur les nouvelles colonnes du fichier 2016 on constate rapidement que ces données concernent les mêmes batiments aux mêmes adresses et c'est mieux
- GHGEmissionsIntensity<=GHGEmissionsIntensity(kgCO2e/ft2)
---

In [ ]:
### Le champ location a l'air d'être un json qui mérite d'être réparti entre plusieurs colonnes afin d'avoir de belles corrélations
results = pd.DataFrame()
for idx, row in df2015.iterrows():
    loc = ast.literal_eval(row['Location'])
    row = row.drop('Location')
    lat = loc['latitude']
    lon = loc['longitude']
    need_recoding = 0
    if('needs_recoding' in loc.keys()):
        need_recoding = loc['needs_recoding'] 
#'{\'latitude\': \'47.61219025\', \'human_address\': \'{"address": "405 OLIVE WAY", "city": "SEATTLE", "state": "WA", "zip": "98101"}\', \'longitude\': \'-122.33799744\'}'        
    normalize = pd.Series(json.loads(loc['human_address']))
    
    cols = list(row.index) + ['latitude', 'longitude', 'need_recoding'] + list(normalize.index)
    x = pd.DataFrame([list(row) + [lat, lon, need_recoding] + list(normalize)], columns = cols )
    results = results.append(x).reset_index(drop=True)

results['longitude']=results['longitude'].astype(float)
results['latitude']=results['latitude'].astype(float)
df2015=results
del results
p(date())

2020-11-09 07:09:56.238169


---
## Harmonisation colonnes
- Harmonizons les noms de colonnes entre les deux datasets
---



In [ ]:
df2016['address']=df2016['Address']
df2016['city']=df2016['City']
df2016['state']=df2016['State']
df2016['zip']=df2016['ZipCode']#Zip Codes
df2016['latitude']=df2016['Latitude']
df2016['longitude']=df2016['Longitude']
df2016['Comment']=df2016['Comments']
df2016['GHGEmissionsIntensity(kgCO2e/ft2)']=df2016['GHGEmissionsIntensity']### ou multiplier celle ci par superficie et tester si résultat pertinent
df2016['GHGEmissions(MetricTonsCO2e)']=df2016['TotalGHGEmissions']#### this one
df2016=df2016.drop(['Address','City','State','ZipCode','Latitude','Longitude','Comments','GHGEmissionsIntensity','TotalGHGEmissions'],axis=1)
p(date())

2020-11-09 07:10:14.363873


---
## OK ?
- Reste t-il des colonnes uniques à 2016 ??? Si oui l'execution s'interrompt !
---

In [ ]:
indf2016Only=set(df2016.columns) - set(df2015.columns)#
if(len(indf2016Only)>0):
  print('Données présentes dans 2016 uniquement:\n\t\t'+','.join(indf2016Only))#
  assert(False)
p(date())

2020-11-09 07:10:55.516046


---
### Merge & Clean
- Valeurs non remplies : potentiellement à supprimer, notamment pour les relevés à prédire : 
- OtherFuelUse(kBtu) => empty 6699 99.75%
---
- SteamUse(kBtu) => empty : 6456 96.13%
- ENERGYSTARScore => empty : 1623 24.17%
---

In [ ]:
def cleanData(inputDf,fillStrings='na',fillInt=0,considerEmpty=[np.inf,-np.inf,np.nan,0,'na','']): 
  #test = pd.read_csv('../input/test.csv')
  df=inputDf.copy(deep=True)
  rows=df.shape[0]
  cols=df.shape[1]  
  dfs=df.size;

  nanInfZeroNaEmpty=train.isin(considerEmpty).sum().sort_values(ascending=False)  
  nv=nanInfZeroNaEmpty.sum()

  print('Total Rows in dataset : '+str(rows));
  print('Total Cols in dataset : '+str(cols));
  print('Total Cells : '+str(dfs))
  print('Cells containing null,inf,NaN,0 or empty values : '+str(nv)+' ( '+str(round(nv*100/dfs,2))+'% )');#Diagnose null columns
  print('_'*80)  
  # Fournissent une bonne indication des colonnes à dropper pour le modèke
  for i in nanInfZeroNaEmpty.index:
    nbempty=nanInfZeroNaEmpty[i]
    per=round(nbempty*100/rows,2);
    print(i+' => empty : '+str(nbempty)+' '+str(per)+'%')

  print('_'*140)
  
  for i in df.columns:
    if df.dtypes[i]==object:
      df[i].fillna(fillStrings,inplace=True)#as strings ! boljemoi !
    else:
      df.replace([np.inf,-np.inf,np.nan],fillInt,inplace=True)#replace infinite by Nan
  #df.fillna(df.mean(),inplace=True)#Numeric types corrected
  #df.fillna(fillInt,inplace=True)#2 rows of Nan -> is valid either for strings or numeric datatype
  #dtype as str please !!
  
  print('_'*140)
  print('Cells with null values then : '+str(nullValues(df).size))#CAUTION !!! WONT FIT INTO MODEL OTHERWISE
  print('_'*140)
  return df;

train=df2015.append(df2016)
train['zip']=train['zip'].fillna(0).astype('int')
train=cleanData(train)
deli('cleaned,df2016,df2015')
print(','.join(train.columns))

Total Rows in dataset : 6716
Total Cols in dataset : 53
Total Cells : 355948
Cells containing null,inf,NaN,0 or empty values : 97464 ( 27.38% )
________________________________________________________________________________
need_recoding => empty : 6716 100.0%
Comment => empty : 6703 99.81%
OtherFuelUse(kBtu) => empty : 6699 99.75%
Outlier => empty : 6600 98.27%
City Council Districts => empty : 6503 96.83%
2010 Census Tracts => empty : 6492 96.66%
YearsENERGYSTARCertified => empty : 6487 96.59%
SteamUse(kBtu) => empty : 6456 96.13%
ThirdLargestPropertyUseTypeGFA => empty : 5652 84.16%
ThirdLargestPropertyUseType => empty : 5560 82.79%
PropertyGFAParking => empty : 5288 78.74%
SecondLargestPropertyUseTypeGFA => empty : 3715 55.32%
SecondLargestPropertyUseType => empty : 3478 51.79%
SPD Beats => empty : 3378 50.3%
Seattle Police Department Micro Community Policing Plan Areas => empty : 3378 50.3%
Zip Codes => empty : 3376 50.27%
DefaultData => empty : 3264 48.6%
NaturalGas(kBtu) => emp

---
## Describe
- On ne retient que les valeurs uniques par type de colonne: string, on constate rapidement que : location et propertyname ont bcp de valeurs uniques qu'il sera approprié d'abandonner avant de les insérer dans le modèle ( les autres généront des colonnes booléenne pour chaucune de leurs valeurs respectives si la corrélation est suffisante pour ces dernières )
- ListOfAllPropertyUseTypes: contient tous les usages d'un batiment avec des valeurs séparées par des virgules
---

In [ ]:
p(','.join(train.columns))
train.describe()

OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),ListOfAllPropertyUseTypes,LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,YearsENERGYSTARCertified,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),DefaultData,Comment,ComplianceStatus,Outlier,2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes,latitude,longitude,need_recoding,address,city,state,zip


,OSEBuildingID,DataYear,CouncilDistrictCode,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseTypeGFA,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),OtherFuelUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),2010 Census Tracts,Seattle Police Department Micro Community Policing Plan Areas,City Council Districts,SPD Beats,Zip Codes,latitude,longitude,need_recoding,zip
count,6716.000000,6716.00000,6716.000000,6716.000000,6716.000000,6716.000000,6.716000e+03,6716.000000,6.716000e+03,6.716000e+03,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6.716000e+03,6.716000e+03,6.716000e+03,6.716000e+03,6.716000e+03,6.716000e+03,6.716000e+03,6.716000e+03,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6716.000000,6716.0,6716.000000
mean,20867.420488,2015.50268,4.445057,1968.101549,1.070131,4.690143,9.281410e+04,9519.597975,8.329450e+04,7.528895e+04,13535.428275,1938.571679,51.495980,54.045191,57.171992,133.132594,137.965381,5.183071e+06,5.227626e+06,2.617650e+05,1.049481e+06,3.580899e+06,1.307513e+04,1.307513e+06,3.541373e+03,114.610478,1.078096,4.104527,16.093657,0.042734,12.338892,9342.988684,47.624062,-122.334904,0.0,97883.092912
std,11878.968174,0.50003,2.123749,33.020391,1.592984,5.475352,1.875062e+05,33868.050881,1.733470e+05,1.666020e+05,39689.257069,11702.088239,37.396423,55.099179,56.392690,135.670587,135.310681,1.811787e+07,1.499324e+07,3.699003e+06,3.655015e+06,1.247106e+07,5.678004e+04,5.678004e+06,1.382460e+05,478.237575,1.733259,22.123759,21.426485,0.250906,16.309583,9403.069492,0.047837,0.027185,0.0,4783.718163
min,1.000000,2015.00000,1.000000,1900.000000,0.000000,0.000000,1.128500e+04,-3.000000,-5.055000e+04,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.100000,0.000000e+00,0.000000e+00,0.000000e+00,-3.382680e+04,-1.154170e+05,0.000000e+00,0.000000e+00,0.000000e+00,-0.800000,-0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,47.499170,-122.414250,0.0,0.000000
25%,19934.000000,2015.00000,3.000000,1948.000000,1.000000,2.000000,2.848700e+04,0.000000,2.727300e+04,2.448000e+04,0.000000,0.000000,1.000000,27.500000,29.500000,73.900000,79.074999,9.175949e+05,9.788606e+05,0.000000e+00,1.861307e+05,6.350782e+05,0.000000e+00,0.000000e+00,0.000000e+00,9.290000,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,47.599857,-122.350603,0.0,98105.000000
50%,23064.000000,2016.00000,4.000000,1974.000000,1.000000,4.000000,4.400800e+04,0.000000,4.218500e+04,3.829700e+04,0.000000,0.000000,62.000000,37.900002,40.800000,95.500000,101.900000,1.786446e+06,1.922098e+06,0.000000e+00,3.409549e+05,1.163338e+06,3.004665e+03,3.004665e+05,0.000000e+00,33.040000,0.530000,0.000000,0.000000,0.000000,0.000000,0.000000,47.618608,-122.332630,0.0,98115.000000
75%,25868.250000,2016.00000,7.000000,1996.000000,1.000000,5.000000,8.943125e+04,0.000000,8.112000e+04,7.200000e+04,10206.000000,0.000000,86.000000,60.099998,64.900000,143.800003,149.425000,4.120209e+06,4.369269e+06,0.000000e+00,8.155168e+05,2.782606e+06,1.143626e+04,1.143626e+06,0.000000e+00,91.167500,1.270000,0.000000,32.250000,0.000000,26.000000,18390.000000,47.657135,-122.319733,0.0,98122.000000
max,50226.000000,2016.00000,7.000000,2015.000000,111.000000,99.000000,9.320156e+06,512608.000000,9.320156e+06,9.320156e+06,686750.000000,459748.000000,100.000000,834.400024,834.400024,2620.000000,2620.000000,8.739237e+08,4.716139e+08,1.349435e+08,1.925775e+08,6.570744e+08,2.979090e+06,2.979090e+08,8.269669e+06,16870.980000,34.090000,135.000000,61.000000,2.000000,51.000000,19584.000000,47.733870,-122.220470,0.0,98272.000000


---
### Unique Values ? 
- Nombreuses valeurs distinctes ( pour les strings ) => Risque de sur-apprentissage !
- TaxParcelIdentificationNumber;PropertyName;Comment;YearsENERGYSTARCertified;address;city;state
---

In [ ]:
unikValuesPerDataframe(train)

________________________________________________________________________________________________________________________
 🗲 Different values in OSEBuildingID : 3432 (mean:20867.420488385946)
OSEBuildingID
23109    2
20928    2
21097    2
21096    2
21095    2
Name: OSEBuildingID, dtype: int64
________________________________________________________________________________________________________________________
 🗲 Different values in DataYear : 2 (mean:2015.502680166766)
DataYear
2016    3376
2015    3340
Name: DataYear, dtype: int64
________________________________________________________________________________________________________________________
 🗲 Different values in BuildingType : 8
BuildingType
nonresidential          2921
multifamily lr (1-4)    2047
multifamily mr (5-9)    1134
multifamily hr (10+)     217
sps-district k-12        197
Name: BuildingType, dtype: int64
____________________________________________________________________________________________________________

---
## >Latlon
- Arrondir à 3 décimales: 100 mètres près et créer une clé composite, cela peut - il servir à regrouper, effectuer des corrélations géographiques ?
---

In [13]:
train['shortlat']=train['latitude'].round(2)#3:110mètre près,2: 1 km
train['shortlon']=train['longitude'].round(2)
train['shortlatlon']=train['shortlat'].astype('str')+','+train['shortlon'].astype('str')

plt.rcParams["figure.figsize"] = (24,24)
print('Here is the form Elliott Bay of the left')#,title='EliottBay'
scatter(train,'longitude','latitude',fn='eliottBayLatLonPrint.png')#
plt.rcParams["figure.figsize"] = (22,6)
plt.scatter(train.index,train['latitude']);plt.title('latitude');plt.savefig('lat.png', bbox_inches = 'tight',dpi=dpi);webp('lat.png');#show();
plt.scatter(train.index,train['longitude']);plt.title('longitude');plt.savefig('lon.png', bbox_inches = 'tight',dpi=dpi);webp('lon.png');#show();
#Needs X Avis to be numeric
#train.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4)
#https://www.bigendiandata.com/2017-06-27-Mapping_in_Jupyter/
#save2()#all variables as one zip to be restored via function :: load2()
#save(globals(),'df2015,df2016,train'.split(','))#globals().update(resume('shortlatlon')),df,results,x1,y1,correlations,x2,y2,row,heatmap,topCorrelations

Here is the form Elliott Bay of the left
put : http://1.x24.fr/a/jupyter/seattle3//eliottBayLatLonPrint.webp?a=20201111-082457
put : http://1.x24.fr/a/jupyter/seattle3//lat.webp?a=20201111-082500
put : http://1.x24.fr/a/jupyter/seattle3//lon.webp?a=20201111-082504


---
### ֍ Interesting Metrics > KDE
- > Suppress : OtherFuelUse(kBtu) : filled at 0.25%
- http://1.x24.fr/a/jupyter/seattle/kde2.webp
---

In [15]:
interestingMetrics='ENERGYSTARScore,Electricity(kBtu),GHGEmissions(MetricTonsCO2e),SiteEUIWN(kBtu/sf),SourceEUIWN(kBtu/sf),shortlat,shortlon,SiteEnergyUse(kBtu),GHGEmissionsIntensity(kgCO2e/ft2),OtherFuelUse(kBtu),NaturalGas(kBtu),SteamUse(kBtu)'.split(',')
rows=train.shape[0]
completion={}
for i in interestingMetrics:
  completion[i]=round(train[train[i]!=0].shape[0]*100/rows,2)

print(asort(completion))
interestingMetrics.remove('OtherFuelUse(kBtu)')#0.25%

lowQuantileLimit=0.01
highQuantileLimit=0.99
#shortlatlon is not numeric ! train['shortlatlon'] 
plt.rcParams["figure.figsize"] = (24,36)
heightPerGraph=widthPerGraph=10
j=0;axx=2;#Per Col
axy=math.ceil(len(interestingMetrics)/axx);#tjrs 1 de plus
print(str(axx*axy)+' tot slots for '+str(len(interestingMetrics))+' to display : ok')#=) provisionner plus d'emplacements n'est pas grave
f,axes=plt.subplots(axy,axx,figsize=(axy*heightPerGraph,axx*widthPerGraph))
f.patch.set_facecolor('white');axes=axes.flatten()#pour les parcourir plus facilement sur une seule dimension :)

for i in interestingMetrics:  
  #print(i)
  x=train[train[i]!=0][i]
  if x.shape[0]==0:
    continue;
  ax=axes[j]    
  minx=train[i].quantile(lowQuantileLimit);
  maxx=train[i].quantile(highQuantileLimit);
  ax.set_xlim(minx,maxx)#Avec les limites adaptées à chaque type de données  
  x.plot(kind='kde',ax=ax,title=i);
  j=j+1;#prochain axe

fn='kde2.png';f.savefig(fn, bbox_inches = 'tight',dpi=dpi);webp(fn);#show()

[('OtherFuelUse(kBtu)', 0.25), ('SteamUse(kBtu)', 3.87), ('NaturalGas(kBtu)', 62.28), ('ENERGYSTARScore', 75.83), ('SourceEUIWN(kBtu/sf)', 99.0), ('SiteEUIWN(kBtu/sf)', 99.15), ('GHGEmissionsIntensity(kgCO2e/ft2)', 99.42), ('Electricity(kBtu)', 99.48), ('SiteEnergyUse(kBtu)', 99.48), ('GHGEmissions(MetricTonsCO2e)', 99.55), ('shortlat', 100.0), ('shortlon', 100.0)]
12 tot slots for 11 to display : ok
put : http://1.x24.fr/a/jupyter/seattle3//kde2.webp?a=20201111-082542


---
## Feature Engineering : Electric Consumption per GFA ?
- Certaines colonnes numériques sont des totaux : metrictons, Kwh
- D'autres sont reportées / sqft => Il est intéressant de reporter certaines métriques absolues à la surface du batiment afin de matcher d'autres variables
---
- Utile pour rapporter les métriques totales à une surface afin de mieux corréler certaines variables
=> Kbtu => Kbtu/sqft
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-ghgemissions-mt-gfa-ghgemissionsintensity-kgco2e-ft2--1.0.webp#Validation du POC
- http://1.x24.fr/a/jupyter/seattle/correlationsInterestingMetricsAndComposite.webp Meilleures corrélations pour Electricité etc ..
---

In [ ]:
#train[['PropertyGFATotal','PropertyGFABuilding(s)']][:10]#
train['Electricity-KBtu/Gfa']=train['Electricity(kBtu)']/train['PropertyGFATotal']
train['SiteEnergyUseWN-KBtu/Gfa']=train['SiteEnergyUseWN(kBtu)']/train['PropertyGFATotal']
train['Steam-KBtu/Gfa']=train['SteamUse(kBtu)']/train['PropertyGFATotal']
train['GHGEmissions-Mt/Gfa']=train['GHGEmissions(MetricTonsCO2e)']/train['PropertyGFATotal']
train['NaturalGas-KBtu/Gfa']=train['NaturalGas(kBtu)']/train['PropertyGFATotal']
#train['OtherFuelUse-kBtu/Gfa']=train['OtherFuelUse(kBtu)']/train['PropertyGFATotal']##,OtherFuelUse-kBtu/Gfa
#on les ajoute naturellement
newKeys='Electricity-KBtu/Gfa,SiteEnergyUseWN-KBtu/Gfa,Steam-KBtu/Gfa,GHGEmissions-Mt/Gfa,NaturalGas-KBtu/Gfa'.split(',')

---
### ! Dummies for Properties
- Obtenir des valeurs uniques pour chaque délimitation par virgule

In [ ]:
#del train;resumed=0;globals().update(resume('shortlatlon'))
dummies={}
results = pd.DataFrame()
colName='ListOfAllPropertyUseTypes';
train[colName].fillna('',inplace=True)#
train[colName]=train[colName].astype('str')

for idx, row in train.iterrows():
  x=re.sub(r"[^a-z0-9, \.\-_:]+",'-', row[colName].lower());
  #row=row.drop(colName)#does nothing ?
  if(type(x)==int):
    continue;

  r=x.split(',')
  nc=list(map(str.strip,r));#strip spaces
  nc=['z_'+colName+'_'+format(j) for j in nc]
  newData=[];
#  for i in range(len(nc)):
#    newData=newData+[1];
  for i in nc:    
    if i not in dummies:
      dummies[i]=0;
    dummies[i]+=1;
    newData=newData+[1];

  data = list(row) + newData;#Données anciennes + nouvelles
  cols = list(row.index) + nc;#len(cols)#53
  ndf = pd.DataFrame([data], columns=cols)
#AssertionError: Number of manager items must equal union of block items
# manager items: 607, # tot_items: 608  
  results = results.append(ndf).reset_index(drop=True)
    
results.fillna(0,inplace=True)#replace na values foreach new column
FPCP('train', train)
train=results;del results;#replace dataframe with the new one
FPCP('results', train)
#train[:1]
p('TopDummies',arsort(dummies)[:5]);

if False:
  newColumns=[]
  for i in train['ListOfAllPropertyUseTypes'].unique():    
      r=i.replace("'",'').split(',')
      r=list(map(str.strip,r));
      newColumns=list(set(newColumns+r));
  #print(Counter(newColumns))
  len(newColumns)#70 out of 443 unique concatened values
  #print();break;
dummiesk=list(dummies.keys());  
p(','.join(dummiesk))
save(globals(),'df,results,df2015,df2016,x1,y1,correlations,x2,y2,row,heatmap,topCorrelations'.split(','),'dummies')#globals().update(resume('dummies'))#dummies checkpoiunt

z_ListOfAllPropertyUseTypes_hotel,z_ListOfAllPropertyUseTypes_parking,z_ListOfAllPropertyUseTypes_restaurant,z_ListOfAllPropertyUseTypes_swimming pool,z_ListOfAllPropertyUseTypes_police station,z_ListOfAllPropertyUseTypes_other - entertainment-public assembly,z_ListOfAllPropertyUseTypes_multifamily housing,z_ListOfAllPropertyUseTypes_,z_ListOfAllPropertyUseTypes_data center,z_ListOfAllPropertyUseTypes_library,z_ListOfAllPropertyUseTypes_fitness center-health club-gym,z_ListOfAllPropertyUseTypes_office,z_ListOfAllPropertyUseTypes_other,z_ListOfAllPropertyUseTypes_other - lodging-residential,z_ListOfAllPropertyUseTypes_social-meeting hall,z_ListOfAllPropertyUseTypes_courthouse,z_ListOfAllPropertyUseTypes_k-12 school,z_ListOfAllPropertyUseTypes_college-university,z_ListOfAllPropertyUseTypes_automobile dealership,z_ListOfAllPropertyUseTypes_retail store,z_ListOfAllPropertyUseTypes_self-storage facility,z_ListOfAllPropertyUseTypes_non-refrigerated warehouse,z_ListOfAllPropertyUseTypes_other

---
## Drop Btu/Kw/Therms Unit Duplicates
- Certaines variables sont en double ( en comportant des unités différentes qui vons forcément corréler à 100%

In [ ]:
dropped=[];#Conserver que Kbtu ( majeure partie du dataframe )
#Retirer les corrélations "évidentes" crées entre des champs ayant peu de valeurs ou bcp trop en commun    
toDrop='DataYear;OSEBuildingID;TaxParcelIdentificationNumber;PropertyName;Comment;YearsENERGYSTARCertified;address;city;state'.split(';')
for i in train.columns:
  if('therms' in  i or 'kWh' in i or i in toDrop):    
    dropped+=[i]

train=train.drop(dropped,axis=1)
save(globals(),'df,results,df2015,df2016,x1,y1,correlations,x2,y2,row,heatmap'.split(','),'dropedColumns')#globals().update(resume('dropedColumns'))
p('Dropped Columns :')
p(dropped)
#RFM,R,Frequency,avgCartPrice,avgItemPrice

[('train', 10529278), ('installed', 6873), ('dummies', 2280), ('ndf', 1659), ('In', 768), ('dummiesk', 768), ('Out', 648), ('completion', 648), ('cols', 592), ('data', 592), ('normalize', 529), ('nbRdKeysPerModel', 240), ('votedm', 240), ('res', 240), ('pred', 240), ('acy', 240), ('nfm', 240), ('fi', 240), ('ev', 240), ('bestparams', 240), ('bss', 240), ('repeatedGridSearchCVSameResults', 240), ('bestRandomStateValues', 240), ('bestRandom', 240), ('loc', 240), ('dropped', 192), ('requiredModules', 160), ('energies', 160), ('toPredictBase', 160), ('toPredict', 160), ('interestingMetrics', 160), ('newKeys', 160), ('r', 160), ('toDrop', 160), ('npRandomKeys', 144), ('randomKeys', 144), ('x', 141), ('exclusions', 136), ('nc', 128), ('considerEmpty', 112), ('newData', 104), ('i', 97), ('colName', 74), ('energy', 66), ('threads', 64), ('bestNP', 64), ('lon', 61), ('lat', 60), ('module', 59), ('fn', 57), ('lastVotingKey', 53), ('scoring', 51), ('heightPerGraph', 28), ('widthPerGraph', 28), ('

---
## Corrélations
- TOP Des corrélations, le changement d'unité de mesure est une évidence à exclure ( ne conserver qu'une unité de mesure pour chaque )
- Nb : les corrélations négatives sont des corrélations : par exemple plus la batiment a des habitats multifamilliaux et plus sa consommation électrique globale diminue, ici nous allons appliquer une valeur absolue afin de mieux pouvoir classifier ces dernières
---
- Quincailleries, cordonniers, certains commerces possèdent des virgules dans leurs descriptifs pour décrire ce qu'ils sont d'où ces corrélations .. + Features crées

In [19]:
heatmap=train.copy(deep=True);#   
plt.rcParams["figure.figsize"]=(heatmap.shape[1],heatmap.shape[1])#Massive Output to register to jpeg as much columns * 45px
for i in heatmap.columns.values:
  if 'numpy' not in str(type(heatmap[i][0])): 
    heatmap[i],levels=pd.factorize(heatmap[i])
correlations=heatmap.corr().abs()#Not Absolute We Want to know what might decrease
np.fill_diagonal(correlations.values, np.nan)#no more ones
#exclude z_ ?
topCorrelations=(correlations.where(np.triu(np.ones(correlations.shape),k=1).astype(np.bool)).stack().sort_values(ascending=False))    
pd.set_option('display.max_rows',900)
print(topCorrelations.head(180))

z_ListOfAllPropertyUseTypes_repair services -vehicle                    z_ListOfAllPropertyUseTypes_locksmith                                1.000000
z_ListOfAllPropertyUseTypes_shoe                                        z_ListOfAllPropertyUseTypes_locksmith                                1.000000
z_ListOfAllPropertyUseTypes_personal services -health-beauty            z_ListOfAllPropertyUseTypes_dry cleaning                             1.000000
z_ListOfAllPropertyUseTypes_repair services -vehicle                    z_ListOfAllPropertyUseTypes_shoe                                     1.000000
DefaultData                                                             Zip Codes                                                            0.998740
latitude                                                                shortlat                                                             0.998245
SourceEUI(kBtu/sf)                                                      SourceEUIWN(kBtu/sf)        

---
### Heatmaps
- On constate les évidentes corrélations entre certains types de commerces présents dans une tour, on regroupe les statistiques qui nous intéressent afin d'en faire un relevé Thermographique avec les autres variables
---

In [12]:
def heatmapx(x,y,fn=0):
  plt.rcParams["figure.figsize"]=(len(x),len(y))
  plt.figure(figsize=(len(x),len(y)));
  if fn==0:
    fn='correlations-'+filename(','.join(y))+'.png'
  sns.heatmap(correlations.loc[y][x],xticklabels=x,yticklabels=y,annot=True,cmap="YlGnBu").get_figure().savefig(fn, bbox_inches = 'tight',dpi=dpi);
  webp(fn);#show() 

c=correlations.shape[0]
im=interestingMetrics.copy()
im.sort()
x=interestingMetrics+newKeys
x.sort()
y=correlations.columns.values

heatmapx(im,im,'correlationsInteresting.png')
heatmapx(y,x,'correlationsGlobales.png')
heatmapx(x,x,'correlationsIFeatures.png')
print('### \t familles, data center, supermarché consomment bcp electricité - Meilleures corrélations avec /ft²')
heatmapx(interestingMetrics+dummiesk,['Electricity(kBtu)','Electricity-KBtu/Gfa'])
heatmapx(interestingMetrics+dummiesk,['SiteEnergyUse(kBtu)','SiteEUIWN(kBtu/sf)','SiteEnergyUseWN-KBtu/Gfa'])
heatmapx(interestingMetrics+dummiesk,['GHGEmissions(MetricTonsCO2e)','GHGEmissionsIntensity(kgCO2e/ft2)', 'GHGEmissions-Mt/Gfa'])
heatmapx(interestingMetrics+dummiesk,['NaturalGas(kBtu)', 'NaturalGas-KBtu/Gfa'])
heatmapx(interestingMetrics+dummiesk,['SteamUse(kBtu)','Steam-KBtu/Gfa'])

put : http://1.x24.fr/a/jupyter/seattle3//correlationsInteresting.webp?a=20201111-082352
put : http://1.x24.fr/a/jupyter/seattle3//correlationsGlobales.webp?a=20201111-082409
put : http://1.x24.fr/a/jupyter/seattle3//correlationsIFeatures.webp?a=20201111-082414
### 	 familles, data center, supermarché consomment bcp electricité - Meilleures corrélations avec /ft²
put : http://1.x24.fr/a/jupyter/seattle3//correlations-electricity-kbtu-,electricity-kbtu-gfa.webp?a=20201111-082421
put : http://1.x24.fr/a/jupyter/seattle3//correlations-siteenergyuse-kbtu-,siteeuiwn-kbtu-sf-,siteenergyusewn-kbtu-gfa.webp?a=20201111-082429
put : http://1.x24.fr/a/jupyter/seattle3//correlations-ghgemissions-metrictonsco2e-,ghgemissionsintensity-kgco2e-ft2-,ghgemissions-mt-gfa.webp?a=20201111-082437
put : http://1.x24.fr/a/jupyter/seattle3//correlations-naturalgas-kbtu-,naturalgas-kbtu-gfa.webp?a=20201111-082444
put : http://1.x24.fr/a/jupyter/seattle3//correlations-steamuse-kbtu-,steam-kbtu-gfa.webp?a=2020111

In [ ]:
heatmapx(interestingMetrics+dummiesk,['ENERGYSTARScore'])#http://1.x24.fr/a/jupyter/seattle/correlations-energystarscore.webp

---
### Conclusions : 
- http://1.x24.fr/a/jupyter/seattle/violinPerCatNoLog-electricity-kbtu-gfa-largestpropertyusetype.webp #Violins consommation electrique selon utilisation immeuble
- http://1.x24.fr/a/jupyter/seattle/correlationsGlobales.webp # pour les dummies également !
- http://1.x24.fr/a/jupyter/seattle/correlations-electricity-kbtu-electricity-kbtu-gfa.webp #familles, data center, supermarché consomment bcp electricité - Meilleures corrélations avec /ft²
- On peut supprimer les deux en double
----

In [ ]:
train=train.drop(['SiteEnergyUseWN-KBtu/Gfa','GHGEmissions-Mt/Gfa'],axis=1)
heatmap=heatmap.drop(['SiteEnergyUseWN-KBtu/Gfa','GHGEmissions-Mt/Gfa'],axis=1)

newKeys.remove('SiteEnergyUseWN-KBtu/Gfa')
newKeys.remove('GHGEmissions-Mt/Gfa')
p(x)
p(dummies.keys())
save(globals(),'df,results'.split(','),'0pairplots')#globals().update(resume('0pairplots'))

['ENERGYSTARScore', 'Electricity(kBtu)', 'Electricity-KBtu/Gfa', 'GHGEmissions(MetricTonsCO2e)', 'GHGEmissions-Mt/Gfa', 'GHGEmissionsIntensity(kgCO2e/ft2)', 'NaturalGas(kBtu)', 'NaturalGas-KBtu/Gfa', 'SiteEUIWN(kBtu/sf)', 'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN-KBtu/Gfa', 'SourceEUIWN(kBtu/sf)', 'Steam-KBtu/Gfa', 'SteamUse(kBtu)', 'shortlat', 'shortlon']
dict_keys(['z_ListOfAllPropertyUseTypes_hotel', 'z_ListOfAllPropertyUseTypes_parking', 'z_ListOfAllPropertyUseTypes_restaurant', 'z_ListOfAllPropertyUseTypes_swimming pool', 'z_ListOfAllPropertyUseTypes_police station', 'z_ListOfAllPropertyUseTypes_other - entertainment-public assembly', 'z_ListOfAllPropertyUseTypes_multifamily housing', 'z_ListOfAllPropertyUseTypes_', 'z_ListOfAllPropertyUseTypes_data center', 'z_ListOfAllPropertyUseTypes_library', 'z_ListOfAllPropertyUseTypes_fitness center-health club-gym', 'z_ListOfAllPropertyUseTypes_office', 'z_ListOfAllPropertyUseTypes_other', 'z_ListOfAllPropertyUseTypes_other - lodging-residen

---
#### 🗲 !!!!! Pairplots & Basic Régressions !
---

In [ ]:
heightPerUnit=69336/107/9;#72
le=len(interestingMetrics)-1#2 removed rows
#ne jamais excéder 65536 pixels par dimension ..
maxh=65536/72/le;
graphHeight=8
if(graphHeight>maxh):
  graphHeight=int(maxh)
#ͶѠ֍
x=interestingMetrics.copy()
x+=newKeys
x.sort()
pairs={}

for i in x:  
  cols=interestingMetrics.copy()
  cols+=newKeys
  cols.remove(i)
  cols.sort()
  g=sns.pairplot(heatmap,kind='scatter',height=graphHeight,aspect=1,x_vars=cols,y_vars=[i],plot_kws={'alpha':0.5},markers="+");
  fn='pairplots-'+filename(i)+'.png';
  g.fig.savefig(fn, bbox_inches = 'tight',dpi=dpi);
  webp(fn);#show();  
  #assert(false);

  pairs[i]=[]

  for j in cols:
    if(j in pairs.keys() & i in pairs[j]):
      print('-');
      continue
    pairs[i]+=[j]
    sep='+'
    corrScore=round(correlations[i][j],2);
    if(corrScore<0):
      sep='';
    fn='scatRegressionHeatmap-'+filename(i+'-'+j)+sep+str(corrScore)+'.png'
    snsscat(heatmap,i,j,opacity=0.5,color=[1,0,0.3,0.5],kind='reg',fn=fn,minx=heatmap[i].min(),maxx=heatmap[i].max(),miny=heatmap[j].min(),maxy=heatmap[j].max())#Reg
  #Et des régressions / KDE pour chaque facteur ??

put : http://1.x24.fr/a/jupyter/seattle3//pairplots-energystarscore.webp?a=20201111-082704
put : http://1.x24.fr/a/jupyter/seattle3//scatRegressionHeatmap-energystarscore-electricity-kbtu-+0.04.webp?a=20201111-082707
put : http://1.x24.fr/a/jupyter/seattle3//scatRegressionHeatmap-energystarscore-electricity-kbtu-gfa+0.25.webp?a=20201111-082711
put : http://1.x24.fr/a/jupyter/seattle3//scatRegressionHeatmap-energystarscore-ghgemissions-metrictonsco2e-+0.08.webp?a=20201111-082715
put : http://1.x24.fr/a/jupyter/seattle3//scatRegressionHeatmap-energystarscore-ghgemissionsintensity-kgco2e-ft2-+0.24.webp?a=20201111-082719
put : http://1.x24.fr/a/jupyter/seattle3//scatRegressionHeatmap-energystarscore-naturalgas-kbtu-+0.08.webp?a=20201111-082723
put : http://1.x24.fr/a/jupyter/seattle3//scatRegressionHeatmap-energystarscore-naturalgas-kbtu-gfa+0.21.webp?a=20201111-082727
put : http://1.x24.fr/a/jupyter/seattle3//scatRegressionHeatmap-energystarscore-siteeuiwn-kbtu-sf-+0.3.webp?a=20201111-082

---
#### EnergyStarScore
- Est construit à partir de ces relevés : 
 - http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-energystarscore-siteeuiwn-kbtu-sf-+0.3.webp
 - http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-energystarscore-sourceeuiwn-kbtu-sf-+0.27.webp
 - http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-energystarscore-electricity-kbtu-gfa+0.25.webp
 - http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-energystarscore-naturalgas-kbtu-gfa+0.21.webp
---
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-electricity-kbtu--siteenergyuse-kbtu-+0.95.webp #l'électricité est l'énergie la plus consommée
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-electricity-kbtu-gfa-ghgemissionsintensity-kgco2e-ft2-+0.43.webp #et a une forte incidence sur l'émission de gazs à effet de serre .. 
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-naturalgas-kbtu-gfa-ghgemissionsintensity-kgco2e-ft2-+0.9.webp #mais pollue moins que l'usage de le gaz (hydrolien, renouvellable, nucléaire )

In [ ]:
#

---
- Pas de corrélation Lat/Long
- SteamUse : réseau de chaleur urbaine = géant chauffage collectif !
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-siteenergyusewn-kbtu-gfa-siteeuiwn-kbtu-sf--0.93.webp #Plus energivore au m² => SiteEUI 
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-electricity-kbtu-gfa-sourceeuiwn-kbtu-sf--0.92.webp #La production de chauffage sur place augmente la consommation electrique
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-ghgemissions-mt-gfa-naturalgas-kbtu-gfa-0.89.webp # et intrasèquement chaque consommation énergétique augment l'émission de GHG
----
- 1) Plus un batiment produit une source d'énergie sur place ( vapeur ) plus il va émettre de gazs à effet de serre ( chaudière ), l'apport d'énergies extérieures aussi, mais dans une moindre importance
- 2) Les corrélations sont différentes, notamment rapportées par ft² vu que les unités employées n'ont pas le même rapport, il faut mieux considérer la métrique linéaire pour la plupart
- 3) Mais EUIWN est très cohérent avec les émission par mettre carré ( Energy Use Intensity ) => Energy Star Score est calculé notamment à partir de ces deux variables
- 4) Une centrale électrique ou un hopital auront tendance à émettre plus de gazs à effets de serre que d'autres types de batiments ! 
=> Les batiments de génération énergétique / ou de haute consommation énergétique

----
## Best Correlation Keys
- Les longues Heatmap étant difficiles à lire autant investir dans de simple tableaux
- Certaines corrélations sont simplement dues à la répartition statistique des données au sein de leur jeu
- Shortlatlon arrondi à 2 décimales = 1km² environ => 42% corrélation with neighborhood ( quartier ), à 3 décimales : valeurs trop uniques; risque sur-apprentissage ( mauvais voisinage si une maison juxtant un immeuble haute consommation des années 60 vient à être démolie pour construire un beau batiment BBC !!
- Corrélation avec nombre d'étages => plotter sur une carte de chaleur downtown

In [ ]:
bestCorrelationsKeys={}
for i in x:
    print('_'*140+'\n')
    print('Best correlations for : '+i)
    print('_'*140+'\n')
    #x=correlations.loc[i].filter(regex='^(?!^(Site|Source).*|.*GFA$).*', axis=0).sort_values(ascending=False)
    y=correlations.loc[i].sort_values(ascending=False)
    print(y[:20])
    bestCorrelationsKeys[i]=list(y.index)

____________________________________________________________________________________________________________________________________________

Best correlations for : ENERGYSTARScore
____________________________________________________________________________________________________________________________________________

SiteEUIWN(kBtu/sf)                                                   0.299048
SiteEUI(kBtu/sf)                                                     0.296288
SiteEnergyUseWN-KBtu/Gfa                                             0.292912
SourceEUIWN(kBtu/sf)                                                 0.274112
SourceEUI(kBtu/sf)                                                   0.272240
Electricity-KBtu/Gfa                                                 0.253794
LargestPropertyUseType                                               0.246455
GHGEmissions-Mt/Gfa                                                  0.243669
GHGEmissionsIntensity(kgCO2e/ft2)                   

---
### ߷ EnergyStarScore ?
- Best correlations : SiteEUIWN Intensité Usage Energétique : 30%
- http://1.x24.fr/a/jupyter/seattle/scatRegressionHeatmap-energystarscore-siteeuiwn-kbtu-sf--0.3.webp

In [ ]:
i='ENERGYSTARScore'
y=correlations.loc[i].sort_values(ascending=False)
print(y[:20])

SiteEUIWN(kBtu/sf)                                                   0.299048
SiteEUI(kBtu/sf)                                                     0.296288
SiteEnergyUseWN-KBtu/Gfa                                             0.292912
SourceEUIWN(kBtu/sf)                                                 0.274112
SourceEUI(kBtu/sf)                                                   0.272240
Electricity-KBtu/Gfa                                                 0.253794
LargestPropertyUseType                                               0.246455
GHGEmissions-Mt/Gfa                                                  0.243669
GHGEmissionsIntensity(kgCO2e/ft2)                                    0.236465
z_ListOfAllPropertyUseTypes_other                                    0.228454
NaturalGas-KBtu/Gfa                                                  0.209963
SecondLargestPropertyUseType                                         0.208664
z_ListOfAllPropertyUseTypes_multifamily housing                 

---
## IO :: export results checkpoint
---

In [ ]:
#'Electricity-KBtu/Gfa,SiteEnergyUseWN-KBtu/Gfa,Steam-KBtu/Gfa,GHGEmissions-Mt/Gfa,NaturalGas-KBtu/Gfa,OtherFuelUse-kBtu/Gfa,Electricity(kBtu),SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SteamUse(kBtu),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),NaturalGas(kBtu)'.split(',')
#energies=x
#save(globals(),'df,results,df2015,df2016,x1,y1,x2,y2,row,heatmap'.split(','),'export1')#globals().update(resume('export1'))

[('train', 10421822), ('correlations', 138458), ('topCorrelations', 108670), ('y', 13290), ('installed', 6873), ('0pairplots', 2280), ('dummies', 2280), ('ndf', 1659), ('In', 816), ('dummiesk', 768), ('data', 704), ('Out', 648), ('completion', 648), ('pairs', 648), ('bestCorrelationsKeys', 648), ('normalize', 529), ('nbRdKeysPerModel', 240), ('votedm', 240), ('res', 240), ('pred', 240), ('acy', 240), ('nfm', 240), ('fi', 240), ('ev', 240), ('bestparams', 240), ('bss', 240), ('repeatedGridSearchCVSameResults', 240), ('bestRandomStateValues', 240), ('bestRandom', 240), ('loc', 240), ('x', 232), ('cols', 232), ('requiredModules', 216), ('energies', 216), ('toPredictBase', 216), ('toPredict', 216), ('interestingMetrics', 208), ('dropped', 208), ('im', 208), ('npRandomKeys', 200), ('randomKeys', 200), ('toDrop', 192), ('considerEmpty', 136), ('exclusions', 136), ('r', 128), ('nc', 128), ('newData', 128), ('newKeys', 112), ('fn', 97), ('j', 93), ('colName', 74), ('energy', 66), ('threads', 6

---
### LargestPropertyUseType : Violins
- Afin de prédire les émissions carbone d'un batiment GHGEmissionsIntensity,(kgCO2e/ft2), nous avons toutes ces variables à disposition dans les meilleures corrélations dont l'usage de gaz est la première, puis le type de Résidence/Commerce/Facilité/Usine présente dans ce dernier d'où l'utilité des dummies sur ce dernier

- LargestPropertyUseType                   0.245083
- Energy/Power Station                     0.234361 ( le batiment est une centrale thermique => la surface industrielle va t-elle être un facteur multipliant ? )
- Multifamily Housing                      0.231731
- Laboratory                               0.206927
- Other - Recreation                       0.199315
- Supermarket/Grocery Store                0.192647
- Hospital (General Medical & Surgical)    0.168149
- BuildingType                             0.167943
- YearBuilt                                0.149865
- Hotel                                    0.125551
- Fitness Center/Health Club/Gym           0.122078
- SecondLargestPropertyUseType             0.105966
- Swimming Pool                            0.104221
- Restaurant                               0.104108
- Other - Technology/Science               0.092024
---

In [10]:
plt.rcParams["figure.figsize"] = (12,12)
i='GHGEmissionsIntensity(kgCO2e/ft2)';j='NaturalGas-KBtu/Gfa'
#scatter(train,i,j)
snsscat(train,i,j,opacity=0.5,color=[1,0,0.3,0.5],kind='reg',fn='ghg-gaz.png',minx=train[i].min(),maxx=train[i].max(),miny=train[j].min(),maxy=train[j].max())#Reg

cats='LargestPropertyUseType'.split(',')
#j='LargestPropertyUseType';
for cat in cats:
  print('}'+cat+'{')
  for i in energies:
    print('_'*180)
    print(i,end=' : ')
    #i='GHGEmissions(MetricTonsCO2e)';
    ts=20;
    f, ax = plt.subplots(figsize=(70,12))#large !! 70/12
    #ax.set(yscale="log")#, yscale="log"
    ax.set_xlim(auto=True)
    sns.set_style("ticks")
    ax.set_xticklabels(labels=train[i],rotation=(90),fontsize = 10,backgroundcolor='white')     

    #sns.regplot("x", "y", data, ax=ax, scatter_kws={"s": 100}), violin polot x axis not continuons
    fn='violinPerCatNoLog-'+filename(i+'-'+cat)+'.png';
    y=sns.violinplot( y=train[i], x=train[cat], ax=ax,scale='width',height=40,gridspec_kws={'wspace':.03})
    #!find . -name '*.png' -o -name '*.webp' -exec rm {} \;    
    y.get_figure().savefig(fn, bbox_inches='tight',dpi=dpi);
    webp(fn);#show()
    #y.get_figure()
#plt.savefig(fn, bbox_inches='tight'); ftpput(fn) 

#df[(df[j]=='Hospital (General Medical & Surgical)')][['PropertyName',i]]
#', '.join(map(str,train[j].unique()))

put : http://1.x24.fr/a/jupyter/seattle3//ghg-gaz.webp?a=20201111-080951
}LargestPropertyUseType{
____________________________________________________________________________________________________________________________________________________________________________________
ENERGYSTARScore : put : http://1.x24.fr/a/jupyter/seattle3//violinPerCatNoLog-energystarscore-largestpropertyusetype.webp?a=20201111-081000
____________________________________________________________________________________________________________________________________________________________________________________
Electricity(kBtu) : put : http://1.x24.fr/a/jupyter/seattle3//violinPerCatNoLog-electricity-kbtu--largestpropertyusetype.webp?a=20201111-081006
____________________________________________________________________________________________________________________________________________________________________________________
SiteEUI(kBtu/sf) : put : http://1.x24.fr/a/jupyter/seattle3//violinPerCatNo

---
## Hue Pairplots ( classifiés )
- http://1.x24.fr/a/jupyter/seattle/violinPerCatNoLog-electricity-kbtu--largestpropertyusetype.webp #Datacenter
-> Tracer le plus grand pairplot jamais réalisé avec un bon processeur une bonne fois pour toutes en exportant le .png résultant afin de voir chacune des corrélations sous forme graphique ..
---


In [9]:
df=train
plt.rcParams['figure.figsize']=(320,20);#Really Large png, cant be converted to webp with width > 16383 
#ValueError: max must be larger than min in range parameter.
# https://github.com/stanleychris2/graph-a-day/blob/master/2:7:16%20-%202014%20San%20Francisco%20Energy%20Score%20by%20Zip.ipynb?short_path=b066f80
if True:
  import matplotlib.image as mpimg
  #plt.rcParams['figure.figsize']=(320,320)#hugeeeeFIGSize for All parameters in 
  dfKeys=df.keys()

#print(df.dtypes['YearsENERGYSTARCertified'])
  for i in bestCorrelationsKeys.keys():
    if((df.dtypes[i]==object) | (i.find('z_')>-1)):
      print('skipping:'+i)
      continue;

    #print(i+' is '+str(df.dtypes[i]))#float64,object is okay for x axis ( categorized )      
    top30=bestCorrelationsKeys[i][:30];#respectivement
    for j in top30:
      if(j not in dfKeys):
        rm.append(j);
      #print(j)
      elif(j.find('z_')>-1):
        rm.append(j);
      elif('z_' in j):#splitted dummie data
        rm.append(j);
      elif(j=='ThirdLargestPropertyUseType'):
        rm.append(j);
      elif(df.dtypes[j]==object):
        rm.append(j);
    
    if(True & (i in top30)):#Removes itself Mutually Exclusive
      rm.append(i);#remove self key which has 100% correlation

    #print('.Removed plot data : '+','.join(rm))
    for z in rm:
      if(z in top30):
        top30.remove(z)
    #top30

    x='top30correlations-pairplot-scatter.'+filename(i)+'.png'
    #getFile(x);#fatal
    if(False & os.path.exists(x)):
      print('resuming:'+x)
      img=mpimg.imread(x);imgplot=plt.imshow(img);show()
    else:
    #g.fig.set_figheight(320);g.fig.set_figwidth(320);
  #AttributeError: 'PairGrid' object has no attribute 'get_figure'  
      
#Electricity(kBtu)/ThirdLargestPropertyUseType
#sns.pairplot ValueError: could not convert string to float: '2015, 2014' After : YearsEnergyStarCertified    ,
#uniqueValuesPerColumn('Electricity(kBtu)',df)df['Electricity(kBtu)'].unique(),NotFound:top30correlations-pairplot-scatter.electricitykbtu.png,ValueError: could not convert string to float: 'Restaurant'
#print(i)
#diag_kind='kde', <= seulement si stats croisées
      #sns.pairplot(df,kind="scatter",height=10,aspect=1,y_vars=[i],x_vars=top30).fig.savefig(x,bbox_inches='tight');#width=height*aspect
      sns.pairplot(df,kind="scatter",height=10,aspect=1,y_vars=[i],x_vars=top30,hue='LargestPropertyUseType').fig.savefig('hue'+x,bbox_inches='tight',dpi=dpi);#width=height*aspect
      webp('hue'+x);#show()

#Erreur rencontrées en chemin ...
if False:
  #ThirdLargestPropertyUseType
  i='ThirdLargestPropertyUseType';uniqueValuesPerColumn(i,df)#ValueError: could not convert string to float: 'Restaurant'
  i='GHGEmissions(MetricTonsCO2e)';df[i]=df[i].astype(float)
  df.dtypes[i]
  df[i]=df[i].astype(float);  

dpi=100
plt.rcParams['figure.figsize']=(20,6);#Restrict back dimensions

put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.energystarscore.webp?a=20201111-080327
put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.electricity-kbtu-.webp?a=20201111-080401
put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.electricity-kbtu-gfa.webp?a=20201111-080430
put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.ghgemissions-metrictonsco2e-.webp?a=20201111-080509
put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.ghgemissionsintensity-kgco2e-ft2-.webp?a=20201111-080538
put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.naturalgas-kbtu-.webp?a=20201111-080611
put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.naturalgas-kbtu-gfa.webp?a=20201111-080635
put : http://1.x24.fr/a/jupyter/seattle3//huetop30correlations-pairplot-scatter.siteeuiwn-kbtu-sf-.webp?a=20201111-080658
pu

#.-------------------------------------------------------------


---
## 🗲🗲 TPU 🗲🗲
- Bring it inline if available
---

In [ ]:
import tensorflow as tf
if 'select right processor type':#not tpu:
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
  except ValueError:
    tpu = None

  if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    message('tpu inline')
  else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    message('not tpu')
    strategy = tf.distribute.get_strategy()     
     
clear_output()
p("REPLICAS: ", strategy.num_replicas_in_sync)
nbSplits=10;bs=8*128;roundIt=True
p(date())

2020-11-09 09:23:38.369540


---
# III) => Models
---
- L'inconvénient principal à faire rentrer les données dans le modèle réside dans chacune des valeurs vides, ou infinies ou de type chaine de caractères, la fonctione get_dummies convertit chacune des valeurs uniques d'un champ 'string' en autant de colonne booléenne donc il faut cloner le dataframe, puis lui retirer certaines colonnes non pertinentes afin d'en extraire ensuite les "dummies".
- On divise donc arbitrairement ce dataframe en 80% de données pour déduire le modèle et les 20% restants afin d'estimer sa précision
---

In [ ]:
#globals().update(resume('export1'))

allModelsP={};
allModelsP['dummy']=sklearn.dummy.DummyRegressor
allModelsP['LinearSVR']=sklearn.svm.LinearSVR
allModelsP['OrthogonalMatchingPursuit']=sklearn.linear_model.OrthogonalMatchingPursuit
allModelsP['XGBRegressor']=xgboost.XGBRegressor
allModelsP['RandomForestRegressor']=sklearn.ensemble.RandomForestRegressor
allModelsP['ExtraTreesRegressor']=sklearn.ensemble.ExtraTreesRegressor
allModelsP['AdaBoostRegressor']=sklearn.ensemble.AdaBoostRegressor
allModelsP['GradientBoostingRegressor']=sklearn.ensemble.GradientBoostingRegressor
allModelsP['KernelRidge']=sklearn.kernel_ridge.KernelRidge
allModelsP['PassiveAggressiveRegressor']=sklearn.linear_model.PassiveAggressiveRegressor
allModelsP['MLPRegressor']=sklearn.neural_network.MLPRegressor
allModelsP['LinearRegression']=sklearn.linear_model.LinearRegression
allModelsP['Ridge']=sklearn.linear_model.Ridge
allModelsP['HistGradientBoostingRegressor']=sklearn.ensemble.HistGradientBoostingRegressor
allModelsP['KNeighborsRegressor']=sklearn.neighbors.KNeighborsRegressor
allModelsP['HuberRegressor']=sklearn.linear_model.HuberRegressor
allModelsP['Lars']=sklearn.linear_model.Lars
allModelsP['LassoLars']=sklearn.linear_model.LassoLars
allModelsP['BayesianRidge']=sklearn.linear_model.BayesianRidge
allModelsP['SVR']=sklearn.svm.SVR
allModelsP['StackingRegressor']=sklearn.ensemble.StackingRegressor
allModelsP['DecisionTreeRegressor']=sklearn.tree.DecisionTreeRegressor
#ValueError: Invalid parameter random_state for estimator SVR(
import inspect
hasRandomState=[]
for i in allModelsP.keys():
  has='random_state' in inspect.getargspec(allModelsP[i])[0]    
  #has='random_state' in dir(allModels[i])#SVR false positive
  if has:    
    hasRandomState+=[i];
p('Models having random state : '+' , '.join(hasRandomState))
#
allModels={};
allModels['PassiveAggressiveRegressor']=sklearn.linear_model.PassiveAggressiveRegressor()
allModels['dummy']=sklearn.dummy.DummyRegressor()
allModels['LinearSVR']=sklearn.svm.LinearSVR()
allModels['OrthogonalMatchingPursuit']=sklearn.linear_model.OrthogonalMatchingPursuit()
allModels['XGBRegressor']=xgboost.XGBRegressor()
allModels['GradientBoostingRegressor']=sklearn.ensemble.GradientBoostingRegressor()
allModels['KernelRidge']=sklearn.kernel_ridge.KernelRidge()
allModels['MLPRegressor']=sklearn.neural_network.MLPRegressor()
allModels['LinearRegression']=sklearn.linear_model.LinearRegression()
allModels['Ridge']=sklearn.linear_model.Ridge()
allModels['HistGradientBoostingRegressor']=sklearn.ensemble.HistGradientBoostingRegressor()
allModels['KNeighborsRegressor']=sklearn.neighbors.KNeighborsRegressor()
allModels['HuberRegressor']=sklearn.linear_model.HuberRegressor()
allModels['Lars']=sklearn.linear_model.Lars()
allModels['LassoLars']=sklearn.linear_model.LassoLars()
allModels['BayesianRidge']=sklearn.linear_model.BayesianRidge()
allModels['SVR']=sklearn.svm.SVR()
allModels['StackingRegressor']=sklearn.ensemble.StackingRegressor([('lr',sklearn.linear_model.RidgeCV()),('svr', sklearn.svm.LinearSVR(random_state=42))],sklearn.ensemble.RandomForestRegressor(random_state=42))
allModels['RandomForestRegressor']=sklearn.ensemble.RandomForestRegressor()
allModels['ExtraTreesRegressor']=sklearn.ensemble.ExtraTreesRegressor()
allModels['AdaBoostRegressor']=sklearn.ensemble.AdaBoostRegressor()
allModels['DecisionTreeRegressor']=sklearn.tree.DecisionTreeRegressor(random_state=42);

if False:
  param_grid={}
  randomKeys=range(1,nbRdKeys)
  npRandomKeys=range(1,nbRdKeys)

  my_cv=5;#sklearn.model_selection.KFold()

  nbRdKeys=60
  models=allModels
  for energy in toPredict:  
    print(energy)
    x1,y1,x2,y2,y_mean,mean=getSplitted(energy,dfNoEnergies)
    k1='test-rd-'+energy
    if k1 not in r2s.keys():
      r2s[k1]={};
    bestRandom[energy]={}
    for i in hasRandomState:
      if i in models.keys():
        gsm(allModels,i,k1,abort=True)    
        bestRandom[energy][i]=bsr=bestParameters[k2]['random_state']
        print(i+'\tr2:'+str(round(r2,2))+'\trd:'+str(bsr))
    print(arsort(bestRandomStateValues))
    break
  #del allModels;  
  assert(False)

  globals().update(resume('prepa1'))#avoir le split x1,x2
  #d'où l'importance de conserver plusieurs random_state : Nombreux sont ceux qui affectent 42, afin de fournir de meilleurs modèles sur ceux qui acceptent ce paramètre !


  #explication des classements des valeurs
  cle='test-rd-Electricity(kBtu)-LinearSVR'
  scores={}
  resultats=list(cvResults[cle]['rank_test_score'])
  j=0;
  for i in resultats:
    k=cvResults[cle]['params'][j]
    scores[i]=k
    j+=1;
  print(sorted(scores.items()))



  print(grid_search.cv_results_.rank_test_score)
  grid_search_cv
  results = pd.DataFrame(grid.cv_results_.rank_test_score)
  results.sort_values(by='rank_test_score', inplace=True)


  allModels={};
  for i in allModelsP.keys():
    p={}
    if(i=='StackingRegressor'):
      p=[('lr',sklearn.linear_model.RidgeCV()),('svr', sklearn.svm.LinearSVR(random_state=42))],sklearn.ensemble.RandomForestRegressor(random_state=42)
    allModels[i]=allModelsP[i](p)
  #allModels['StackingRegressor']

Models having random state : LinearSVR , XGBRegressor , RandomForestRegressor , ExtraTreesRegressor , AdaBoostRegressor , GradientBoostingRegressor , PassiveAggressiveRegressor , MLPRegressor , Ridge , HistGradientBoostingRegressor , DecisionTreeRegressor


---
##A) Prépa / Splitting Data# 
- Peut influencer sur les scores des modèles entrainés ci dessous
- Nécessité : non shuffling splits afin de pouvoir comparer les modèles sur les mêmes jeux d'entrainements !!
---

In [ ]:
#resume();#print(acy.keys())#YearsENERGYSTARCertified
#globals().update(resume('export1'))
y_means={};resultss={};resultsArray={};splittedData={};mainPrediction={};models={};history={};cvResults={};
param_grid={};voting_grid={};ev={};fi={};pred={};bestPredictions={};t={};bestParameters={};r2s={}#reset

grid_search=0;_df=0;k2=0;
my_cv=nbFold=5;includeEnergyStar=0;includeOriginalDf=0;sshuffleData=0;stopAtOne=1;#more accurate predictions
#obj:73kilo, barracuda !
pd.set_option('display.max_columns',train.shape[1])

train['zip']=train['zip'].astype('int')
train['Outlier']=train['Outlier'].astype('str')
train['DefaultData']=train['DefaultData'].astype('bool')#works with No,False
train['LargestPropertyUseType']=train['LargestPropertyUseType'].astype('str')
#train['YearsENERGYSTARCertified']=train['YearsENERGYSTARCertified'].astype('str')
train['ThirdLargestPropertyUseType']=train['ThirdLargestPropertyUseType'].astype('str')
train['SecondLargestPropertyUseType']=train['SecondLargestPropertyUseType'].astype('str')
#train['TaxParcelIdentificationNumber']=train['TaxParcelIdentificationNumber'].astype('str')#invalide litterals for int .. dropped

#YearsENERGYSTARCertified,
#dropper les relevés que l'ont aura pas sur les nouveaux batiments .. 
if True:
  labelEncoded=train.copy(deep=True);#&

  #Original Préparé
  if True:
    for col in labelEncoded:
      if(labelEncoded.dtypes[col]==object):
        print('converting:'+col)
        labelEncoded[col]=sklearn.preprocessing.LabelEncoder().fit_transform(labelEncoded[col].astype('str'))
      elif col.startswith('z_'):#dummies are boolen
        labelEncoded[col]=labelEncoded[col].astype('bool')
      else:
        labelEncoded[col]=labelEncoded[col].fillna(0).astype('float')

  dfNewKeys=labelEncoded[newKeys]
  labelEncoded=labelEncoded.drop(newKeys,axis=1);#Toutes sauf newKeys
  labEncCols=list(labelEncoded.columns.values)

  print(datetime.datetime.now())
  print(labelEncoded.dtypes)

  dfNoEnergies=labelEncoded.copy(deep=True);#& celui sur lequel on va travailler également en retirant des valeurs
  df2Energies=dfNoEnergies[energies]#mise de cotés une fois converties en float avec fillna
  dfNoEnergies=dfNoEnergies.drop(energies,axis=1)#puis dropées
  others=list(dfNoEnergies.columns.values);#cols non energies
save(globals(),['allModels','allModelsP'],'prepa1')
#globals().update(resume('prepa1'))

converting:BuildingType
converting:PrimaryPropertyType
converting:Neighborhood
converting:ListOfAllPropertyUseTypes
converting:LargestPropertyUseType
converting:SecondLargestPropertyUseType
converting:ThirdLargestPropertyUseType
converting:ComplianceStatus
converting:Outlier
converting:shortlatlon
2020-11-09 09:25:37.523238
BuildingType                                                                int64
PrimaryPropertyType                                                         int64
CouncilDistrictCode                                                       float64
Neighborhood                                                                int64
YearBuilt                                                                 float64
NumberofBuildings                                                         float64
NumberofFloors                                                            float64
PropertyGFATotal                                                          float64
PropertyGFAParking

---
## B) Models 𒀱
- ( on n'utilise qu'une seule métrique que l'on cherche à prédire : Electricity Kbtu )
- Dummy sur un vote correspond à tricher en induisant la valeur médiane ou moyenne des valeurs à prédires dont nous ne sommes pas sensés disposer
---

---
#### 1) Comparaisons des modèles sur les données d'origine avec tous les champs fortement corrélées
----
- Sur le dataframe originel sans dropper les colonnes énergies afin de mesurer les écarts sur les prédictions => on voit que ceci fonctionne très bien http://1.x24.fr/a/jupyter/seattle/scatterPredictionsNoParameters-electricity-kbtu-gfa-linearregression-1.0.webp
- Certains modèles performent mieux que d'autres, même en présence de tous les relevés !!
- => ce qui permet de disqualifier les modèles présentants de faibles performances vis à vis de ce dataset
- <b><u>Attention cependant à tester multiples random_state pour le retour du r2score pour certains modèles nécessitant plusieurs initialisations aléatoires</u></b>
- On constate que la régression linéaire de base suffit le cas écheant !
---

In [ ]:
toPredict='Electricity(kBtu)'.split(',')
energy=toPredict[0]
rd=42
dpi=100

x1,y1,x2,y2,y_mean,mean=getSplitted(energy,labelEncoded.drop([energy],axis=1))
Kbtu_y_mean=ElectKbtu_y_mean=y_mean
Kbtu_mean=mean
Kbtu_y2=y2

import sklearn.tree
models={}
models['DecisionTreeRegressor']=sklearn.tree.DecisionTreeRegressor(random_state=rd);
models['dummy']=sklearn.dummy.DummyRegressor()
models['LinearRegression']=sklearn.linear_model.LinearRegression(n_jobs=-1)  
models['Lasso']=sklearn.linear_model.Lasso(random_state=rd)#Ridge less sensitive to outliers(abs)
models['ElasticNet']=sklearn.linear_model.ElasticNet(random_state=rd)#=Ridge + Lasso Penalties with their own λ

models['ExtraTreesRegressor']=sklearn.ensemble.ExtraTreesRegressor(random_state=rd)
models['RandomForestRegressor']=sklearn.ensemble.RandomForestRegressor(random_state=rd)


models['HistGradientBoostingRegressor']=sklearn.ensemble.HistGradientBoostingRegressor(random_state=rd) 
models['GradientBoostingRegressor']=sklearn.ensemble.GradientBoostingRegressor(random_state=rd)  
models['DecisionTreeRegressor']=sklearn.tree.DecisionTreeRegressor(random_state=rd)
models['KNeighborsRegressor']=sklearn.neighbors.KNeighborsRegressor()
models['HuberRegressor']=sklearn.linear_model.HuberRegressor()
models['Lars']=sklearn.linear_model.Lars()
models['LassoLars']=sklearn.linear_model.LassoLars()
models['BayesianRidge']=sklearn.linear_model.BayesianRidge()  
models['LinearSVR']=sklearn.svm.LinearSVR(random_state=rd)
models['OrthogonalMatchingPursuit']=sklearn.linear_model.OrthogonalMatchingPursuit()  
models['SVR']=sklearn.svm.SVR()
models['StackingRegressor']=sklearn.ensemble.StackingRegressor([('Ridge',sklearn.linear_model.RidgeCV()),('BayesianRidge', sklearn.linear_model.BayesianRidge()  )],sklearn.ensemble.RandomForestRegressor(n_estimators=300))
models['dummy']=sklearn.dummy.DummyRegressor()
models['KernelRidge']=sklearn.kernel_ridge.KernelRidge()#0.72 !!!
models['PassiveAggressiveRegressor']=sklearn.linear_model.PassiveAggressiveRegressor(random_state=rd)
models['MLPRegressor']=sklearn.neural_network.MLPRegressor(random_state=rd)#
models['LinearRegression']=sklearn.linear_model.LinearRegression()
models['Ridge']=sklearn.linear_model.Ridge(random_state=rd)
models['XGBRegressor']=xgboost.XGBRegressor(random_state=rd)#~has importance metrics ;) -> when not voted
models['AdaBoostRegressor']=sklearn.ensemble.AdaBoostRegressor(random_state=rd)#trunks=Stumps
allModels1=models.copy()
timePerModel={}
scorePerModel={}
timePerModel[1]={}
scorePerModel[1]={}

for model in models:
  start = tim()
  models[model].fit(x1,x2)
  predictions=models[model].predict(y1)
  consommation=tim() - start
  timePerModel[1][model]=consommation
  
  predMean=predictions.mean()
  r2=sklearn.metrics.r2_score(y2,predictions) 
  r2s['Original-'+energy+model]=r2;

  scorePerModel[1][model]=r2
  print(model+' = time:'+str(consommation)+' , score :'+str(r2))
    
  _df=pd.DataFrame(list(zip(y2,predictions,y_mean)),columns=['Real','Guess','Mean'])
  fn='sp-npFullDF-'+filename(energy+'-'+model)+'-'+str(round(r2,2))+'.png'  

  f=_df[['Real','Guess','Mean']].plot(alpha=0.4,style=['o','r^','g-'],title='Données Réelles vs prédictions '+energy+'-'+model+' - '+str(round(r2,2)));    
  #plt.figure(figsize=(24,6))
  plt.yscale('log');
  f.get_figure().savefig(fn,bbox_inches='tight',dpi=dpi);#results in nothing !!!!
#Maximum width and height allowed is 16383 pixels  
  #!stat sp-npFullDF-electricity-kbtu--decisiontreeregressor-0.11.png
  webp(fn);#show();  
  #assert(False)

display(arsort(timePerModel[1]))
display(arsort(scorePerModel[1]))


if False:
  os.system("find . -name '*.webp' -exec rm -f {} \;find . -name '*.png' -exec rm -f {} \;");
  def webp(x):
    if(GG('webpBroken')):
      show();
      return x
    x2 = x.replace('.png', '') + '.webp'
    #p(x,'=>',x2)
    res=webplib.cwebp(x, x2, '-q 70')
    if res['stderr']:
      err=res['stderr'].decode('ascii')
      if "cannot open input file" in err:
        p(err);
        assert(False)
      if "Cannot encode picture as WebP" in err:
        p(err)
        assert(False)
    if removepng:
      os.system('rm -f ' + x)
    if sendimages2ftp:
      ftpput(x2)
    if(GG('dontshow')):
      plt.close()
    return x2
  f.get_figure().savefig(fn,bbox_inches='tight',dpi=100);#results in nothing !!!!  
  webp(fn);#show();  
  #f.get_figure().savefig(fn,bbox_inches='tight');#results in nothing !!!!,dpi=30  

non shuffled
DecisionTreeRegressor = time:0.25005102157592773 , score :0.11022339906731071
put : http://1.x24.fr/a/jupyter/seattle3//sp-npFullDF-electricity-kbtu--decisiontreeregressor-0.11.webp?a=20201110-162528
dummy = time:0.001054525375366211 , score :-0.0051843761288563606
put : http://1.x24.fr/a/jupyter/seattle3//sp-npFullDF-electricity-kbtu--dummy--0.01.webp?a=20201110-162531
LinearRegression = time:0.08564949035644531 , score :0.9999999990205852
put : http://1.x24.fr/a/jupyter/seattle3//sp-npFullDF-electricity-kbtu--linearregression-1.0.webp?a=20201110-162534
Lasso = time:0.7138428688049316 , score :0.927911761840808
put : http://1.x24.fr/a/jupyter/seattle3//sp-npFullDF-electricity-kbtu--lasso-0.93.webp?a=20201110-162538
ElasticNet = time:0.7615311145782471 , score :0.9326519561095342
put : http://1.x24.fr/a/jupyter/seattle3//sp-npFullDF-electricity-kbtu--elasticnet-0.93.webp?a=20201110-162542
ExtraTreesRegressor = time:7.570325136184692 , score :0.2853828531462239
put : http:/

[('KernelRidge', 26.24553370475769),
 ('RandomForestRegressor', 12.684064388275146),
 ('SVR', 8.62802267074585),
 ('ExtraTreesRegressor', 7.570325136184692),
 ('StackingRegressor', 4.98056697845459),
 ('GradientBoostingRegressor', 4.189891576766968),
 ('AdaBoostRegressor', 3.965222120285034),
 ('HistGradientBoostingRegressor', 2.9157795906066895),
 ('MLPRegressor', 2.330601692199707),
 ('XGBRegressor', 1.7197182178497314),
 ('LinearSVR', 0.9872734546661377),
 ('ElasticNet', 0.7615311145782471),
 ('Lasso', 0.7138428688049316),
 ('HuberRegressor', 0.667478084564209),
 ('DecisionTreeRegressor', 0.25005102157592773),
 ('KNeighborsRegressor', 0.23041343688964844),
 ('Lars', 0.11325311660766602),
 ('LassoLars', 0.10446524620056152),
 ('BayesianRidge', 0.10036706924438477),
 ('LinearRegression', 0.08564949035644531),
 ('PassiveAggressiveRegressor', 0.07894659042358398),
 ('OrthogonalMatchingPursuit', 0.06838345527648926),
 ('Ridge', 0.06626558303833008),
 ('dummy', 0.001054525375366211)]

[('LinearRegression', 0.9999999990205852),
 ('Ridge', 0.9999999975273928),
 ('KernelRidge', 0.9998628675594565),
 ('LinearSVR', 0.9638311952296262),
 ('OrthogonalMatchingPursuit', 0.9634279703977365),
 ('MLPRegressor', 0.9625361016938792),
 ('BayesianRidge', 0.9602733717653016),
 ('HuberRegressor', 0.9602009425112779),
 ('PassiveAggressiveRegressor', 0.9463959965197531),
 ('ElasticNet', 0.9326519561095342),
 ('Lasso', 0.927911761840808),
 ('LassoLars', 0.8944930529477635),
 ('StackingRegressor', 0.6360139668065562),
 ('XGBRegressor', 0.5837449948881015),
 ('KNeighborsRegressor', 0.4938825840664002),
 ('GradientBoostingRegressor', 0.35291422287373686),
 ('RandomForestRegressor', 0.3209819565237204),
 ('HistGradientBoostingRegressor', 0.3186564521571754),
 ('ExtraTreesRegressor', 0.2853828531462239),
 ('AdaBoostRegressor', 0.18757559373842936),
 ('DecisionTreeRegressor', 0.11022339906731071),
 ('SVR', -0.005040903933333674),
 ('dummy', -0.0051843761288563606),
 ('Lars', -11.1544931496981

---
#### 2) Amputation : Df sans les mesures énergétiques & GFA
---
- La suppression des colonnes rend la prévision plus difficile qu'auparavant ..
- Electricity Kbtu > Ridge : http://1.x24.fr/a/jupyter/seattle/spNoEnergies-electricity-kbtu--ridge01-0.25.webp
- http://1.x24.fr/a/jupyter/seattle/spNoEnergies-electricity-kbtu--bayesianridge-0.83.webp #basé sur score r2 que nous ne somme pas sensé avoir
---
 - Electricity-KBtu/Gfa > ExtraTrees : http://1.x24.fr/a/jupyter/seattle/spNoEnergies-electricity-kbtu-gfa-extratreesregressor-0.83.webp ( sans multiplication )
 - http://1.x24.fr/a/jupyter/seattle/spNoEnergiesMultiplied-electricity-kbtu-gfa-stackingregressor-0.95.webp #0.95
 ---
- => Electricity Kbtu/Gfa plus façile à pré dire que Kbtu tout cours, du moins avec les modèles "simples", grâce aux corrélations
- Puis reconstruction en le multipliant par la surface du batiment > on passe de  <a href='http://1.x24.fr/a/jupyter/seattle/spNoEnergies-electricity-kbtu-gfa-linearregression--21.12.webp'>-21.12</a> à <a href='http://1.x24.fr/a/jupyter/seattle/spNoEnergiesMultiplied-electricity-kbtu-gfa-linearregression-0.93.webp'>0.93</a> ( meilleur modèle à l'aveugle )
---
- Selon le feat engineering ou pas, les modèles nous retournent différents coefficients de détermination : Bayesian Ridge performe le mieux ( surtout une fois avoir multiplié la valeur prédite par la surface totale du batiment )
---

In [ ]:
#save(globals(),'df,results,df2015,df2016,x1,y1,x2,y2,row,heatmap,df2Energies'.split(','),'amputee1')#globals().update(resume('amputee1'))
toPredict='Electricity-KBtu/Gfa,Electricity(kBtu)'.split(',')
timePerModel[2]={}
scorePerModel[2]={}

for energy in toPredict: 
  print('_'*180) 
  print(energy)
  x1,y1,x2,y2,y_mean,mean=getSplitted(energy,dfNoEnergies)
  for model in models:
    start=tim()
    models[model].fit(x1,x2)
    predictions=models[model].predict(y1)
    consommation=tim() - start
    timePerModel[2][energy+':'+model]=consommation    
    predMean=predictions.mean()
    r2=sklearn.metrics.r2_score(y2,predictions) 
    r2s['Empty-'+energy+':'+model]=r2;

    scorePerModel[2][energy+':'+model]=r2

    print(model+' : '+str(r2))
      
    _df2=pd.DataFrame(list(zip(y2,predictions,y_mean)),columns=['Real','Guess','Mean'])
    x='spNoEnergies-'+filename(energy+'-'+model)+'-'+str(round(r2,2))+'.png'
    f=_df2[['Real','Guess','Mean']].plot(alpha=0.4,style=['o','r^','g-'],title='Données Réelles vs prédictions '+energy+'-'+model+' - '+str(round(r2,2)));  
    plt.yscale('log');f.get_figure().savefig(x,bbox_inches='tight',dpi=dpi);webp(x);#show();     

    if(energy=='Electricity-KBtu/Gfa'):
      multiplyPredicitions='PropertyGFATotal'#matchAgainst='Electricity(kBtu)'      
      predictions*=y1[multiplyPredicitions]
      r2=sklearn.metrics.r2_score(Kbtu_y2,predictions) 
      r2s['Empty-Multiplied-'+energy+model]=r2;
      scorePerModel[2][energy+':'+model+':multiplied']=r2
      print(model+' : multiplied : '+str(r2))

      _df2=pd.DataFrame(list(zip(Kbtu_y2,predictions,Kbtu_y_mean)),columns=['Real','Guess','Mean'])
      x='spNoEnergiesMultiplied-'+filename(energy+'-'+model)+'-'+str(round(r2,2))+'.png'
      f=_df2[['Real','Guess','Mean']].plot(alpha=0.4,style=['o','r^','g-'],title='Données Réelles vs prédictions '+energy+'-'+model+' - '+str(round(r2,2)));  
      plt.yscale('log');f.get_figure().savefig(x,bbox_inches='tight',dpi=dpi);webp(x);#show();     

#display(arsort(timePerModel[2]))
display(arsort(scorePerModel[2]))

____________________________________________________________________________________________________________________________________________________________________________________
Electricity-KBtu/Gfa
non shuffled
DecisionTreeRegressor : 0.6699652898606043
put : http://1.x24.fr/a/jupyter/seattle3//spNoEnergies-electricity-kbtu-gfa-decisiontreeregressor-0.67.webp?a=20201110-163754
DecisionTreeRegressor : multiplied : 0.887214569930778
put : http://1.x24.fr/a/jupyter/seattle3//spNoEnergiesMultiplied-electricity-kbtu-gfa-decisiontreeregressor-0.89.webp?a=20201110-163757
dummy : -0.010783767705889957
put : http://1.x24.fr/a/jupyter/seattle3//spNoEnergies-electricity-kbtu-gfa-dummy--0.01.webp?a=20201110-163801
dummy : multiplied : 0.709322182577547
put : http://1.x24.fr/a/jupyter/seattle3//spNoEnergiesMultiplied-electricity-kbtu-gfa-dummy-0.71.webp?a=20201110-163804
LinearRegression : -21.142485759732434
put : http://1.x24.fr/a/jupyter/seattle3//spNoEnergies-electricity-kbtu-gfa-linearregr

[('Electricity-KBtu/Gfa:BayesianRidge:multiplied', 0.94558957762832),
 ('Electricity-KBtu/Gfa:LinearRegression:multiplied', 0.9271930607423154),
 ('Electricity-KBtu/Gfa:Ridge:multiplied', 0.9134502192173347),
 ('Electricity-KBtu/Gfa:DecisionTreeRegressor:multiplied', 0.887214569930778),
 ('Electricity-KBtu/Gfa:KernelRidge:multiplied', 0.870142548630377),
 ('Electricity-KBtu/Gfa:XGBRegressor:multiplied', 0.850932125527954),
 ('Electricity(kBtu):BayesianRidge', 0.8290972802979177),
 ('Electricity(kBtu):MLPRegressor', 0.8257143674348972),
 ('Electricity-KBtu/Gfa:OrthogonalMatchingPursuit:multiplied',
  0.8247801892393299),
 ('Electricity-KBtu/Gfa:GradientBoostingRegressor:multiplied',
  0.8230412819775972),
 ('Electricity-KBtu/Gfa:HistGradientBoostingRegressor:multiplied',
  0.8191246412612967),
 ('Electricity-KBtu/Gfa:ExtraTreesRegressor', 0.8145317212920129),
 ('Electricity-KBtu/Gfa:KNeighborsRegressor:multiplied', 0.8141653038673422),
 ('Electricity(kBtu):LassoLars', 0.7988949625602312

---
####3) +EnergyStar ⇩
- Ré-inclure cet indice au sein du dataframe ne permet pas de meilleurs résultats sur les prévisions, selon certaines métriques à prédire ⇩
- 0.82 pour extratrees sur kbtu/gfa & 0.25 sur ridge sur kbtu de base => Cette variable n'a aucune incidence sur la qualité des prévisions ( avec plus de modèles )
---
- <u><b>Extratrees sur kbtu/gfa : 0.83<b></u>
---

In [ ]:
#save(globals(),'df,results,df2015,df2016,x1,y1,x2,y2,row,heatmap,df2Energies'.split(','),'amputee2');#globals().update(resume('amputee2'))
dfEStar=dfNoEnergies.copy(deep=True);dfEStar['ENERGYSTARScore']=labelEncoded['ENERGYSTARScore'];#Ré-intégrer cet indice !
toPredict='Electricity(kBtu)'.split(',')
timePerModel[3]={}
scorePerModel[3]={}
for energy in toPredict:
  x1,y1,x2,y2,y_mean,mean=getSplitted(energy,dfNoEnergies)

  for model in models:
    start=tim()
    models[model].fit(x1,x2)
    predictions=models[model].predict(y1)
    consommation=tim() - start
    timePerModel[3][energy+model]=consommation    
    predMean=predictions.mean()
    r2=sklearn.metrics.r2_score(y2,predictions) 
    r2s['EnergyStar-'+energy+model]=r2;
    scorePerModel[3][energy+model]=r2
    print(model+' : '+str(r2))
      
    _df=pd.DataFrame(list(zip(y2,predictions,y_mean)),columns=['Real','Guess','Mean'])
    x='spEnergyStarOnly-'+filename(energy+'-'+model)+'-'+str(round(r2,2))+'.png'
    f=_df[['Real','Guess','Mean']].plot(alpha=0.4,style=['o','r^','g-'],title='Données Réelles vs prédictions '+energy+'-'+model+' - '+str(round(r2,2)));  
    plt.yscale('log');f.get_figure().savefig(x,bbox_inches='tight',dpi=dpi);webp(x);#show(); 

#display(asort(timePerModel[3]))
display(arsort(scorePerModel[3]))

non shuffled
DecisionTreeRegressor : 0.19450455776714082
put : http://1.x24.fr/a/jupyter/seattle3//spEnergyStarOnly-electricity-kbtu--decisiontreeregressor-0.19.webp?a=20201110-163543
dummy : -0.0051843761288563606
put : http://1.x24.fr/a/jupyter/seattle3//spEnergyStarOnly-electricity-kbtu--dummy--0.01.webp?a=20201110-163546
LinearRegression : -1.3098847061697918
put : http://1.x24.fr/a/jupyter/seattle3//spEnergyStarOnly-electricity-kbtu--linearregression--1.31.webp?a=20201110-163549
Lasso : -1.314846747646826
put : http://1.x24.fr/a/jupyter/seattle3//spEnergyStarOnly-electricity-kbtu--lasso--1.31.webp?a=20201110-163554
ElasticNet : -2.614179603878726
put : http://1.x24.fr/a/jupyter/seattle3//spEnergyStarOnly-electricity-kbtu--elasticnet--2.61.webp?a=20201110-163558
ExtraTreesRegressor : 0.20370754912464373
put : http://1.x24.fr/a/jupyter/seattle3//spEnergyStarOnly-electricity-kbtu--extratreesregressor-0.2.webp?a=20201110-163607
RandomForestRegressor : 0.167038160922513
put : http://1.

[('Electricity(kBtu)dummy', 0.0011181831359863281),
 ('Electricity(kBtu)Ridge', 0.058255672454833984),
 ('Electricity(kBtu)OrthogonalMatchingPursuit', 0.061758995056152344),
 ('Electricity(kBtu)LinearRegression', 0.07408547401428223),
 ('Electricity(kBtu)PassiveAggressiveRegressor', 0.07431268692016602),
 ('Electricity(kBtu)LassoLars', 0.07470202445983887),
 ('Electricity(kBtu)BayesianRidge', 0.08936715126037598),
 ('Electricity(kBtu)Lars', 0.10282611846923828),
 ('Electricity(kBtu)DecisionTreeRegressor', 0.20817041397094727),
 ('Electricity(kBtu)KNeighborsRegressor', 0.49170851707458496),
 ('Electricity(kBtu)HuberRegressor', 0.5406420230865479),
 ('Electricity(kBtu)ElasticNet', 0.6359987258911133),
 ('Electricity(kBtu)Lasso', 0.6538257598876953),
 ('Electricity(kBtu)LinearSVR', 0.8605520725250244),
 ('Electricity(kBtu)XGBRegressor', 1.4341540336608887),
 ('Electricity(kBtu)AdaBoostRegressor', 1.6348884105682373),
 ('Electricity(kBtu)KernelRidge', 1.830826759338379),
 ('Electricity(kBt

[('Electricity(kBtu)BayesianRidge', 0.8290972802979177),
 ('Electricity(kBtu)MLPRegressor', 0.8257143674348972),
 ('Electricity(kBtu)LassoLars', 0.7988949625602312),
 ('Electricity(kBtu)OrthogonalMatchingPursuit', 0.7556925279502517),
 ('Electricity(kBtu)HuberRegressor', 0.6827125037767094),
 ('Electricity(kBtu)LinearSVR', 0.6278775126232591),
 ('Electricity(kBtu)KernelRidge', 0.5880043700802279),
 ('Electricity(kBtu)PassiveAggressiveRegressor', 0.40166586558715767),
 ('Electricity(kBtu)StackingRegressor', 0.3749858362569527),
 ('Electricity(kBtu)Ridge', 0.29280242474137785),
 ('Electricity(kBtu)KNeighborsRegressor', 0.2245117253356267),
 ('Electricity(kBtu)HistGradientBoostingRegressor', 0.2107242969888885),
 ('Electricity(kBtu)ExtraTreesRegressor', 0.20370754912464373),
 ('Electricity(kBtu)DecisionTreeRegressor', 0.19450455776714082),
 ('Electricity(kBtu)GradientBoostingRegressor', 0.177616343564584),
 ('Electricity(kBtu)XGBRegressor', 0.17045243077927175),
 ('Electricity(kBtu)Random

---
##### Comparaison & Conclusion
- Bien que certaines variables, sous certains modèles soient moins précis en retirant cette variable, 
- Dans la globalité on ne trouve aucune variable dont aucun modèle donne un meilleur résultat en ajoutant la variable EnergyStarScore
- <u>**Global R2 Diff : 0**</u> => Les mêmes scores sont atteints avec ou sans ces indices
---

In [ ]:
#save(globals(),'df,results,df2015,df2016,x1,y1,x2,y2,row,heatmap,df2Energies'.split(','),'amputee3');#globals().update(resume('amputee3'))
p1='EnergyStar-'#EnergyStar-SiteEUIWN(kBtu/sf)RandomForestRegressor
p2='Empty-'#Empty-Electricity(kBtu)Ridge01

globalDiff=0
for energy in toPredict:  
  pf1=p1+energy;pf1k=[]#with EnergyStarScore
  pf2=p2+energy;pf2k=[]#without
  for key in r2s.keys():
    if key.startswith(pf1):
      pf1k+=[key]
    if key.startswith(pf2):
      pf2k+=[key]
#not done yet      
  if(len(pf1k)==0):
    continue;
  pf1kb = { nk: r2s[nk] for nk in pf1k }
  pf2kb = { nk: r2s[nk] for nk in pf2k }
  print(pf1kb)
  best1=arsort(pf1kb)[0]
  best2=arsort(pf2kb)[0]
  diff=best1[1]-best2[1];
  globalDiff+=diff
  if(diff>0):
    print(energy)
  print('Best score for '+energy+':with ESC:'+str(best1)+'/without:'+str(best2))
  print('\tDiff:'+str(diff))
  
print('Global R2 Diff : '+str(globalDiff/len(toPredict)))  

if False:
  print(r2s['Empty-SiteEnergyUseWN(kBtu)RandomForestRegressor'])
  #EnergyStar-Electricity(kBtu)Ridge01s
  differences={}
  for i1 in list(r2s.keys()):
    if(i1.startswith(p1)):
      i3=i1.replace(p1,'')
      i2=i1.replace(p1,p2)
      differences[i3]=diff=r2s[i1]-r2s[i2]
      #print('difference for '+i3+':'+str(diff))
  print(arsort(differences))#Cela pénalise la prédiction des variables corréllées à EnergyStarScore

{'EnergyStar-Electricity(kBtu)DecisionTreeRegressor': 0.19450455776714082, 'EnergyStar-Electricity(kBtu)dummy': -0.0051843761288563606, 'EnergyStar-Electricity(kBtu)LinearRegression': -1.3098847061697918, 'EnergyStar-Electricity(kBtu)Lasso': -1.314846747646826, 'EnergyStar-Electricity(kBtu)ElasticNet': -2.614179603878726, 'EnergyStar-Electricity(kBtu)ExtraTreesRegressor': 0.20370754912464373, 'EnergyStar-Electricity(kBtu)RandomForestRegressor': 0.167038160922513, 'EnergyStar-Electricity(kBtu)HistGradientBoostingRegressor': 0.2107242969888885, 'EnergyStar-Electricity(kBtu)GradientBoostingRegressor': 0.177616343564584, 'EnergyStar-Electricity(kBtu)KNeighborsRegressor': 0.2245117253356267, 'EnergyStar-Electricity(kBtu)HuberRegressor': 0.6827125037767094, 'EnergyStar-Electricity(kBtu)Lars': -2.9647910771247683, 'EnergyStar-Electricity(kBtu)LassoLars': 0.7988949625602312, 'EnergyStar-Electricity(kBtu)BayesianRidge': 0.8290972802979177, 'EnergyStar-Electricity(kBtu)LinearSVR': 0.627877512623

---
####4) Ridge Individuels, puis votés par CV
---
- Grid Search ne semble pas renvoyer les meilleurs paramètres, car il ne se base pas sur les résultats du jeu de test !!
		 ==> Best Reel Model r2:[('Ridge20', 0.8308315043300891)] est différent du score retourné par GridSearchCrossValidation
		 ==> Best Reel Cv Score:[('Ridge02', -48117657941128.47)] # somme résidus
     ====> a un coeff r2 de -0.45 sur le jeu de test, mais -0.05 lorque votés parmis les 3 meilleurs : http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting.r2:-0.05.p:5.25.webp #cv
----
Sur le réel jeu de validation, plus alpha augmente plus le coefficient de détermination augmente ( attention nous mesurons ici ce coefficient basé sur les résultats que nous ne sommes pas sensés disposer )
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-ridge20.r2:+0.83.p:2.1.webp
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting.r2:+0.83.p:2.11.webp #r2

In [ ]:
#Qualification sur rd=42 sur jeu de test / non sensé connaitre les résultats finaux
y_means={};a=time();#scorer_
x1,y1,x2,y2,y_mean,mean=getSplitted('Electricity(kBtu)',dfNoEnergies)
param_grid={'Ridge01':{'random_state':[42]},'Ridge02':{'random_state':[42]},'Ridge1':{'random_state':[42]},'Ridge3':{'random_state':[42]},'Ridge5':{'random_state':[42]},'Ridge10':{'random_state':[42]},'Ridge20':{'random_state':[42]},'Ridge30':{'random_state':[42]}}
models={'Ridge01':sklearn.linear_model.Ridge(alpha=0.1),'Ridge02':sklearn.linear_model.Ridge(alpha=0.2),'Ridge1':sklearn.linear_model.Ridge(alpha=1),
'Ridge3':sklearn.linear_model.Ridge(alpha=3),
'Ridge5':sklearn.linear_model.Ridge(alpha=5),#best
'Ridge10':sklearn.linear_model.Ridge(alpha=10),
'Ridge20':sklearn.linear_model.Ridge(alpha=20),
'Ridge30':sklearn.linear_model.Ridge(alpha=30),
}

voted=2;votingOnReel=0;
modelsEvalutation();best=arsort(r2s[k1])[:1][0];print(best)
b='modelsNoParameters';t[b]=time()-a;print(t[b])

non shuffled
____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-Ridge01{


Ridge01 :: param grid :: {'random_state': [42]}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.3s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-ridge01.r2:-0.77.p:6.81.webp?a=20201110-152148

	{'r2': -0.7726987698374297, 'sqrt': 24392807.0, 'bs': 595009042622367.8, 'nmse': 595009042622367.8, 'mean': 3580898.96, 'PredMean': 76508.39, 'time': 1.3, 'explained_variance_score': -0.7548268545184964, 'per': 6.81192}
	Best Parameters :: 1{'random_state': 42}
	BS:0.3002067034700261

	}Electricity(kBtu)-trimmed-Ridge02{


Ridge02 :: param grid :: {'random_state': [42]}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-ridge02.r2:-0.39.p:6.03.webp?a=20201110-152152

	{'r2': -0.38798928494776463, 'sqrt': 21584270.0, 'bs': 465880717953345.6, 'nmse': 465880717953345.6, 'mean': 3580898.96, 'PredMean': 405447.62, 'time': 0.6, 'explained_variance_score': -0.3745955132399541, 'per': 6.02761}
	Best Parameters :: 1{'random_state': 42}
	BS:0.3069736076870777

	}Electricity(kBtu)-trimmed-Ridge1{


Ridge1 :: param grid :: {'random_state': [42]}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-ridge20.r2:+0.82.p:2.15.webp?a=20201110-152213

	{'r2': 0.8230472828483997, 'sqrt': 7706779.0, 'bs': 59394449081417.32, 'nmse': 59394449081417.32, 'mean': 3580898.96, 'PredMean': 2897035.37, 'time': 0.6, 'explained_variance_score': 0.8234580071443782, 'per': 2.15219}
	Best Parameters :: 1{'random_state': 42}
	BS:0.43879946508753187

	}Electricity(kBtu)-trimmed-Ridge30{


Ridge30 :: param grid :: {'random_state': [42]}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-ridge30.r2:+0.73.p:2.66.webp?a=20201110-152217

	{'r2': 0.7291595986706545, 'sqrt': 9534568.0, 'bs': 90907993304023.48, 'nmse': 90907993304023.48, 'mean': 3580898.96, 'PredMean': 3325939.97, 'time': 0.6, 'explained_variance_score': 0.7310672913237456, 'per': 2.66262}
	Best Parameters :: 1{'random_state': 42}
	BS:0.4399215076235617

		 ==> Best Reel Model r2:[('voting-Ridge30,Ridge20,Ridge10,Ridge5', 0.8335711696624738)]
		 ==> Best Reel Cv Score:[('Ridge30', 0.4399215076235617)]


voting between : Ridge30,Ridge20,Ridge10,Ridge5
{'Ridge30__random_state': [42], 'Ridge20__random_state': [42], 'Ridge10__random_state': [42], 'Ridge5__random_state': [42]}

voting-Ridge30,Ridge20,Ridge10,Ridge5 :: param grid :: {'Ridge30__random_state': [42], 'Ridge20__random_state': [42], 'Ridge10__random_state': [42], 'Ridge5__random_state': [42]}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-voting-ridge30,ridge20,ridge10,ridge5.r2:+0.83.p:2.09.webp?a=20201110-152221

	{'r2': 0.8335711696624738, 'sqrt': 7474095.0, 'bs': 55862090440201.21, 'nmse': 55862090440201.21, 'mean': 3580898.96, 'PredMean': 2576585.99, 'time': 0.6, 'explained_variance_score': 0.8335788720318964, 'per': 2.08721}


		 ==> best model r2 with voting is :[('voting-Ridge30,Ridge20,Ridge10,Ridge5', 0.8335711696624738)]
Best CV score
('Ridge30', 0.4399215076235617)
		Voting Classifier final r2:0.8335711696624738
[('r2s', 4704), ('pred', 368), ('resultsArray', 240), ('energy', 66), ('dfn', 56)]
saved in:0.055916547775268555
b'  adding: pendingModels-electricity-kbtu-.pickle (deflated 28%)\n'
('voting-Ridge30,Ridge20,Ridge10,Ridge5', 0.8335711696624738)
45.745742082595825


---
#####A) CrossValidation
---
- Retient les meilleurs paramètres de prédiction du jeu de test vis à vis du jeu de validation
- Ne sont pas les même résultats que précédemment : -0.45 sur le jeu de test réel avec meilleurs parametres :: {'alpha': 0.2, 'random_state': 42}
- 0.83 en mode voting - 0.76 avec rd:1, alpha:27
- On ne peut prédire les meilleurs paramètres vis à vis d'un jeu de données non connu, le plus de données on aura en entrée et le plus de chance d'être exact sera vrai
=> Autant voter entre différentes valeurs de modèles
---

In [ ]:
a=time();#Hyper Spécialisation
x1,y1,x2,y2,y_mean,mean=getSplitted('Electricity(kBtu)',dfNoEnergies)
#autant de combinaisons que de permutations possibles !!!
#param_grid={'Ridge':{'alpha':[0,0.01,0.1,0.2,1,3,5,10,20,30,60,90],'random_state':[1,2,3,4,5,6,7,8,9,10,42]}}
param_grid={'Ridge':{'alpha':[0,0.01,0.1,0.2,1,3,5,10,20,23,25,26,27,28,29,30,35,60,90],'random_state':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,42]}}
models={'Ridge':sklearn.linear_model.Ridge()}
#voted single ..
votingOnReel=0;voted=2;modelsEvalutation(reset=1);best=arsort(r2s[k1])[:1][0];print(best)
b='modelsNoParameters';t[b]=time()-a;print(t[b])

non shuffled
____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-Ridge{


Ridge :: param grid :: {'alpha': [0, 0.01, 0.1, 0.2, 1, 3, 5, 10, 20, 23, 25, 26, 27, 28, 29, 30, 35, 60, 90], 'random_state': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 42]}
Fitting 5 folds for each of 285 candidates, totalling 1425 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 660 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 1346 out of 1425 | elapsed:    7.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done 1425 out of 1425 | elapsed:    7.9s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-ridge.r2:+0.76.p:2.49.webp?a=20201110-152459

	{'r2': 0.7630264082119453, 'sqrt': 8918551.0, 'bs': 79540547088846.42, 'nmse': 79540547088846.42, 'mean': 3580898.96, 'PredMean': 3211197.24, 'time': 7.9, 'explained_variance_score': 0.7644262278659174, 'per': 2.49059}
	Best Parameters :: 2{'alpha': 27, 'random_state': 1}
	BS:0.4401337942206414

		 ==> Best Reel Model r2:[('Ridge', 0.7630264082119453)]
		 ==> Best Reel Cv Score:[('Ridge', 0.4401337942206414)]


voting between : Ridge
{'Ridge__alpha': [27], 'Ridge__random_state': [1]}

voting-Ridge :: param grid :: {'Ridge__alpha': [27], 'Ridge__random_state': [1]}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-voting-ridge.r2:+0.76.p:2.49.webp?a=20201110-152502

	{'r2': 0.7630264082119453, 'sqrt': 8918551.0, 'bs': 79540547088846.42, 'nmse': 79540547088846.42, 'mean': 3580898.96, 'PredMean': 3211197.24, 'time': 0.2, 'explained_variance_score': 0.7644262278659174, 'per': 2.49059}


		 ==> best model r2 with voting is :[('Ridge', 0.7630264082119453)]
Best CV score
('Ridge', 0.4401337942206414)
		Voting Classifier final r2:0.7630264082119453
[('r2s', 4704), ('pred', 648), ('resultsArray', 240), ('energy', 66), ('dfn', 56)]
saved in:0.058684587478637695
b'  adding: pendingModels-electricity-kbtu-.pickle (deflated 31%)\n'
('Ridge', 0.7630264082119453)
20.76740264892578


---
###### B) BR CV
---

In [ ]:
a=time();#Hyper Spécialisation
#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.BayesianRidge.html
param_grid={'BayesianRidge':{'n_iter':[1,5,10,100,300,1000],'tol':[1e-6,1e-3,1e-1,1,10,100,1000]}}#,'alpha_1':[1e-6],'alpha_2':[1e-6],'lambda_1':[1e-6],'lambda_2':[1e-6]
#for param in bestParameters[k2]:
models={'BayesianRidge':sklearn.linear_model.BayesianRidge()}
#voted single ..
#if((_ignoreParameters == 1) or (noPG)):
noPG=_ignoreParameters=0
list(dfNoEnergies.keys())
x1,y1,x2,y2,y_mean,mean=getSplitted('Electricity(kBtu)',dfNoEnergies)
votingOnReel=0;voted=2;modelsEvalutation(reset=1)

x1,y1,x2,y2,y_mean,mean=getSplitted('Electricity-KBtu/Gfa',dfNoEnergies)#as Globals
modelsEvalutation(reset=1,matchAgainst='Electricity(kBtu)');#'PropertyGFATotal'best=arsort(r2s[k1])[:1][0];print(best)

if False:#Totally out of Bounds !!
  modelsEvalutation(reset=1,multiplyPredicitions='PropertyGFATotal',matchAgainst='Electricity(kBtu)');
  b='br';t[b]=time()-a;print(t[b])

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-BayesianRidge{

multiplyPredicitions False
multiplyPredicitions False

BayesianRidge :: param grid :: {'n_iter': [1, 5, 10, 100, 300, 1000], 'tol': [1e-06, 0.001, 0.1, 1, 10, 100, 1000]}
Fitting 5 folds for each of 42 candidates, totalling 210 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:    2.7s finished


		matchedAgainst:Electricity(kBtu)
Keeping previous splitted data
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-bayesianridge.r2:+0.83.p:2.11.webp?a=20201110-172254

	{'r2': 0.8302161461523841, 'sqrt': 7549054.0, 'bs': 56988209192400.2, 'nmse': 56988209192400.2, 'mean': 3580898.96, 'PredMean': 3064226.37, 'time': 2.8, 'explained_variance_score': 0.8310800407622632, 'per': 2.10814}
	Best Parameters :: 2{'n_iter': 1, 'tol': 1e-06}
	BS:0.40232185885060134

		 ==> Best Reel Model r2:[('BayesianRidge', 0.8302161461523841)]
		 ==> Best Reel Cv Score:[('BayesianRidge', 0.40232185885060134)]


voting between : BayesianRidge
{'BayesianRidge__n_iter': [1], 'BayesianRidge__tol': [1e-06]}
multiplyPredicitions False

voting-BayesianRidge :: param grid :: {'BayesianRidge__n_iter': [1], 'BayesianRidge__tol': [1e-06]}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


		matchedAgainst:Electricity(kBtu)
Keeping previous splitted data
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-voting-bayesianridge.r2:+0.83.p:2.11.webp?a=20201110-172258

	{'r2': 0.8302161461523841, 'sqrt': 7549054.0, 'bs': 56988209192400.2, 'nmse': 56988209192400.2, 'mean': 3580898.96, 'PredMean': 3064226.37, 'time': 0.3, 'explained_variance_score': 0.8310800407622632, 'per': 2.10814}


		 ==> best model r2 with voting is :[('BayesianRidge', 0.8302161461523841)]
Best CV score
('BayesianRidge', 0.40232185885060134)
		Voting Classifier final r2:0.8302161461523841
[('r2s', 9320), ('pred', 368), ('resultsArray', 240), ('energy', 66), ('dfn', 56)]
saved in:0.09179520606994629
b'  adding: pendingModels-electricity-kbtu-.pickle (deflated 74%)\n'


---
####5) Vote aveugle sans parametres
---
- Est une bonne chose ! http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting.r2:+0.6.p:3.25.webp

In [ ]:
nbRdKeys=2;noPG=1;votingOnReel=0;models={}
toPredict='Electricity(kBtu),Electricity-KBtu/Gfa,SiteEnergyUseWN-KBtu/Gfa,Steam-KBtu/Gfa,GHGEmissions-Mt/Gfa,NaturalGas-KBtu/Gfa,OtherFuelUse-kBtu/Gfa,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),GHGEmissions(MetricTonsCO2e),GHGEmissionsIntensity(kgCO2e/ft2),SteamUse(kBtu),NaturalGas(kBtu),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu)'.split(',')

models['LinearRegression']=sklearn.linear_model.LinearRegression()  
models['Lasso']=sklearn.linear_model.Lasso(random_state=42)#Ridge less sensitive to outliers(abs)
models['ElasticNet']=sklearn.linear_model.ElasticNet(random_state=42)#=Ridge + Lasso Penalties with their own λ
models['ExtraTreesRegressor']=sklearn.ensemble.ExtraTreesRegressor(random_state=42)
models['RandomForestRegressor']=sklearn.ensemble.RandomForestRegressor(random_state=42)

a=time();voted=1;modelsEvalutation();
best=arsort(r2s[k1])[:1][0];print('_'*180);print(best)
b='votingWithoutParameterIs';t[b]=time()-a;print(t[b])
#('Ridge30', 0.8463011893002228) in 168.24609279632568 sec

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

		 ==> Best Reel Model r2:[('BayesianRidge', 0.8302161461523841)]
		 ==> Best Reel Cv Score:[]


voting between : LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor
{}
multiplyPredicitions False

voting-LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor-np :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   16.3s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-voting-linearregression,.r2:+0.6.p:3.25.webp?a=20201110-172902

	{'r2': 0.5962224604003759, 'sqrt': 11641673.0, 'bs': 135528546280663.64, 'nmse': 135528546280663.64, 'mean': 3580898.96, 'PredMean': 2079176.36, 'time': 34.8, 'explained_variance_score': 0.5968165858253218, 'per': 3.25105}


		 ==> best model r2 with voting is :[('BayesianRidge', 0.8302161461523841)]
		Voting Classifier final r2:0.5962224604003759
[('r2s', 9320), ('pred', 368), ('resultsArray', 240), ('energy', 66), ('dfn', 56)]
saved in:0.09278273582458496
b'  adding: pendingModels-electricity-kbtu-.pickle (deflated 64%)\n'
____________________________________________________________________________________________________________________________________________________________________________________
('BayesianRidge', 0.8302161461523841)
48.01541185379028


---
####6) Modèles sans paramètres estimés sur jeu de test réel
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-ridge20.r2:+0.83.p:2.1.webp #is cheating !
---

In [ ]:
nbRdKeys=2#minimal:range(1,2)=[1]
param_grid={};a=time();votingOnReel=1;
voted=0;modelsEvalutation();best=arsort(r2s[k1])[:1][0];print(best)
b='modelsNoParameters';t[b]=time()-a;print(t[b])

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-LinearRegression{

multiplyPredicitions False
multiplyPredicitions False
LinearRegression no within {'LinearRegression': {}}

LinearRegression-np :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-linearregression-np.r2:-1.31.p:7.78.webp?a=20201110-172916

	{'r2': -1.3098847061697918, 'sqrt': 27844503.0, 'bs': 775316320500510.5, 'nmse': 775316320500510.5, 'mean': 3580898.96, 'PredMean': -308424.76, 'time': 0.8, 'explained_variance_score': -1.2859536661025657, 'per': 7.77584}
	 no parameters..
	BS:0.2864134437514378

	}Electricity(kBtu)-trimmed-Lasso{

multiplyPredicitions False
multiplyPredicitions False
Lasso no within {'LinearRegression': {}, 'Lasso': {}}

Lasso-np :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-lasso-np.r2:-1.31.p:7.78.webp?a=20201110-172922

	{'r2': -1.314846747646826, 'sqrt': 27874394.0, 'bs': 776981837281443.0, 'nmse': 776981837281443.0, 'mean': 3580898.96, 'PredMean': -311940.39, 'time': 1.8, 'explained_variance_score': -1.2908563004324995, 'per': 7.78419}
	 no parameters..
	BS:0.2866859433494292

	}Electricity(kBtu)-trimmed-ElasticNet{

multiplyPredicitions False
multiplyPredicitions False
ElasticNet no within {'LinearRegression': {}, 'Lasso': {}, 'ElasticNet': {}}

ElasticNet-np :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-elasticnet-np.r2:-2.61.p:9.73.webp?a=20201110-172928

	{'r2': -2.614179603878726, 'sqrt': 34829655.0, 'bs': 1213104889877248.8, 'nmse': 1213104889877248.8, 'mean': 3580898.96, 'PredMean': 6579942.77, 'time': 2.0, 'explained_variance_score': -2.5652104834819127, 'per': 9.72651}
	 no parameters..
	BS:0.37901308098765024

	}Electricity(kBtu)-trimmed-ExtraTreesRegressor{

multiplyPredicitions False
multiplyPredicitions False
ExtraTreesRegressor no within {'LinearRegression': {}, 'Lasso': {}, 'ElasticNet': {}, 'ExtraTreesRegressor': {}}
range(1, 2)

ExtraTreesRegressor-np :: param grid :: {'random_state': range(1, 2)}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.1s finished


put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-extratreesregressor-np0.2.webp?a=20201110-172947
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-extratreesregressor-np.r2:+0.2.p:4.57.webp?a=20201110-172951

	{'r2': 0.2032249145140428, 'sqrt': 16353555.0, 'bs': 267438771249236.16, 'nmse': 267438771249236.16, 'mean': 3580898.96, 'PredMean': 2125010.73, 'time': 12.7, 'explained_variance_score': 0.20370333917953654, 'per': 4.56689}
	Best Parameters :: 1{'random_state': 1}
	BS:0.8596643436069714

	}Electricity(kBtu)-trimmed-RandomForestRegressor{

multiplyPredicitions False
multiplyPredicitions False
RandomForestRegressor no within {'LinearRegression': {}, 'Lasso': {}, 'ElasticNet': {}, 'ExtraTreesRegressor': {}, 'RandomForestRegressor': {}}
range(1, 2)

RandomForestRegressor-np :: param grid :: {'random_state': range(1, 2)}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.2s finished


put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-randomforestregressor-np0.19.webp?a=20201110-173016
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-randomforestregressor-np.r2:+0.19.p:4.6.webp?a=20201110-173020

	{'r2': 0.19245613617347745, 'sqrt': 16463697.0, 'bs': 271053328104386.38, 'nmse': 271053328104386.38, 'mean': 3580898.96, 'PredMean': 2275918.08, 'time': 19.9, 'explained_variance_score': 0.19264207604759898, 'per': 4.59764}
	Best Parameters :: 1{'random_state': 1}
	BS:0.7515940252918368

		 ==> Best Reel Model r2:[('BayesianRidge', 0.8302161461523841)]
		 ==> Best Reel Cv Score:[('ExtraTreesRegressor', 0.8596643436069714)]
no voting
('BayesianRidge', 0.8302161461523841)
68.42087697982788


####7) Modèles avec paramètres sur jeu test réel
---
    ==> Best Reel Model r2:[('Ridge20', 0.8308315043300891)] évidemment, is cheating !
    ==> Best Reel Cv Score:[('ExtraTreesRegressor', -7307477498056.606)] sera apparement le meilleur modèle pour prédictions => 0.21 sur jeu réel
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting.r2:+0.6.p:3.25.webp > supérieur sur jeu réel


In [ ]:
toPredict='Electricity(kBtu)'.split(',')
votingOnReel=1;#noPG=1#no Parameters Grid
#pred={};r2s={};bestPredictions={}#reset
param_grid={
'dummy':{'strategy':['mean','median']},
'LinearRegression':{'normalize':[True,False]},
'Ridge':{'alpha':[0.01,10],'random_state':[42]},#lambda
'Lasso':{'alpha':[0.01,10]},#random_state,max_iter
'ElasticNet':{'alpha':[0.0001,0.01,10]},#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html#sklearn.linear_model.ElasticNetCV
'RandomForestRegressor':{'n_estimators':[10,100],'random_state':[42]},
'ExtraTreesRegressor':{'n_estimators':[10,20,30,100,200],'random_state':[42]},
}
#ElasticNet', 0.84426018686573)
a=time();noPG=0;
voted=2;modelsEvalutation();best=arsort(r2s[k1])[:1][0];
print(bestParameters)
print('_'*180);print(best)
b='modelsOnlyWithParameters';t[b]=time()-a;
print(t[b])
votingOnReel=0;

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-LinearRegression{

multiplyPredicitions False
multiplyPredicitions False

LinearRegression :: param grid :: {'normalize': [True, False]}
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    0.8s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.8s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-linearregression.r2:-1.31.p:7.78.webp?a=20201110-173026

	{'r2': -1.3098847061697918, 'sqrt': 27844503.0, 'bs': 775316320500510.5, 'nmse': 775316320500510.5, 'mean': 3580898.96, 'PredMean': -308424.76, 'time': 0.9, 'explained_variance_score': -1.2859536661025657, 'per': 7.77584}
	Best Parameters :: 1{'normalize': False}
	BS:0.2864134437514378

	}Electricity(kBtu)-trimmed-Lasso{

multiplyPredicitions False
multiplyPredicitions False

Lasso :: param grid :: {'alpha': [0.01, 10]}
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    0.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.8s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-lasso.r2:-1.33.p:7.8.webp?a=20201110-173032

	{'r2': -1.3250682229816566, 'sqrt': 27935867.0, 'bs': 780412691048957.8, 'nmse': 780412691048957.8, 'mean': 3580898.96, 'PredMean': -319220.48, 'time': 1.4, 'explained_variance_score': -1.3009545225963843, 'per': 7.80136}
	Best Parameters :: 1{'alpha': 10}
	BS:0.2870824996857653

	}Electricity(kBtu)-trimmed-ElasticNet{

multiplyPredicitions False
multiplyPredicitions False

ElasticNet :: param grid :: {'alpha': [0.0001, 0.01, 10]}
Fitting 5 folds for each of 3 candidates, totalling 15 fits


...

put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-extratreesregressor0.2.webp?a=20201110-173057
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-extratreesregressor.r2:+0.2.p:4.59.webp?a=20201110-173101

	{'r2': 0.19673763545300127, 'sqrt': 16419995.0, 'bs': 269616236348785.16, 'nmse': 269616236348785.16, 'mean': 3580898.96, 'PredMean': 2115970.95, 'time': 13.5, 'explained_variance_score': 0.19723788850317714, 'per': 4.58544}
	Best Parameters :: 2{'n_estimators': 30, 'random_state': 42}
	BS:0.8704130183189276

	}Electricity(kBtu)-trimmed-RandomForestRegressor{

multiplyPredicitions False
multiplyPredicitions False

RandomForestRegressor :: param grid :: {'n_estimators': [10, 100], 'random_state': [42]}
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed:    8.7s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    8.8s finished


put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-randomforestregressor0.17.webp?a=20201110-173126
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-randomforestregressor.r2:+0.17.p:4.67.webp?a=20201110-173130

	{'r2': 0.167038160922513, 'sqrt': 16720793.0, 'bs': 279584910219074.9, 'nmse': 279584910219074.9, 'mean': 3580898.96, 'PredMean': 2305494.94, 'time': 19.4, 'explained_variance_score': 0.16718267955217903, 'per': 4.66944}
	Best Parameters :: 2{'n_estimators': 100, 'random_state': 42}
	BS:0.7413828822019501

		 ==> Best Reel Model r2:[('BayesianRidge', 0.8302161461523841)]
		 ==> Best Reel Cv Score:[('ExtraTreesRegressor', 0.8704130183189276)]


voting between : ElasticNet,ExtraTreesRegressor,RandomForestRegressor
{'ElasticNet__alpha': [0.01], 'ExtraTreesRegressor__n_estimators': [30], 'ExtraTreesRegressor__random_state': [42], 'RandomForestRegressor__n_estimators': [100], 'RandomForestRegressor__random_state': [42]}
multiplyP

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.1s finished


put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-voting-elasticnet,extratreesregressor,randomforestregressor.r2:+0.47.p:3.74.webp?a=20201110-173158

	{'r2': 0.46618026412214186, 'sqrt': 13385717.0, 'bs': 179177407567524.06, 'nmse': 179177407567524.06, 'mean': 3580898.96, 'PredMean': 2542351.25, 'time': 24.5, 'explained_variance_score': 0.4661810862028939, 'per': 3.73809}


		 ==> best model r2 with voting is :[('BayesianRidge', 0.8302161461523841)]
Best CV score
('ExtraTreesRegressor', 0.8704130183189276)
		Voting Classifier final r2:0.46618026412214186
[('r2s', 9320), ('pred', 648), ('resultsArray', 240), ('energy', 66), ('dfn', 56)]
saved in:0.0926210880279541
b'  adding: pendingModels-electricity-kbtu-.pickle (deflated 47%)\n'
{'Electricity(kBtu)-trimmed-BayesianRidge-np': {}, 'Electricity(kBtu)-trimmed-voting-BayesianRidge-np': {}, 'Electricity(kBtu)-trimmed-br-np': {}, 'Electricity(kBtu)-trimmed-voting-br-np': {}, 'Electricity(kBtu)-trimmed-BayesianRidge': {'

---
##### CV Sans Grille sans réel
- plus rapide ! 
---

In [ ]:
voted=0;noPG=1;b='modelsOnlyNoParamNotReel';
a=time();modelsEvalutation();best=arsort(r2s[k1])[:1][0];
print(bestParameters)
print('_'*180);print(best)
t[b]=time()-a;

SyntaxError: ignored

###### Best Scores

In [ ]:
arsort(bss)#best CV scoring per model on blind crosscv validation :)

[('ExtraTreesRegressor', -7273409881501.717),
 ('RandomForestRegressor', -15659337779148.855),
 ('Ridge02', -48117657941128.47),
 ('Ridge01', -48137434502514.05),
 ('Lasso', -48212725298528.37),
 ('Ridge1', -48214182352883.984),
 ('Ridge3', -48529187694608.55),
 ('Ridge5', -48823910709692.305),
 ('Ridge10', -49493088623796.016),
 ('Ridge20', -50523605044095.92),
 ('Ridge30', -51261513509743.48),
 ('ElasticNet', -56814404881633.42),
 ('LinearRegression', -63135109818437.29),
 ('dummy', -113820320359838.17)]

#### 8) GridSearchCV + params + voted only
- Excluant jeu test réel de validation
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting.r2:+0.6.p:3.25.webp

In [ ]:
votingOnReel=0;
toPredict='Electricity-KBtu/Gfa,Electricity(kBtu)'.split(',')
plt.rcParams["figure.figsize"] = (24,12)
nbRdKeysPerModel['RandomForestRegressor']=nbRdKeysPerModel['ExtraTreesRegressor']=2;#Prend plus de temps mais au moins ..
nbRdKeys=2;#limiter le nombre de clés aléatoires en présence d'autres paramètres ==> Mais pas bon pour extratreeregressor

a=time();b='singleThenVotedWithGrid';voted=1;#voted=2;models then voting
arretAnticipe=0;stopAtOne=1;modelsEvalutation();#- on peut donc passer le paramètre voting à 1 pour seulement opérer avec ce dernier ( mais on perd les paramètres gridcv ou il faudrait les ré-écrire .. )
best=arsort(r2s[k1])[:1][0];bestPredictions[k1+'-'+best[0]]=pred[k1+'-'+best[0]];print('_'*180);print(best)#0.72 without voting_grid parameters
t[b]=time()-a;print(t[b])

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity-KBtu/Gfa{
0    41.682611
1    37.709393
Name: Electricity-KBtu/Gfa, dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity-KBtu/Gfa-trimmed{
Keeping previous splitted data

	}Electricity-KBtu/Gfa-trimmed-dummy{


dummy :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-dummy.r2:-0.01.p:0.99.webp?a=20200117180205

	{'r2': -0.010783767705889957, 'sqrt': 33.0, 'bs': 1120.774125621514, 'nmse': 1120.774125621514, 'mean': 33.27, 'PredMean': 33.97, 'time': 0.3, 'explained_variance_score': -2.220446049250313e-16, 'per': 0.99177}
	 no parameters..
	BS:-1310.5329538536166

		 ==> Best Reel Model r2:[('Ridge5', 0.9711907383429009)]
		 ==> Best Reel Cv Score:[('dummy', -1310.5329538536166)]


voting between : LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor,HistGradientBoostingRegressor,GradientBoostingRegressor,KNeighborsRegressor,HuberRegressor,Lars,LassoLars,BayesianRidge,LinearSVR,OrthogonalMatchingPursuit,SVR,StackingRegressor,KernelRidge,PassiveAggressiveRegressor,MLPRegressor,Ridge,XGBRegressor,AdaBoostRegressor
{'Lasso__alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Socket exception: Connection reset by peer (104)
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 11.1min


######A) Faster without parameters
---
- Voting, puis multiplication par PropertyGFATotal : <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-voting-m:propertygfatotal-a:electricity-kbtu--ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor-np.r2:+0.95.p:1.17.webp'> => 0.95 ( Meilleur à l'aveugle )</a>
- <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-voting-m:propertygfatotal-a:electricity-kbtu--dummy-ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor-np.r2:+0.96.p:1.0.webp'>+1 point avec dummy que nous ne sommes pas sensés utiliser</a>

In [ ]:
toPredict='Electricity-KBtu/Gfa'.split(',');voted=1;votingOnReel=0;noPG=1;
modelsEvalutation(multiplyPredicitions='PropertyGFATotal',matchAgainst='Electricity(kBtu)');#- on peut donc passer le paramètre voting à 1 pour seulement opérer avec ce dernier ( mais on perd les paramètres gridcv ou il faudrait les ré-écrire .. )

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity-KBtu/Gfa{
0    41.682611
1    37.709393
Name: Electricity-KBtu/Gfa, dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity-KBtu/Gfa-trimmed{
Keeping previous splitted data

	}Electricity-KBtu/Gfa-trimmed-dummy{


dummy-np :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits

		predictions multiplied by : PropertyGFATotal
		matchedAgainst:Electricity(kBtu)
Keeping previous splitted data


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-dummy-m:propertygfatotal-a:electricity-kbtu--np.r2:+0.71.p:2.76.webp?a=20200117122419

	{'r2': 0.709322182577547, 'sqrt': 9877573.0, 'bs': 97566452235962.58, 'nmse': 97566452235962.58, 'mean': 3580898.96, 'PredMean': 2550873.2, 'time': 0.1, 'explained_variance_score': 0.7093240645183889, 'per': 2.75841}
	 no parameters..
	BS:-1310.5329538536166

		 ==> Best Reel Model r2:[('Ridge5', 0.9711907383429009)]
		 ==> Best Reel Cv Score:[('dummy', -1310.5329538536166)]


voting between : Ridge01,Ridge02,Ridge1,Ridge3,Ridge5,Ridge10,Ridge20,Ridge30,LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor
{}

voting-Ridge01,Ridge02,Ridge1,Ridge3,Ridge5,Ridge10,Ridge20,Ridge30,LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor-np :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   38.6s finished



		predictions multiplied by : PropertyGFATotal
		matchedAgainst:Electricity(kBtu)
Keeping previous splitted data
put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-voting-m:propertygfatotal-a:electricity-kbtu--ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor-np.r2:+0.95.p:1.17.webp?a=20200117122523

	{'r2': 0.9472608361162898, 'sqrt': 4207372.0, 'bs': 17701980700324.195, 'nmse': 17701980700324.195, 'mean': 3580898.96, 'PredMean': 2809568.7, 'time': 57.9, 'explained_variance_score': 0.9475008431196463, 'per': 1.17495}


		 ==> best model r2 with voting is :[('Ridge5', 0.9711907383429009)]
		Voting Classifier final r2:0.9472608361162898
[('pred', 2280), ('resultsArray', 648), ('r2s', 240), ('energy', 69), ('dfn', 56)]
saved in:0.060312509536743164
b'  adding: pendingModels-electricity-kbtu-gfa.pickle (deflated 52%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-electricity-

##### B) avec Parameters
- Cela est long et fournit un point de plus => <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-voting-m:propertygfatotal-a:electricity-kbtu--dummy-ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor.r2:+0.97.p:0.95.webp'>0.97</a>

In [ ]:
toPredict='Electricity-KBtu/Gfa'.split(',');voted=1;votingOnReel=0;
#multiplie le nombre de paramètres par candidat de manière exponentielle en fittant des paramètres non appropriés pour chaque ..
modelsEvalutation(multiplyPredicitions='PropertyGFATotal',matchAgainst='Electricity(kBtu)');#- on peut donc passer le paramètre voting à 1 pour seulement opérer avec ce dernier ( mais on perd les paramètres gridcv ou il faudrait les ré-écrire .. )

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity-KBtu/Gfa{
0    41.682611
1    37.709393
Name: Electricity-KBtu/Gfa, dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity-KBtu/Gfa-trimmed{
Keeping previous splitted data

		 ==> Best Reel Model r2:[('Ridge5', 0.9711907383429009)]
		 ==> Best Reel Cv Score:[]


voting between : dummy,Ridge01,Ridge02,Ridge1,Ridge3,Ridge5,Ridge10,Ridge20,Ridge30,LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor
{'dummy__strategy': ['mean', 'median'], 'LinearRegression__normalize': [True, False], 'Lasso__alpha': [0.01, 10], 'ElasticNet__alpha': [0.0001, 0.01, 10], 'ExtraTreesRegressor__n_estimators': [10, 20, 30, 100, 200], 'Ra

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 27.2min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 49.6min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 75.4min finished



		predictions multiplied by : PropertyGFATotal
		matchedAgainst:Electricity(kBtu)
Keeping previous splitted data
put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-voting-m:propertygfatotal-a:electricity-kbtu--dummy-ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor.r2:+0.97.p:0.95.webp?a=20200117110442

	{'r2': 0.9655162327346283, 'sqrt': 3402136.0, 'bs': 11574529015137.223, 'nmse': 11574529015137.223, 'mean': 3580898.96, 'PredMean': 2550413.15, 'time': 4535.6, 'explained_variance_score': 0.9655180464104272, 'per': 0.95008}


		 ==> best model r2 with voting is :[('Ridge5', 0.9711907383429009)]
		Voting Classifier final r2:0.9655162327346283
[('pred', 1184), ('resultsArray', 648), ('r2s', 240), ('energy', 69), ('dfn', 56)]
saved in:0.056710243225097656
b'  adding: pendingModels-electricity-kbtu-gfa.pickle (deflated 58%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-electr

---
####9) => Voted vs Models basé sur score gridCV et non r2 !
- http://1.x24.fr/a/jupyter/seattle/GridCV-BestvsVoted-electricity-kbtu-.webp #voted proche de extratrees individuel
- http://1.x24.fr/a/jupyter/seattle/GridCV-4BestModels-electricity-kbtu-.webp
---

In [ ]:
k1='Electricity(kBtu)-trimmed'
r2sSorted=arsort(bss)
ok=[];

for i in range(0,2):
  if(k1+'-'+r2sSorted[i][0] not in pred.keys()):
    print('missing:'+r2sSorted[i][0]+'=> deleting')
    del r2sSorted[i]
  else:
    ok+=[r2sSorted[i]]

p1=pred[k1+'-'+r2sSorted[0][0]]
p2=pred[k1+'-'+r2sSorted[1][0]]
p3=pred[k1+'-'+r2sSorted[2][0]]
p4=pred[k1+'-'+r2sSorted[3][0]]
pv=pred[voted]
voted=lastVotingKey;

ziped=zip(ElectKbtu_y_mean,y2,p1,p2,p3,p4,pv)
_df=pd.DataFrame(list(ziped),columns=['Mean','Real',r2sSorted[0][0],r2sSorted[1][0],r2sSorted[2][0],r2sSorted[3][0],'voted'])

fn='GridCV-BestvsVoted-'+filename(energy)+'.png'
f=_df[['Mean','Real',r2sSorted[0][0],'voted']].plot(alpha=0.4,style=['k-','bo','r^','g*','m>','cx'],title='Données Réelles vs prédictions '+energy);plt.yscale('log');
f.get_figure().savefig(fn);webp(fn);#show();

fn='GridCV-4BestModels-'+filename(energy)+'.png'
f=_df[['Mean','Real',r2sSorted[0][0],r2sSorted[1][0],r2sSorted[2][0],r2sSorted[3][0]]].plot(alpha=0.4,style=['k-','bo','r^','g*','m>','cx'],title='Données Réelles vs prédictions '+energy);  
plt.yscale('log');f.get_figure().savefig(fn);webp(fn);#show();

missing:XGBRegressor=> deleting
put:http://1.x24.fr/a/jupyter/seattle/GridCV-BestvsVoted-electricity-kbtu-.webp?a=20200117115505
put:http://1.x24.fr/a/jupyter/seattle/GridCV-4BestModels-electricity-kbtu-.webp?a=20200117115512


# # -#-------------------------

---
###A) Features Importance
---
- In a decision tree avec evaluation GridCV des paramètres
- http://1.x24.fr/a/jupyter/seattle/#q=FeatImportance
- <a href='http://1.x24.fr/a/jupyter/seattle/FeatImportance-electricity-kbtu--trimmed-extratreesregressor0.21.webp'>Extratrees : on voit très clairement PropertyGFATotal, nbfloors comme premier critère ( retenu par scoring gridsearchcv )</a>

In [ ]:
toPredict='Electricity(kBtu)'.split(',')
param_grid={
'XGBRegressor':{'reg':['squarederror'],'objective':['reg:squarederror'],'learning_rate':[0.01,0.1,0.3,1],'random_state':[42]},
'RandomForestRegressor':{'n_estimators':[10,100,300,600],'random_state':[42]},
'ExtraTreesRegressor':{'n_estimators':[10,100,300,600],'random_state':[42]},
'AdaBoostRegressor':{'n_estimators':[10,600],'learning_rate':[0.01,10],'loss':['linear','square'],'random_state':[42]},    
'GradientBoostingRegressor':{'learning_rate':[0.001,0.01,0.1,1,2],'random_state':[42]},#like adaboost
}
nbRdKeys=2;#cela va prendre bcp de temps : réduire le nb de clés aléatoires
models={}
models['XGBRegressor']=xgboost.XGBRegressor()#~has importance metrics ;) -> when not voted
models['RandomForestRegressor']=sklearn.ensemble.RandomForestRegressor()#~has importance metrics ;) -> when not voted
models['ExtraTreesRegressor']=sklearn.ensemble.ExtraTreesRegressor()
models['AdaBoostRegressor']=sklearn.ensemble.AdaBoostRegressor()#trunks=Stumps
models['GradientBoostingRegressor']=sklearn.ensemble.GradientBoostingRegressor()  

noPG=0;voted=0;modelsEvalutation();
best=arsort(r2s[k1])[:1][0];print(best)

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-XGBRegressor{

multiplyPredicitions False
multiplyPredicitions False

XGBRegressor :: param grid :: {'reg': ['squarederror'], 'objective': ['reg:squarederror'], 'learning_rate': [0.01, 0.1, 0.3, 1], 'random_state': [42]}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  20 | elapsed:    3.0s remaining:   12.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.3s finished


put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-xgbregressor0.17.webp?a=20201110-174519
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-xgbregressor.r2:+0.17.p:4.65.webp?a=20201110-174523

	{'r2': 0.17254981814157255, 'sqrt': 16665381.0, 'bs': 277734914077047.9, 'nmse': 277734914077047.9, 'mean': 3580898.96, 'PredMean': 2007913.9, 'time': 4.8, 'explained_variance_score': 0.17334869386287743, 'per': 4.65397}
	Best Parameters :: 4{'learning_rate': 0.3, 'objective': 'reg:squarederror', 'random_state': 42, 'reg': 'squarederror'}
	BS:0.7660493311863784

	}Electricity(kBtu)-trimmed-RandomForestRegressor{

multiplyPredicitions False
multiplyPredicitions False

RandomForestRegressor :: param grid :: {'n_estimators': [10, 100, 300, 600], 'random_state': [42]}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  20 | elapsed:    1.6s remaining:    6.2s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   52.9s finished


put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-randomforestregressor0.18.webp?a=20201110-174726
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-randomforestregressor.r2:+0.18.p:4.64.webp?a=20201110-174730

	{'r2': 0.17816843115371717, 'sqrt': 16608703.0, 'bs': 275849018060135.62, 'nmse': 275849018060135.62, 'mean': 3580898.96, 'PredMean': 2304642.69, 'time': 116.6, 'explained_variance_score': 0.17831407039100833, 'per': 4.63814}
	Best Parameters :: 2{'n_estimators': 600, 'random_state': 42}
	BS:0.7553551639655977

	}Electricity(kBtu)-trimmed-ExtraTreesRegressor{

multiplyPredicitions False
multiplyPredicitions False

ExtraTreesRegressor :: param grid :: {'n_estimators': [10, 100, 300, 600], 'random_state': [42]}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-extratreesregressor0.2.webp?a=20201110-174816
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-extratreesregressor.r2:+0.2.p:4.57.webp?a=20201110-174819

	{'r2': 0.20370754912464373, 'sqrt': 16348602.0, 'bs': 267276774206943.16, 'nmse': 267276774206943.16, 'mean': 3580898.96, 'PredMean': 2130809.27, 'time': 40.2, 'explained_variance_score': 0.20417222838325322, 'per': 4.5655}
	Best Parameters :: 2{'n_estimators': 100, 'random_state': 42}
	BS:0.8653852790065031

	}Electricity(kBtu)-trimmed-AdaBoostRegressor{

multiplyPredicitions False
multiplyPredicitions False

AdaBoostRegressor :: param grid :: {'n_estimators': [10, 600], 'learning_rate': [0.01, 10], 'loss': ['linear', 'square'], 'random_state': [42]}
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  40 | elapsed:    1.4s remaining:   26.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   49.0s finished


put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-adaboostregressor0.09.webp?a=20201110-174915
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-adaboostregressor.r2:+0.09.p:4.88.webp?a=20201110-174919

	{'r2': 0.0919936860843793, 'sqrt': 17457769.0, 'bs': 304773702521125.9, 'nmse': 304773702521125.9, 'mean': 3580898.96, 'PredMean': 2624707.35, 'time': 49.9, 'explained_variance_score': 0.09202286674101556, 'per': 4.87525}
	Best Parameters :: 4{'learning_rate': 0.01, 'loss': 'square', 'n_estimators': 10, 'random_state': 42}
	BS:0.5634290181025725

	}Electricity(kBtu)-trimmed-GradientBoostingRegressor{

multiplyPredicitions False
multiplyPredicitions False

GradientBoostingRegressor :: param grid :: {'learning_rate': [0.001, 0.01, 0.1, 1, 2], 'random_state': [42]}
Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.1s finished


put : http://1.x24.fr/a/jupyter/seattle3//FeatImportance-electricity-kbtu--trimmed-gradientboostingregressor0.18.webp?a=20201110-174930
put : http://1.x24.fr/a/jupyter/seattle3//sp-electricity-kbtu--trimmed-gradientboostingregressor.r2:+0.18.p:4.64.webp?a=20201110-174934

	{'r2': 0.177616343564584, 'sqrt': 16614281.0, 'bs': 276034327100477.47, 'nmse': 276034327100477.47, 'mean': 3580898.96, 'PredMean': 2207559.76, 'time': 5.8, 'explained_variance_score': 0.17791796197589071, 'per': 4.6397}
	Best Parameters :: 2{'learning_rate': 0.1, 'random_state': 42}
	BS:0.7650332489600442

		 ==> Best Reel Model r2:[('BayesianRidge', 0.8302161461523841)]
		 ==> Best Reel Cv Score:[('ExtraTreesRegressor', 0.8653852790065031)]
no voting
('BayesianRidge', 0.8302161461523841)


###B) Mesures de Performances


In [ ]:
print(arsort(t))

[('modelsOnlyNoParamNotReel', 233.06974077224731), ('modelsOnlyWithParameters', 229.3119204044342), ('modelsNoParameters', 166.36594247817993), ('votingWithoutParameterIs', 73.88316679000854), ('singleThenVotedWithGrid', 73.63651752471924)]


####1) Hyper-paramètres ++
---
- Learning_rate ou alpha ou n_estimators pour les trees
- Best model r2 with voting is : voting, 0.65, moins qu'un modèle choisi individuellement, mais quelles sont les composantes principales ???
- 'BayesianRidge', 0.84 - valeurs restant dans la "norme" >> Il suffit parfois d'une unique valeur pour exploser les carrés résiduels
- Attention, parfois les paramètres par défaut retournent de meilleurs valeurs

In [ ]:
nbCols=dfNoEnergies.shape[1]
bigParamGrid= {  
  'SVR':{'kernel':['rbf'],'degree':[3],'gamma':['scale'],'coef0':[0.0],'tol':[0.001],'C':[1.0],'epsilon':[0.1],'shrinking':[True],'cache_size':[200],'verbose':[False],'max_iter':[-1]},#rbf,'poly=> too much time','linear','sigmoid'
  'LinearSVR':{'random_state':[=0,42],'tol':[0.0001],'C':[1,100,400],'epsilon':[0,1,10],'loss':['epsilon_insensitive'],'fit_intercept':[True],'intercept_scaling':[1],'dual':[True],'verbose':[0],'random_state':[None],'max_iter':[1000]},#'C': 100, 'dual': True, 'epsilon': 1
  'OrthogonalMatchingPursuit':{'n_nonzero_coefs':[None,round(nbCols*0.1),round(nbCols*0.2),round(nbCols*0.9)],'precompute':[True,False]},#
  'Lars':{'n_nonzero_coefs':[1,500,np.inf]},
  'KernelRidge':{'alpha':[0.0001,0.01,1]},#,10 :: too much time !!!
  'LassoLars':{'alpha':[0.0001,0.01,1]},#,10
  'PassiveAggressiveRegressor':{'max_iter':[10,100,300,500]},
  'HuberRegressor':{'alpha':[0.0001,0.01,1,10],'max_iter':[10,100,300,500],'epsilon':[1.35],'tol':[0.00001]},
  'HistGradientBoostingRegressor':{'learning_rate':[0.0001,0.01,1,2]},#(loss='ls', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, alpha=0.9, verbose=0, max_leaf_nodes=None, warm_start=False, presort='deprecated', validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)
  'StackingRegressor':{'cv':[30,60]},#estimators,final_estimator,cv:1,5,30
  
'ElasticNet':{'alpha':[0.0001,0.01,1,10]},#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html#sklearn.linear_model.ElasticNetCV  
'ExtraTreesRegressor':{'n_estimators':[10,100,300]},
'RandomForestRegressor':{'n_estimators':[10,100,300]},

'AdaBoostRegressor':{'n_estimators':[10,600],'learning_rate':[0.01,10],'loss':['linear','square']},    
'BaggingRegressor':{'n_estimators':[100],'max_samples':[20],'max_features':[200]},#base_estimator:DecisionTree
'MLPRegressor':{'activation':['relu'],'hidden_layer_sizes':[(100,)],'solver':['adam'],'learning_rate':['constant','adaptive'],'learning_rate_init':[0.1,0.001]},#constant
'dummy':{'strategy':['mean','median']},#quantile,constant
'KNeighborsRegressor':{'n_neighbors':[1,10,21]},#list(range(1, 21))},
'XGBRegressor':{'reg':['squarederror'],'learning_rate':[0.01,0.1,0.3,1],'objective':['reg:squarederror']} ,#0.10, 0.15, 0.20, 0.25, },
'GaussianNB':{'var_smoothing':[0.1,0.000001]},#classifier
'GradientBoostingRegressor':{'learning_rate':[0.001,0.01,0.1,1,2]},#like adaboost
#ValueError: Invalid parameter verbose for estimator AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',n_estimators=50, random_state=None). Check the list of available parameters with `estimator.get_params().keys()`.
'Ridge':{'alpha':[0.1,1,10,30]},#lambda,_ridge UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.

'Lasso':{'alpha': np.logspace(-4, -0.5, 30)},#random_state,max_iter
'BayesianRidge':{'lambda_1':[0.001,0.000001]},
'LinearRegression':{'normalize':[True,False]},
'SGDRegressor':{'max_iter':[50000],'verbose':[1],'loss':['huber'],'learning_rate':['optimal'],'eta0':[0.1],'power_t':[0.35],'alpha':[0.2]},#learning rate
#(loss='squared_loss', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, random_state=None, learning_rate='invscaling', eta0=0.01, power_t=0.25, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, warm_start=False, average=False)  
#(base_score=0.5, booster='gbtree', colsample_bylevel=1,colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,max_depth=3, min_child_weight=1, missing=None, n_estimators=100,n_jobs=1, nthread=None, objective='reg:linear', random_state=0,reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,silent=True, subsample=1)  
'XGBRegressor2':{
#XGBRegressor ValueError: Parameter values for parameter (objective) reg:squarederror need to be a sequence(but not a string) or np.ndarray.      
#'objective':'reg:squarederror','verbose':[100] 
'learning_rate':    [0.05, 0.30] ,#0.10, 0.15, 0.20, 0.25, 
'max_depth'        : [3, 15],#4, 5, 6, 8, 10, 12, 
'min_child_weight' : [1, 7],#3, 5, 
#ValueError: Invalid parameter colsample_bytree for estimator GaussianNB(priors=None, var_smoothing=1e-09). Check the list of available parameters with `estimator.get_params().keys()`. 
'gamma'            : [0.0,  0.4],#0.1, 0.2 , 0.3,
'colsample_bytree' : [0.3,  0.7],#0.4, 0.5 ,
}
}

param_grid=bigParamGrid

####2) Ajout autres modèles
---
- Meilleurs modèles de prédiction de la consommation électrique directs : d'après retour r²
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-passiveaggressiveregressor-np-np.r2:+0.94.p:1.26.webp #<<<depends on random_states !!!
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-ridge20.r2:+0.83.p:2.1.webp
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-mlpregressor.r2:+0.83.p:2.09.webp
- http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting.r2:+0.81.p:2.23.webp
---
- A l'aveugle : <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-extratreesregressor.r2:+0.2.p:4.56.webp'>Voting 0.2</a>


In [ ]:
toPredict='Electricity(kBtu)'.split(',')
param_grid['LinearSVR']={'random_state':[0,42],'tol':[0.0001],'C':[1,100,400],'epsilon':[0,1,10],'loss':['epsilon_insensitive'],'fit_intercept':[True],'intercept_scaling':[1],'dual':[True],'verbose':[0],'random_state':[None],'max_iter':[1000]}
models=allModels1.copy()
ignoreParameters=1;voted=2;votingOnReel=0;noPG=0;#d'abord évalués individuellement, puis votés !
modelsEvalutation();
print('_'*180)
print('Best R² score')
best=arsort(r2s[k1])[:1][0];print(best)
print('Best CV score')
best=arsort(bss)[:1][0];print(best)
#print(bestNP)

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-dummy{


dummy :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-dummy.r2:-0.01.p:5.13.webp?a=20200117133838

	{'r2': -0.0051843761288563606, 'sqrt': 18368224.0, 'bs': 337391667143901.06, 'nmse': 337391667143901.06, 'mean': 3580898.96, 'PredMean': 3844885.11, 'time': 0.1, 'explained_variance_score': 0.0, 'per': 5.1295}
	 no parameters..
	BS:-113820320359838.17

	}Electricity(kBtu)-trimmed-LinearRegression{


LinearRegression :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-linearregression.r2:-1.39.p:7.91.webp?a=20200117133845

	{'r2': -1.3894035520853008, 'sqrt': 28319727.0, 'bs': 802006942270932.1, 'nmse': 802006942270932.1, 'mean': 3580898.96, 'PredMean': -359955.39, 'time': 0.3, 'explained_variance_score': -1.364594375223616, 'per': 7.90855}
	 no parameters..
	BS:-63135109818437.29

	}Electricity(kBtu)-trimmed-Lasso{


Lasso :: param grid :: {'alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564216e-04,
       9.23670857e-04, 1.21957046e-03, 1.61026203e-03, 2.12611233e-03,
       2.80721620e-03, 3.70651291e-03, 4.89390092e-03, 6.46167079e-03,
       8.53167852e-03, 1.12648169e-02, 1.48735211e-02, 1.96382800e-02,
       2.59294380e-02, 3.42359796e-02, 4.52035366e-02, 5.96845700e-02,
       7.88046282e-02, 1.04049831e-01, 1.37382380e-01, 1.81393069e-01,
       2.39502662e-01, 3.16227766e-01])}
Fitti

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.9s finished


put:http://1.x24.fr/a/jupyter/seattle/FeatImportance-electricity-kbtu--trimmed-adaboostregressor-np-0.46.webp?a=20200117134944
put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-adaboostregressor-np.r2:-0.46.p:6.19.webp?a=20200117134951

	{'r2': -0.46337816580306246, 'sqrt': 22162696.0, 'bs': 491185110659725.3, 'nmse': 491185110659725.3, 'mean': 3580898.96, 'PredMean': 16882500.7, 'time': 8.4, 'explained_variance_score': 0.1507009092023479, 'per': 6.18914}


		==> Best results without parameters
	BS:-79625678067086.6

		 ==> Best Reel Model r2:[('Ridge20', 0.8308315043300891)]
		 ==> Best Reel Cv Score:[('ExtraTreesRegressor', -7455718636613.052)]


voting between : ExtraTreesRegressor,GradientBoostingRegressor,XGBRegressor,RandomForestRegressor
{'ExtraTreesRegressor__random_state': [1], 'GradientBoostingRegressor__random_state': [1], 'XGBRegressor__learning_rate': [0.3], 'XGBRegressor__objective': ['reg:squarederror'], 'XGBRegressor__random_state': [1], 'XGBRegressor__

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   42.8s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting-extratreesregressor-gradientboostingregressor-xgbregressor-randomforestregressor.r2:+0.19.p:4.6.webp?a=20200117135101

	{'r2': 0.19307796315516434, 'sqrt': 16457357.0, 'bs': 270844610930692.94, 'nmse': 270844610930692.94, 'mean': 3580898.96, 'PredMean': 2155258.99, 'time': 63.7, 'explained_variance_score': 0.1934868878613214, 'per': 4.59587}


		 ==> best model r2 with voting is :[('Ridge20', 0.8308315043300891)]
		Voting Classifier final r2:0.19307796315516434
[('r2s', 4704), ('pred', 4704), ('resultsArray', 648), ('energy', 66), ('dfn', 56)]
saved in:0.11338496208190918
b'  adding: pendingModels-electricity-kbtu-.pickle (deflated 51%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-electricity-kbtu-.pickle.zip?a=20200117135108
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-electricity-kbtu-.20200117-1351.pickle.zip?a=20200117135109
[('r2s', 4704), ('pred', 4704), ('resultsArray', 648)]
saved in:0.11

KeyError: ignored

In [ ]:
print('Best R² score')
best=arsort(r2s[k1])[:1][0];print(best)
print('Best CV score')
best=arsort(bss)[:1][0];print(best)

Best R² score
('Ridge20', 0.8308315043300891)
Best CV score
('ExtraTreesRegressor', -7455718636613.052)


In [ ]:
%%script false
if False:#Timing ou performances très mauvaises
  models['GaussianProcessRegressor']=sklearn.gaussian_process.GaussianProcessRegressor()
  models['SGDRegressor']=sklearn.linear_model.SGDRegressor()
  models['BaggingRegressor1-xtraTrees']=sklearn.ensemble.BaggingRegressor(sklearn.ensemble.ExtraTreesRegressor())#,max_samples=0.5, max_features=0.5
  models['BaggingRegressor2-randomForest']=sklearn.ensemble.BaggingRegressor(sklearn.ensemble.RandomForestRegressor())
#Classifiers
  voted=2;modelsEvalutation();best=arsort(r2s[k1])[:1][0];print(best)
  models['SVC']=SVC()#is linear or rbf 183 : une succession de zéros également ..    
  models['GaussianNB']=GaussianNB()#Naive Bayes : intersection de probabilités ( King vs Face )'GaussianNB': array([0, 0, 0, ..., 0, 0, 0]),  
  models['ExtraTreesClassifier']=ExtraTreesClassifier()
  models['BaggingClassifier']=BaggingClassifier()
  models['KNeighborsClassifier']=KNeighborsClassifier()
  models['RandomForestClassifier']=RandomForestClassifier()
  models['DecisionTreeClassifier']=DecisionTreeClassifier()
  #100 => not enough Ram ? Huh .. :(
  #models['RandomForestClassifier100']=RandomForestClassifier(n_estimators=100);      
  models['AdaBoostClassifier']=AdaBoostClassifier()  
  models['Perceptron']=Perceptron()    
#RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
#models['LinearSVC']=LinearSVC()#is linear separator SVM
#MLPClassifier()
  models['SGDClassifier']=SGDClassifier()#squared loss
  models['GradientBoostingClassifier']=GradientBoostingClassifier()  
  models['LogisticRegression']=sklearn.linear_model.LogisticRegression(multi_class='auto',solver='saga',penalty='l1',C=5,max_iter=1000)#S shaped > Classifies Obese or Not
#save(globals(),[],'export2');globals().update(resume('export2'))

____________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu){
0    3686160.0
1    3905411.0
Name: Electricity(kBtu), dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}Electricity(kBtu)-trimmed{
Keeping previous splitted data

	}Electricity(kBtu)-trimmed-HistGradientBoostingRegressor{


HistGradientBoostingRegressor :: param grid :: {'learning_rate': [0.0001, 0.01, 1, 2], 'random_state': [1, 2, 3, 12, 24, 42, 68]}
Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   37.1s
[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:  2.0min finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-histgradientboostingregressor.r2:+0.16.p:4.69.webp?a=20200116132417

	{'r2': 0.15892814983864345, 'sqrt': 16801995.0, 'nmse': 282307047794153.5, 'mean': 3580898.96, 'PredMean': 2104059.68, 'time': 119.5, 'explained_variance_score': 0.15945790854425512, 'per': 4.69212}
	Best Parameters :: 2{'learning_rate': 1, 'random_state': 1}

HistGradientBoostingRegressor-np :: param grid :: {'random_state': [1, 2, 3, 12, 24, 42, 68, 1983]}
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   34.2s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-histgradientboostingregressor-np.r2:+0.21.p:4.55.webp?a=20200116132457

	{'r2': 0.2107242969888885, 'sqrt': 16276412.0, 'nmse': 264921592097007.06, 'mean': 3580898.96, 'PredMean': 2142628.74, 'time': 36.2, 'explained_variance_score': 0.21116157865882512, 'per': 4.54534}


		==> Best results without parameters

	}Electricity(kBtu)-trimmed-GradientBoostingRegressor{


GradientBoostingRegressor :: param grid :: {'learning_rate': [0.001, 0.01, 0.1, 1, 2], 'random_state': [1, 2, 3, 12, 24, 42, 68]}
Fitting 5 folds for each of 35 candidates, totalling 175 fits


...

put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-ridge.r2:+0.8.p:2.3.webp?a=20200116134215

	{'r2': 0.7986664490203493, 'sqrt': 8220579.0, 'nmse': 67577913097525.18, 'mean': 3580898.96, 'PredMean': 2270707.24, 'time': 3.7, 'explained_variance_score': 0.798860226545842, 'per': 2.29567}
	Best Parameters :: 2{'alpha': 10, 'random_state': 1}

Ridge-np :: param grid :: {'random_state': [1, 2, 3, 12, 24, 42, 68, 1983]}
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.1s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-ridge-np.r2:+0.29.p:4.3.webp?a=20200116134219

	{'r2': 0.2928024247414227, 'sqrt': 15406880.0, 'nmse': 237371943479182.1, 'mean': 3580898.96, 'PredMean': 1169457.4, 'time': 1.1, 'explained_variance_score': 0.29828281862873296, 'per': 4.30252}

		 ==> best model r2:[('MLPRegressor', 0.8342480877702528)]


voting between : MLPRegressor,BayesianRidge,Ridge,OrthogonalMatchingPursuit
{'MLPRegressor__activation': ['relu'], 'MLPRegressor__hidden_layer_sizes': [(100,)], 'MLPRegressor__learning_rate': ['constant'], 'MLPRegressor__learning_rate_init': [0.001], 'MLPRegressor__random_state': [3], 'MLPRegressor__solver': ['adam'], 'BayesianRidge__lambda_1': [0.001], 'Ridge__alpha': [10], 'Ridge__random_state': [1], 'OrthogonalMatchingPursuit__n_nonzero_coefs': [11], 'OrthogonalMatchingPursuit__precompute': [True]}

voting-MLPRegressor,BayesianRidge,Ridge,OrthogonalMatchingPursuit :: param grid :: {'MLPRegressor__activation': ['relu'

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.2s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting.r2:+0.81.p:2.23.webp?a=20200116134243

	{'r2': 0.8100216230002111, 'sqrt': 7985395.0, 'nmse': 63766531652730.96, 'mean': 3580898.96, 'PredMean': 2716387.39, 'time': 20.9, 'explained_variance_score': 0.8101299091643114, 'per': 2.23}
Voting Classifier r2:0.8100216230002111

		 ==> best model r2 with voting is :[('MLPRegressor', 0.8342480877702528)]
[('r2s', 2280), ('pred', 2280), ('resultsArray', 240), ('energy', 66), ('dfn', 56)]
saved in:0.09899330139160156
b'  adding: pendingModels-electricity-kbtu-.pickle (deflated 45%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-electricity-kbtu-.pickle.zip?a=20200116134246
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-electricity-kbtu-.20200116-1342.pickle.zip?a=20200116134247
('MLPRegressor', 0.8342480877702528)
['HistGradientBoostingRegressor', 'GradientBoostingRegressor', 'KNeighborsRegressor', 'HuberRegressor', 'LassoLars', 'OrthogonalMatchingPursuit', 'S

##∑) Toutes énergies, en votation à l'aveugle
---
<b><u>No Silver Bullet</u></b>, la multiplication par GFA reste la meilleure piste, sans validation des paramètres GridSearch : pauvres performances globales
- http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-voting-linearregression-la.r2:-1.59.p:1.18.webp
- http://1.x24.fr/a/jupyter/seattle/sp-sourceeui-kbtu-sf--trimmed-voting-linearregression.r2:-12.38.p:3.37.webp
- http://1.x24.fr/a/jupyter/seattle/sp-siteeui-kbtu-sf--trimmed-voting-linearregression-l.r2:-1688973520.68.p:39363.32.webp
- http://1.x24.fr/a/jupyter/seattle/sp-siteeuiwn-kbtu-sf--trimmed-voting-linearregression.r2:-36.67.p:5.68.webp
- http://1.x24.fr/a/jupyter/seattle/sp-sourceeui-kbtu-sf--trimmed-voting-linearregression.r2:-12.38.p:3.37.webp
- http://1.x24.fr/a/jupyter/seattle/sp-sourceeuiwn-kbtu-sf--trimmed-voting-linearregressi.r2:-6.46.p:2.44.webp
- http://1.x24.fr/a/jupyter/seattle/sp-ghgemissions-metrictonsco2e--trimmed-voting-linear.r2:-38.77.p:21.7.webp
- http://1.x24.fr/a/jupyter/seattle/sp-ghgemissionsintensity-kgco2e-ft2--trimmed-voting-l.r2:-497.45.p:36.17.webp
- http://1.x24.fr/a/jupyter/seattle/sp-steamuse-kbtu--trimmed-voting-linearregression-las.r2:-43.38.p:83.95.webp
- http://1.x24.fr/a/jupyter/seattle/sp-naturalgas-kbtu--trimmed-voting-linearregression-l.r2:-44.48.p:13.78.webp
- http://1.x24.fr/a/jupyter/seattle/sp-siteenergyuse-kbtu--trimmed-voting-linearregressio.r2:-1.16.p:7.03.webp
- http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-voting-linearregress.r2:-20.49.p:6.42.webp
---

In [ ]:
#on évince les modèles prennant trop de temps ou avec de mauvais résultats : SVR,StackingRegressor,KernelRidge
param_grid['AdaBoostRegressor']={'learning_rate':[0.01],'loss':['linear'],'n_estimators':[600]}
goodPerformanceWithParams='AdaBoostRegressor,lars,XGBRegressor'.split(',')
betterPerformanceWithoutParams='GradientBoostingRegressor,BayesianRidge,AdaBoostRegressor,ExtraTreesRegressor,RandomForestRegressor,OrthogonalMatchingPursuit,HistGradientBoostingRegressor,dummy,KNeighborsRegressor,HuberRegressor,LassoLars,LinearSVR,SVR,StackingRegressor,KernelRidge,PassiveAggressiveRegressor,MLPRegressor,LinearRegression,Ridge'.split(',')
for i in betterPerformanceWithoutParams:
  param_grid[i]={}

toPredict=toPredictBase.copy()
toPredict.remove('Electricity(kBtu)')#déjà fait
models=allModels1.copy()
votingOnReel=0;shuffleData=0;stopAtOne=0;includeOriginalDf=0;includeEnergyStar=0;ignoreParameters=0;
#voted=1;noPG=0;Très mauvaise idée => Voting 7200 fits
voted=1;noPG=1#Oui ! Nous ne sommes pas sensé avoir les résultats r2
#voted=2;noPG=0;#straight Ahead, no parameters :: Mauvaise Idée: 

modelsEvalutation()
#models['SVR']=sklearn.svm.SVR()
#models['StackingRegressor']=sklearn.ensemble.StackingRegressor([('lr',sklearn.linear_model.RidgeCV()),('svr', sklearn.svm.LinearSVR(random_state=42))],sklearn.ensemble.RandomForestRegressor(n_estimators=10,random_state=42))
save(globals(),[],'checkpoint3')

____________________________________________________________________________________________________________________________________________________________________________________
}ENERGYSTARScore{
0    65.0
1    51.0
Name: ENERGYSTARScore, dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}ENERGYSTARScore-trimmed{
Keeping previous splitted data

	}ENERGYSTARScore-trimmed-dummy{


dummy-np :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.4s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-dummy-np.r2:-0.0.p:0.74.webp?a=20200117143155

	{'r2': -0.0036489160108921226, 'sqrt': 38.0, 'bs': 1453.6370957854544, 'nmse': 1453.6370957854544, 'mean': 51.5, 'PredMean': 51.04, 'time': 1.5, 'explained_variance_score': 0.0, 'per': 0.73792}
	 no parameters..
	BS:-1386.0383629653777

		 ==> Best Reel Model r2:[('ExtraTreesRegressor', 0.687316991344876)]
		 ==> Best Reel Cv Score:[('dummy', -1386.0383629653777)]


voting between : LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor,HistGradientBoostingRegressor,GradientBoostingRegressor,KNeighborsRegressor,HuberRegressor,Lars,LassoLars,BayesianRidge,LinearSVR,OrthogonalMatchingPursuit,SVR,StackingRegressor,KernelRidge,PassiveAggressiveRegressor,MLPRegressor,Ridge,XGBRegressor,AdaBoostRegressor
{'Lasso__alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564216e-04,

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished


[14:33:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
put:http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-voting-linearregression-la.r2:-1.59.p:1.18.webp?a=20200117143410

	{'r2': -1.5882990239416697, 'sqrt': 61.0, 'bs': 3748.7685346596463, 'nmse': 3748.7685346596463, 'mean': 51.5, 'PredMean': 48.37, 'time': 127.8, 'explained_variance_score': -1.571287259715171, 'per': 1.18456}


		 ==> best model r2 with voting is :[('ExtraTreesRegressor', 0.687316991344876)]
Best CV score
('dummy', -1386.0383629653777)
		Voting Classifier final r2:-1.5882990239416697
[('r2s', 4704), ('pred', 4704), ('resultsArray', 648), ('energy', 64), ('dfn', 56)]
saved in:0.11520624160766602
b'  adding: pendingModels-energystarscore.pickle (deflated 49%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-energystarscore.pickle.zip?a=20200117143418
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-energystarscore.20200117-14

...

[14:59:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyuse-kbtu--trimmed-voting-linearregressio.r2:-1.16.p:7.03.webp?a=20200117145922

	{'r2': -1.158327321671874, 'sqrt': 36440292.0, 'bs': 1327894869309816.8, 'nmse': 1327894869309816.8, 'mean': 5183071.11, 'PredMean': 8095188.43, 'time': 163.4, 'explained_variance_score': -1.1287431471551992, 'per': 7.03064}


		 ==> best model r2 with voting is :[('dummy-np', -0.004658725522887819)]
Best CV score
('dummy', -262443552700330.2)
		Voting Classifier final r2:-1.158327321671874
[('r2s', 4704), ('pred', 4704), ('resultsArray', 648), ('energy', 68), ('dfn', 56)]
saved in:0.1182396411895752
b'  adding: pendingModels-siteenergyuse-kbtu-.pickle (deflated 49%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteenergyuse-kbtu-.pickle.zip?a=20200117145930
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteenergyus

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-dummy-np.r2:-0.12.p:1.46.webp?a=20200117145938

	{'r2': -0.11597409962801164, 'sqrt': 7647610.0, 'bs': 58485936786248.5, 'nmse': 58485936786248.5, 'mean': 5227626.32, 'PredMean': 5720990.65, 'time': 0.1, 'explained_variance_score': 0.0, 'per': 1.46292}
	 no parameters..
	BS:-273294171328774.2

		 ==> Best Reel Model r2:[('dummy-np', -0.11597409962801164)]
		 ==> Best Reel Cv Score:[('dummy', -273294171328774.2)]


voting between : LinearRegression,Lasso,ElasticNet,ExtraTreesRegressor,RandomForestRegressor,HistGradientBoostingRegressor,GradientBoostingRegressor,KNeighborsRegressor,HuberRegressor,Lars,LassoLars,BayesianRidge,LinearSVR,OrthogonalMatchingPursuit,SVR,StackingRegressor,KernelRidge,PassiveAggressiveRegressor,MLPRegressor,Ridge,XGBRegressor,AdaBoostRegressor
{'Lasso__alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.995642

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.8min finished


[15:02:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-voting-linearregress.r2:-20.49.p:6.42.webp?a=20200117150226

	{'r2': -20.48513359521809, 'sqrt': 33555808.0, 'bs': 1125992230207569.5, 'nmse': 1125992230207569.5, 'mean': 5227626.32, 'PredMean': 8142262.37, 'time': 162.0, 'explained_variance_score': -20.029494794968134, 'per': 6.41894}


		 ==> best model r2 with voting is :[('dummy-np', -0.11597409962801164)]
Best CV score
('dummy', -273294171328774.2)
		Voting Classifier final r2:-20.48513359521809
[('r2s', 4704), ('pred', 4704), ('resultsArray', 648), ('energy', 70), ('dfn', 56)]
saved in:0.11440396308898926
b'  adding: pendingModels-siteenergyusewn-kbtu-.pickle (deflated 49%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteenergyusewn-kbtu-.pickle.zip?a=20200117150234
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteene

###1) Vote avec retours sur performance des meilleurs modèles via CrossValidation
---
- La cross-validation Permet de meilleurs scores généraux !
---
### Unanimité r2 & CV 
- http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-extratreesregressor.r2:+0.69.p:0.41.webp
- http://1.x24.fr/a/jupyter/seattle/sp-siteeui-kbtu-sf--trimmed-extratreesregressor.r2:+0.83.p:0.39.webp
- http://1.x24.fr/a/jupyter/seattle/sp-siteeuiwn-kbtu-sf--trimmed-extratreesregressor.r2:+0.83.p:0.37.webp
- http://1.x24.fr/a/jupyter/seattle/sp-sourceeui-kbtu-sf--trimmed-extratreesregressor.r2:+0.83.p:0.38.webpp
- http://1.x24.fr/a/jupyter/seattle/sp-sourceeuiwn-kbtu-sf--trimmed-extratreesregressor.r2:+0.83.p:0.36.webp

---
#### Divergences de choix 
- http://1.x24.fr/a/jupyter/seattle/sp-ghgemissions-metrictonsco2e--trimmed-lassolars.r2:+0.71.p:1.86.webp#r2
- http://1.x24.fr/a/jupyter/seattle/sp-ghgemissions-metrictonsco2e--trimmed-randomforestregressor.r2:+0.55.p:2.31.webp#cv
---
- http://1.x24.fr/a/jupyter/seattle/sp-ghgemissionsintensity-kgco2e-ft2--trimmed-extratreesregressor.r2:+0.73.p:0.93.webp#r2
- http://1.x24.fr/a/jupyter/seattle/sp-ghgemissionsintensity-kgco2e-ft2--trimmed-randomforestregressor.r2:+0.67.p:0.93.webp#cv
---
- http://1.x24.fr/a/jupyter/seattle/sp-steamuse-kbtu--trimmed-orthogonalmatchingpursuit.r2:+0.6.p:7.96.webp#r2
- http://1.x24.fr/a/jupyter/seattle/sp-steamuse-kbtu--trimmed-extratreesregressor.r2:+0.31.p:10.48.webp#cv
---
- http://1.x24.fr/a/jupyter/seattle/sp-naturalgas-kbtu--trimmed-extratreesregressor.r2:+0.57.p:1.34.webp#r2
- http://1.x24.fr/a/jupyter/seattle/sp-naturalgas-kbtu--trimmed-xgbregressor.r2:+0.38.p:1.61.webp#cv

In [ ]:
voted=2;noPG=0;votingOnReel=0;modelsEvalutation()
save(globals(),[],'checkpoint4')

____________________________________________________________________________________________________________________________________________________________________________________
}ENERGYSTARScore{
0    65.0
1    51.0
Name: ENERGYSTARScore, dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}ENERGYSTARScore-trimmed{
Keeping previous splitted data

	}ENERGYSTARScore-trimmed-dummy{


dummy :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.4s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-dummy.r2:-0.0.p:0.74.webp?a=20200117150412

	{'r2': -0.0036489160108921226, 'sqrt': 38.0, 'bs': 1453.6370957854544, 'nmse': 1453.6370957854544, 'mean': 51.5, 'PredMean': 51.04, 'time': 1.5, 'explained_variance_score': 0.0, 'per': 0.73792}
	 no parameters..
	BS:-1386.0383629653777

	}ENERGYSTARScore-trimmed-LinearRegression{


LinearRegression :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-linearregression.r2:-108.34.p:7.73.webp?a=20200117150419

	{'r2': -108.3386440027225, 'sqrt': 398.0, 'bs': 158360.86343515, 'nmse': 158360.86343515, 'mean': 51.5, 'PredMean': 6.52, 'time': 0.4, 'explained_variance_score': -106.82576288610969, 'per': 7.72876}
	 no parameters..
	BS:-1188.3948449951772

	}ENERGYSTARScore-trimmed-Lasso{


Lasso :: param grid :: {'alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564216e-04,
       9.23670857e-04, 1.21957046e-03, 1.61026203e-03, 2.12611233e-03,
       2.80721620e-03, 3.70651291e-03, 4.89390092e-03, 6.46167079e-03,
       8.53167852e-03, 1.12648169e-02, 1.48735211e-02, 1.96382800e-02,
       2.59294380e-02, 3.42359796e-02, 4.52035366e-02, 5.96845700e-02,
       7.88046282e-02, 1.04049831e-01, 1.37382380e-01, 1.81393069e-01,
       2.39502662e-01, 3.16227766e-01])}
Fitting 5 folds for each of

...

put:http://1.x24.fr/a/jupyter/seattle/FeatImportance-siteenergyusewn-kbtu--trimmed-xgbregressor0.63.webp?a=20200117163028
put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-xgbregressor.r2:+0.63.p:0.84.webp?a=20200117163034

	{'r2': 0.6327912794994164, 'sqrt': 4386874.0, 'bs': 19244663493279.207, 'nmse': 19244663493279.207, 'mean': 5227626.32, 'PredMean': 3368085.0, 'time': 19.5, 'explained_variance_score': 0.633032549837782, 'per': 0.83917}
	Best Parameters :: 4{'learning_rate': 0.3, 'objective': 'reg:squarederror', 'random_state': 1, 'reg': 'squarederror'}
	BS:-50729660526140.51

	}SiteEnergyUseWN(kBtu)-trimmed-AdaBoostRegressor{

range(1, 2)

AdaBoostRegressor :: param grid :: {'random_state': range(1, 2)}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.8s finished


put:http://1.x24.fr/a/jupyter/seattle/FeatImportance-siteenergyusewn-kbtu--trimmed-adaboostregressor-6.12.webp?a=20200117163054
put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-adaboostregressor.r2:-6.12.p:3.69.webp?a=20200117163101

	{'r2': -6.116335295607241, 'sqrt': 19311983.0, 'bs': 372952684464064.4, 'nmse': 372952684464064.4, 'mean': 5227626.32, 'PredMean': 21642426.72, 'time': 10.6, 'explained_variance_score': 0.33447794088887783, 'per': 3.69422}
	Best Parameters :: 1{'random_state': 1}
	BS:-354403074494602.6

		 ==> Best Reel Model r2:[('ExtraTreesRegressor', 0.7379220834239386)]
		 ==> Best Reel Cv Score:[('ExtraTreesRegressor', -46519624484978.984)]


voting between : ExtraTreesRegressor,XGBRegressor,RandomForestRegressor,GradientBoostingRegressor
{'ExtraTreesRegressor__random_state': [1], 'XGBRegressor__learning_rate': [0.3], 'XGBRegressor__objective': ['reg:squarederror'], 'XGBRegressor__random_state': [1], 'XGBRegressor__reg': ['squarederror'], 'Rando

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   42.6s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-voting-extratreesreg.r2:+0.65.p:0.82.webp?a=20200117163211

	{'r2': 0.6535592379151702, 'sqrt': 4261016.0, 'bs': 18156256958138.207, 'nmse': 18156256958138.207, 'mean': 5227626.32, 'PredMean': 3474847.65, 'time': 63.8, 'explained_variance_score': 0.6544761437270727, 'per': 0.8151}


		 ==> best model r2 with voting is :[('ExtraTreesRegressor', 0.7379220834239386)]
Best CV score
('ExtraTreesRegressor', -46519624484978.984)
		Voting Classifier final r2:0.6535592379151702
[('pred', 18528), ('r2s', 4704), ('resultsArray', 648), ('energy', 70), ('dfn', 56)]
saved in:0.2165837287902832
b'  adding: pendingModels-siteenergyusewn-kbtu-.pickle (deflated 38%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteenergyusewn-kbtu-.pickle.zip?a=20200117163220
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteenergyusewn-kbtu-.20200117-1632.pickle.zip?a=20200117163221
[('pred', 18528), ('r2s', 4704), ('resultsArray', 6

###2) Voting sur R2 réel
---
- <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting-ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor-np.r2:+0.61.p:3.2.webp'>Vote aveugle sans paramètres : 0.61</a>
- Is Cheating : <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu--trimmed-voting-BayesianRidge,LinearSVR,MLPRegressor,OrthogonalMatchingPursuit.r2:+0.81.p:2.22.webp'>0.81 Via R²</a>
- <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-voting-m:propertygfatotal-a:electricity-kbtu--dummy-ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor.r2:+0.97.p:0.95.webp'> CrossValidation puis vote puis multiplication : 0.97</a>


In [ ]:
voted=2;noPG=0;votingOnReel=1;modelsEvalutation()
save(globals(),[],'checkpoint5')

____________________________________________________________________________________________________________________________________________________________________________________
}ENERGYSTARScore{
0    65.0
1    51.0
Name: ENERGYSTARScore, dtype: float64
______________________________________________________________________________________________________________________________________________________________________________________________
}ENERGYSTARScore-trimmed{
Keeping previous splitted data

	}ENERGYSTARScore-trimmed-dummy{


dummy :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-dummy.r2:-0.0.p:0.74.webp?a=20200117163345

	{'r2': -0.0036489160108921226, 'sqrt': 38.0, 'bs': 1453.6370957854544, 'nmse': 1453.6370957854544, 'mean': 51.5, 'PredMean': 51.04, 'time': 0.1, 'explained_variance_score': 0.0, 'per': 0.73792}
	 no parameters..
	BS:-1386.0383629653777

	}ENERGYSTARScore-trimmed-LinearRegression{


LinearRegression :: param grid :: {}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-energystarscore-trimmed-linearregression.r2:-108.34.p:7.73.webp?a=20200117163351

	{'r2': -108.3386440027225, 'sqrt': 398.0, 'bs': 158360.86343515, 'nmse': 158360.86343515, 'mean': 51.5, 'PredMean': 6.52, 'time': 0.3, 'explained_variance_score': -106.82576288610969, 'per': 7.72876}
	 no parameters..
	BS:-1188.3948449951772

	}ENERGYSTARScore-trimmed-Lasso{


Lasso :: param grid :: {'alpha': array([1.00000000e-04, 1.32035178e-04, 1.74332882e-04, 2.30180731e-04,
       3.03919538e-04, 4.01280703e-04, 5.29831691e-04, 6.99564216e-04,
       9.23670857e-04, 1.21957046e-03, 1.61026203e-03, 2.12611233e-03,
       2.80721620e-03, 3.70651291e-03, 4.89390092e-03, 6.46167079e-03,
       8.53167852e-03, 1.12648169e-02, 1.48735211e-02, 1.96382800e-02,
       2.59294380e-02, 3.42359796e-02, 4.52035366e-02, 5.96845700e-02,
       7.88046282e-02, 1.04049831e-01, 1.37382380e-01, 1.81393069e-01,
       2.39502662e-01, 3.16227766e-01])}
Fitting 5 folds for each of

...

put:http://1.x24.fr/a/jupyter/seattle/FeatImportance-siteenergyusewn-kbtu--trimmed-xgbregressor0.63.webp?a=20200117175850
put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-xgbregressor.r2:+0.63.p:0.84.webp?a=20200117175857

	{'r2': 0.6327912794994164, 'sqrt': 4386874.0, 'bs': 19244663493279.207, 'nmse': 19244663493279.207, 'mean': 5227626.32, 'PredMean': 3368085.0, 'time': 19.6, 'explained_variance_score': 0.633032549837782, 'per': 0.83917}
	Best Parameters :: 4{'learning_rate': 0.3, 'objective': 'reg:squarederror', 'random_state': 1, 'reg': 'squarederror'}
	BS:-50729660526140.51

	}SiteEnergyUseWN(kBtu)-trimmed-AdaBoostRegressor{

range(1, 2)

AdaBoostRegressor :: param grid :: {'random_state': range(1, 2)}
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.8s finished


put:http://1.x24.fr/a/jupyter/seattle/FeatImportance-siteenergyusewn-kbtu--trimmed-adaboostregressor-6.12.webp?a=20200117175916
put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-adaboostregressor.r2:-6.12.p:3.69.webp?a=20200117175923

	{'r2': -6.116335295607241, 'sqrt': 19311983.0, 'bs': 372952684464064.4, 'nmse': 372952684464064.4, 'mean': 5227626.32, 'PredMean': 21642426.72, 'time': 10.7, 'explained_variance_score': 0.33447794088887783, 'per': 3.69422}
	Best Parameters :: 1{'random_state': 1}
	BS:-354403074494602.6

		 ==> Best Reel Model r2:[('ExtraTreesRegressor', 0.7379220834239386)]
		 ==> Best Reel Cv Score:[('ExtraTreesRegressor', -46519624484978.984)]


voting between : ExtraTreesRegressor,XGBRegressor,RandomForestRegressor,HistGradientBoostingRegressor
{'ExtraTreesRegressor__random_state': [1], 'XGBRegressor__learning_rate': [0.3], 'XGBRegressor__objective': ['reg:squarederror'], 'XGBRegressor__random_state': [1], 'XGBRegressor__reg': ['squarederror'], 'R

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   43.8s finished


put:http://1.x24.fr/a/jupyter/seattle/sp-siteenergyusewn-kbtu--trimmed-voting-extratreesreg.r2:+0.66.p:0.81.webp?a=20200117180034

	{'r2': 0.6616352555540597, 'sqrt': 4211058.0, 'bs': 17733009270517.004, 'nmse': 17733009270517.004, 'mean': 5227626.32, 'PredMean': 3477442.99, 'time': 64.4, 'explained_variance_score': 0.6625740012865824, 'per': 0.80554}


		 ==> best model r2 with voting is :[('ExtraTreesRegressor', 0.7379220834239386)]
Best CV score
('ExtraTreesRegressor', -46519624484978.984)
		Voting Classifier final r2:0.6616352555540597
[('pred', 18528), ('r2s', 4704), ('resultsArray', 648), ('energy', 70), ('dfn', 56)]
saved in:0.2654390335083008
b'  adding: pendingModels-siteenergyusewn-kbtu-.pickle (deflated 37%)\n'
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteenergyusewn-kbtu-.pickle.zip?a=20200117180042
put:http://1.x24.fr/a/jupyter/seattle/pendingModels-siteenergyusewn-kbtu-.20200117-1800.pickle.zip?a=20200117180044
[('pred', 18528), ('r2s', 4704), ('resultsArray', 

###3) Foreach

In [ ]:
for k in r2s.keys():
  print(k)
  print(arsort(r2s[k]))
  print('_'*180)

message('processes finished, data collected and archived')
print(cvResults.keys())
for i in cvResults.keys():
  print('_'*180)
  print(i)
  print(arsort(cvResults[i].rank_test_score))
assert(False) 

#.-------------------------------------------------------------

---
#Conclusions
---
- On ne doit pas connaitre ni le r2 score ni utiliser le dummy regressor, voter directement entre plusieurs modèles semble avoir le meilleur effet sur les prévisions, exclure tous les modèles ayant eu des performances / scores inférieurs aux prévisions statistiques de ce dernier
- <u>**Voter entre plusieurs modèles peu résulter en un meilleur coefficient de détermination, ou non **</u>
- Le meilleur modèle de prédiction de la consommation électrique est : le modèle voté, sur la Cross Validation de tous les modèles, en retenant les 4 meilleurs, puis en multipliant le résultat par la surface totale de l'immeuble : <a href='http://1.x24.fr/a/jupyter/seattle/sp-electricity-kbtu-gfa-trimmed-voting-m:propertygfatotal-a:electricity-kbtu--dummy-ridge01-ridge02-ridge1-ridge3-ridge5-ridge10-ridge20-ridge30-linearregression-lasso-elasticnet-extratreesregressor-randomforestregressor.r2:+0.97.p:0.95.webp'>Electricity Kbtu : 0.97</a>
---

Cross Validation sur du Dataframe en 20 Folds

In [ ]:
if False:#repeat prépa


  assert(False)



  from sklearn.tree import DecisionTreeRegressor
  cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
  extr=clf = sklearn.ensemble.ExtraTreesRegressor()

  #test2 = df.loc[~df.index.isin(train2.index)]

  nFeatures=[1,2,10,20]#list(range(1,20))
  #('select', SelectKBest()),'select__k':nFeatures,
  pipe = Pipeline([('extr', extr)])#array of Tuple models
  param_grid={'extr__n_estimators':[100,300],'extr__random_state':[42],'extr__min_samples_leaf':[1,8]}#{'extr__n_estimators': 100, 'select__k': 1}

  scores = cross_val_score(clf, X, y, cv=cv, scoring='r2')

  search=GridSearchCV(pipe,param_grid,cv=20,n_jobs=-1,verbose=1,return_train_score=True,scoring='neg_mean_squared_error')#r2
  search=search.fit(X,y)# <== Fitter l'ensemble de données ici, sur l'ensemble != train test split
  print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))#Pas bon du tout !


  #svm_rmse_scores = np.sqrt(-scores)
  #explain_scores(svm_rmse_scores)

  print(search.best_params_)#{'extr__min_samples_leaf': 1, 'extr__n_estimators': 100, 'extr__random_state': 42, 'select__k': 2}


  if False:
    fn=list(X.columns.values)#more features than columns, why ?
    ft=pipe.named_steps['extr'].feature_importances_
    plot_feature_importances(ft,'w',fn,fn='re.png')


AssertionError: ignored

## model.predict(RandomTestSplitResults) 
- Meilleures performances sur Electricity-KBtu/Gfa que | Electricity(KBtu) pour certains modèles et vice-versa
- Cela prend bcp bcp de temps, dépendant des corrélations

In [ ]:
models={};r2ss={};bp={};prefix='';

In [ ]:
#R2: 0.69 (+/- 0.47)
#globals().update(resume('prepa1'))
toPredict='Electricity-KBtu/Gfa'#Electricity(kBtu)
toPredict='Electricity(kBtu)'#E
print(toPredict)
prefix=filename(toPredict)

if(toPredict not in list(labelEncoded.columns.values)):
  labelEncoded[toPredict]=dfNewKeys[toPredict]

train=labelEncoded.drop([toPredict],axis=1);
for i in energies:
  if i in list(train.columns.values):
    train=train.drop([i],axis=1)
    print('drop : '+i)
X=train
test=y=labelEncoded[toPredict]
#df={}
df['gbfa'][1]
df['simple']=[X,y]

0        41.682611
1        37.709393
2        51.728641
3        45.156931
4        50.598424
           ...    
6711     42.644449
6712     24.784128
6713    136.213328
6714     24.740811
6715     23.691217
Name: Electricity-KBtu/Gfa, Length: 6716, dtype: float64

In [ ]:
## model.predict(RandomTestSplitResults) .get r2 scores
columns=list(X.columns.values)
nbCols=len(columns)
#Faire un tableau du r2 score par modèle ( déplacer l)
def modelisme(X, y, mdl, _pg, nbRealTest=10, GridSearchCVFolds=5, CrossValidationFolds=5, isObj=False, n_jobs=-1):
  global bp
#Perform Grid-Search
  a=time();
  if(isObj):
    cm=mdl
  else:
    cm=mdl()#constructeur

  gsc = GridSearchCV(estimator=cm,param_grid=_pg,cv=GridSearchCVFolds,scoring='r2',verbose=1, n_jobs=-1 )   #
  grid_search = gsc.fit(X, y)#Sur l'ensemble des données non splitées
  best_params = grid_search.best_params_ 
  print(grid_search.best_params_)
  print(time()-a);a=time()

#grid_search.feature_importances_
  if hasattr(grid_search,'feature_importances_'): 
    c1=len(grid_search.feature_importances_)
    c2=len(columns)
    if(c1 != c2):
      print('feature importance length:'+str(c1)+'<>'+str(c2))
    else:
      plot_feature_importances(grid_search.feature_importances_,'Volkov', list(X.columns.values),fn='RongoRongo.png')

#Puis Cross Validation : n passes sur ce jeu de données, faible variation : modèle solide, haute variation : pas bon du tout !    
  if(isObj):
    _mdl=mdl
  else:
    _mdl=mdl(**grid_search.best_params_)#max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"], random_state=False, verbose=False
  scores = sklearn.model_selection.cross_validate(_mdl, X, y, cv=CrossValidationFolds, scoring={'mse':'neg_mean_squared_error','r2':'r2'},verbose=1,n_jobs=-1)#r2'neg_mean_squared_error'
  #print("MSE: %0.2f (+/- %0.2f)" % (scores['test_mse'].mean(), scores['test_mse'].std() * 2))
  print("R2: %0.2f (+/- %0.2f)" % (scores['test_r2'].mean(), scores['test_r2'].std() * 2))
  print(time()-a);a=time()
  print('_'*180)
  #r2:0.76,0.43, neg mean:
  #predict=_mdl.predict(X)
  r2s=[]
  for i in list(range(0,nbRealTest)):
    x_train,y_train,x_test,y_test=sklearn.model_selection.train_test_split(X,y,train_size=0.8)
    _mdl.fit(x_train,x_test)
    predictions=_mdl.predict(y_train)
    r2=sklearn.metrics.r2_score(y_test,predictions)
    print(r2)#0.8663738878650655,0.33930533833303655
    r2s+=[r2]
  r2s=np.asarray(r2s);
  print("R2: %0.2f (+/- %0.2f)" % (r2s.mean(), r2s.std() * 2))#R2: 0.60 (+/- 0.53)
  print('_'*180)
  print(time()-a);a=time()
  return scores,_mdl,r2s

###RandomForestRegressor
- Probablement le plus robuste en l'occurence

In [ ]:
#toPredict='Electricity(kBtu)';train=X=labelEncoded.drop([toPredict],axis=1);test=y=labelEncoded[toPredict]
sub='RandomForestRegressor'
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X, y, sklearn.ensemble.RandomForestRegressor,{'max_depth':[1,30,60,100,300],'n_estimators':[10,300,600,1000]})#R2: 0.69 (+/- 0.47)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 23.2min finished


{'max_depth': 30, 'n_estimators': 10}
1396.1888046264648


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.5s finished


R2: 0.82 (+/- 0.52)
2.4596104621887207
____________________________________________________________________________________________________________________________________________________________________________________
0.9681306760848271
0.9840525497439472
0.36897884210768483
0.9254375427647815
0.9085605397094192
0.9826142422255222
0.39359289547776777
0.8412771700825865
0.9635981657979505
0.980574999924759
R2: 0.83 (+/- 0.46)
____________________________________________________________________________________________________________________________________________________________________________________
9.789309740066528


###decisiontreeregressor

In [ ]:
sub='DecisionTreeRegressor'
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.tree.DecisionTreeRegressor,{'max_depth': range(2,40,3),'min_samples_split': range(2,40,3)})#R2: 0.65 (+/- 0.67)#criterion{“mse”, “friedman_mse”, “mae”}, default=”mse” #

Fitting 5 folds for each of 169 candidates, totalling 845 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 376 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done 845 out of 845 | elapsed:   43.0s finished


{'max_depth': 35, 'min_samples_split': 2}
43.21695113182068


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


R2: 0.81 (+/- 0.60)
0.5036442279815674
____________________________________________________________________________________________________________________________________________________________________________________
0.7573953349473758
0.47674223492880463
0.979407271819626
0.9542414895865953
0.8217190604873794
0.9905483746071078
0.9706131318616928
0.9664105159009582
0.945644787281083
0.9661240065417739
R2: 0.88 (+/- 0.31)
____________________________________________________________________________________________________________________________________________________________________________________
1.9387414455413818


###linearregression

In [ ]:
sub='LinearRegression'
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.LinearRegression,{'normalize':[True,False]})#R2: 0.57 (+/- 0.33) plus robuste

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'normalize': False}
0.48671865463256836


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


R2: 0.35 (+/- 0.87)
0.2781238555908203
____________________________________________________________________________________________________________________________________________________________________________________
0.7496131308665804
0.5621062141027631
0.8069313366152634
0.7956393815492928
0.5887171683627628
0.645380966133988
0.5501485944649174
0.7055947388110747
0.7653169896277796
0.7784264006577515
R2: 0.69 (+/- 0.19)
____________________________________________________________________________________________________________________________________________________________________________________
0.9179251194000244


###histgradientboost

In [ ]:
sub='HistGradientBoostingRegressor';#input contains NaN with LR:20
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.ensemble.HistGradientBoostingRegressor,{'learning_rate':[0.0001,0.001,0.01,0.1,1,2]});#R2: 0.46 (+/- 0.38) best=R2-variance

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   27.9s finished


{'learning_rate': 1}
30.046234369277954


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.2s finished


R2: 0.74 (+/- 0.37)
7.235269069671631
____________________________________________________________________________________________________________________________________________________________________________________
0.7862776525031813
0.8576334568367883
0.6549264916331976
0.911078659272331
0.8733545418073798
0.649884141998812
0.8562238176787424
0.8625937548943396
0.8358601004125803
0.7535656058169293
R2: 0.80 (+/- 0.17)
____________________________________________________________________________________________________________________________________________________________________________________
18.715363264083862


###lasso

In [ ]:
sub='Lasso';#input contains NaN with LR:20
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.ElasticNet,{'alpha': [0.1,1,10,20,30]});#R2: 0.46 (+/- 0.38)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    9.9s finished


{'alpha': 30}
10.81537389755249


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.1s finished


R2: 0.38 (+/- 0.75)
2.123455286026001
____________________________________________________________________________________________________________________________________________________________________________________
0.681519300732597
0.6357072441011757
0.8346273761796501
0.8460144971721139
0.7349077699286899
0.724496580209022
0.6675518907557105
0.6445019659770503
0.5896419175632859
0.7928179117107217
R2: 0.72 (+/- 0.17)
____________________________________________________________________________________________________________________________________________________________________________________
7.898242473602295


###ridge

In [ ]:
sub='Ridge';
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.ElasticNet,{'alpha':[0.1,1,10,20,30]});#R2: 0.59 (+/- 0.29)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    9.0s finished


{'alpha': 30}
9.97952151298523


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.1s finished


R2: 0.38 (+/- 0.75)
2.151357412338257
____________________________________________________________________________________________________________________________________________________________________________________
0.6244402308069004
0.7026642140929238
0.6980745793458298
0.6145493752563208
0.6542890516580859
0.535755656816757
0.7067253015741223
0.6683979049398521
0.674199883657026
0.6451292357909193
R2: 0.65 (+/- 0.10)
____________________________________________________________________________________________________________________________________________________________________________________
8.252269268035889


###XGB

In [ ]:
sub='XGBRegressor';
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,xgboost.XGBRegressor,{'objective':['reg:squarederror'],'learning_rate':[0.0001,0.001,0.01,0.1,1,2],#R2: 0.48 (+/- 0.51)
#'max_depth'        : [3, 15],#4, 5, 6, 8, 10, 12, 
#'min_child_weight' : [1, 7],#3, 5, 
#ValueError: Invalid parameter colsample_bytree for estimator GaussianNB(priors=None, var_smoothing=1e-09). Check the list of available parameters with `estimator.get_params().keys()`. 
#'gamma'            : [0.0,  0.4],#0.1, 0.2 , 0.3,
#'colsample_bytree' : [0.3,  0.7],#0.4, 0.5 ,
})

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   17.3s finished


{'learning_rate': 0.1, 'objective': 'reg:squarederror'}
19.050434350967407


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.7s finished


R2: 0.82 (+/- 0.54)
3.6799886226654053
____________________________________________________________________________________________________________________________________________________________________________________
0.9074820733426772
0.8938714024046002
0.9017991130001008
0.9513460764641957
0.4201287948882342
0.9608555103507808
0.9028712390040758
0.8927940611552455
0.9263710661834482
0.975300670820485
R2: 0.87 (+/- 0.31)
____________________________________________________________________________________________________________________________________________________________________________________
14.43119764328003


###gradbbost

In [ ]:
sub='GradientBoostingRegressor';  #R2: 0.92 (+/- 0.09)
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.ensemble.GradientBoostingRegressor,{'learning_rate':[0.0001,0.001,0.01,0.1,1,2,10]});#R2: 0.72 (+/- 0.42)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:   36.3s finished


{'learning_rate': 0.1}
39.68752384185791


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    6.7s finished


R2: 0.82 (+/- 0.54)
6.714620113372803
____________________________________________________________________________________________________________________________________________________________________________________
0.9785032542347022
0.9486235688265748
0.7921450547867691
0.9603734985698664
0.40012764702558956
0.9705521682194642
0.8745921154362605
0.9885283456751811
0.3041839435808519
0.6059882273934983
R2: 0.78 (+/- 0.49)
____________________________________________________________________________________________________________________________________________________________________________________
27.141116857528687


###passagressive

In [ ]:
sub='PassiveAggressiveRegressor';  
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.PassiveAggressiveRegressor,{'max_iter':[10,100,200,300,500,1000,2000,5000]});#R2: 0.44 (+/- 0.52)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'max_iter': 2000}
1.4019429683685303


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


R2: 0.24 (+/- 0.40)
0.2175614833831787
____________________________________________________________________________________________________________________________________________________________________________________
0.35149398088695305
0.6510102785595984
0.15656377016124612
0.38738807790886143
0.9161365587319412
0.6790342051245111
0.7066445157558786
0.18925823014766763
0.1271458109431074
0.41003904172547745
R2: 0.46 (+/- 0.51)
____________________________________________________________________________________________________________________________________________________________________________________
1.127342939376831


###ortho

In [ ]:
sub='OrthogonalMatchingPursuit';  
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.OrthogonalMatchingPursuit,{'n_nonzero_coefs':[None,round(nbCols*0.1),round(nbCols*0.2),round(nbCols*0.9)],'precompute':[True,False]});

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'n_nonzero_coefs': None, 'precompute': False}
1.0728662014007568


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


R2: 0.43 (+/- 0.74)
0.21706295013427734
____________________________________________________________________________________________________________________________________________________________________________________
0.7883543519344234
0.6951864685531843
0.6633923078712287
0.7253445174725717
0.7815879164148448
0.6579220907595515
0.7585676727958255
0.6050077056192977
0.7186742775182535
0.5569023593508262
R2: 0.70 (+/- 0.14)
____________________________________________________________________________________________________________________________________________________________________________________
0.6875419616699219


###elasticnet

In [ ]:
sub='ElasticNet';  
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.ElasticNet,{'alpha':[0.0001,0.01,1,10]});

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    6.6s finished


{'alpha': 0.0001}
7.482032775878906


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s finished


R2: 0.38 (+/- 0.90)
1.9515461921691895
____________________________________________________________________________________________________________________________________________________________________________________
0.705942223982511
0.8124649446989184
0.6965591453579856
0.6833389341341898
0.802526430147571
0.7989544234082674
0.755352902398611
0.6595077907245837
0.7650050320949634
0.6566935185555884
R2: 0.73 (+/- 0.11)
____________________________________________________________________________________________________________________________________________________________________________________
7.473644733428955


###MLP

In [ ]:
sub='MLPRegressor';#R2: 1.00 (+/- 0.00) whaow !
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.neural_network.MLPRegressor,{'activation':['relu'],'hidden_layer_sizes':[(100,)],'solver':['adam'],'learning_rate':['constant','adaptive'],'learning_rate_init':[0.1,0.001]});

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   31.1s finished


{'activation': 'relu', 'hidden_layer_sizes': (100,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.1, 'solver': 'adam'}
33.441823959350586


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.9s finished


R2: 0.45 (+/- 0.60)
2.8935322761535645
____________________________________________________________________________________________________________________________________________________________________________________
0.649790808133506
0.714960663806864
0.5427664619236299
0.6103151353625411
0.6890279669531413
0.39372232398803275
0.29448842571852707
0.5254331796469617
0.9506337738764492
0.29595679117272267
R2: 0.57 (+/- 0.39)
____________________________________________________________________________________________________________________________________________________________________________________
13.45946979522705


###adaboost

In [ ]:
sub='AdaBoostRegressor';#{'learning_rate': 0.01, 'loss': 'square', 'n_estimators': 600},  R2: 0.75 (+/- 0.36) pondérations à la suite sur des stumps
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.ensemble.AdaBoostRegressor,{'n_estimators':[10,600],'learning_rate':[0.01,10],'loss':['linear','square']});

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  4.1min finished


{'learning_rate': 0.01, 'loss': 'linear', 'n_estimators': 600}
306.9471688270569


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.0min finished


R2: 0.59 (+/- 0.49)
122.05926704406738
____________________________________________________________________________________________________________________________________________________________________________________
0.756532268512949
0.8307811377587908
0.3465917509354599
0.364960326028097
0.8366988034214771
0.8851827311549612
0.3466680989554114
0.802227778117031
0.8729805722239758
0.71510256899754
R2: 0.68 (+/- 0.43)
____________________________________________________________________________________________________________________________________________________________________________________
450.7638580799103


###kridge

In [ ]:
sub='KernelRidge';#long à calculer
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.kernel_ridge.KernelRidge,{'alpha':[0.0001,0.001,0.01,1,5,10]});#{'alpha': 1} en 1527 secondes, R2: 0.19 (+/- 2.07) = bof bof
message('KernelRidgeOver')

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 10.4min finished


{'alpha': 10}
630.412529706955


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.0s finished


R2: 0.31 (+/- 0.78)
9.010225296020508
____________________________________________________________________________________________________________________________________________________________________________________
0.8362637409671347
0.58909929624563
0.7158331945543313
0.8096436550183279
0.8025191484850676
0.8584302219568112
0.5830515258077852
0.7744691746889648
0.7149317681451928
0.7816382921811225
R2: 0.75 (+/- 0.18)
____________________________________________________________________________________________________________________________________________________________________________________
24.64918804168701


###kneigh

In [ ]:
sub='KNeighborsRegressor';  
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.neighbors.KNeighborsRegressor,{'n_neighbors':[1,5,10,15,21,30,100]});

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    4.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'n_neighbors': 1}
4.344673156738281


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished


R2: 0.51 (+/- 0.56)
0.5018460750579834
____________________________________________________________________________________________________________________________________________________________________________________
0.8048394582579246
0.7057998600785677
0.16663306376971865
0.23068338681549305
0.3027178942507678
0.7411391429109961
0.5510635059199243
0.8021150778692152
0.6894456068217067
0.49404376711045783
R2: 0.55 (+/- 0.46)
____________________________________________________________________________________________________________________________________________________________________________________
1.8396415710449219


###huber

In [ ]:
sub='HuberRegressor';  
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.HuberRegressor,{'alpha':[0.0001,0.01,1,10],'max_iter':[10,100,300,500],'epsilon':[1.35],'tol':[0.00001]});

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    9.7s finished


{'alpha': 1, 'epsilon': 1.35, 'max_iter': 300, 'tol': 1e-05}
10.056734800338745


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.4s finished


R2: 0.57 (+/- 0.24)
1.4083120822906494
____________________________________________________________________________________________________________________________________________________________________________________
0.46750922341200396
0.48931671349214556
0.2641872189418346
0.5928882633148604
0.6221839919272145
0.5398708994991297
0.3445674750482922
0.6883839112638799
0.3262022221811105
0.6722131569432339
R2: 0.50 (+/- 0.28)
____________________________________________________________________________________________________________________________________________________________________________________
4.70572566986084


###bayesianridge

In [ ]:
sub='BayesianRidge';  
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.BayesianRidge,{'lambda_1':[0.000001,0.001,1,10]});

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'lambda_1': 1e-06}
1.139657735824585


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.4s finished


R2: 0.42 (+/- 0.47)
0.3915574550628662
____________________________________________________________________________________________________________________________________________________________________________________
0.8260958082034346
0.47124531562224026
0.30515581237168976
0.33292368011657403
0.5907966861848508
0.623179657358242
0.5373518476291173
0.5053850491754002
0.8155029773155511
0.5950416750323955
R2: 0.56 (+/- 0.33)
____________________________________________________________________________________________________________________________________________________________________________________
1.0644776821136475


###lars

In [ ]:
sub='Lars';  
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.Lars,{'n_nonzero_coefs':[1,500,np.inf]});

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'n_nonzero_coefs': 500}
0.6727132797241211


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished


R2: 0.23 (+/- 0.82)
0.2723388671875
____________________________________________________________________________________________________________________________________________________________________________________
0.7577140771971871
0.6711009615608977
0.5859959470090856
0.7285842117693628
0.7918030226868393
0.6136089213941267
0.612236379044204
0.7343323280822202
0.6990549737347671
0.7706533454016387
R2: 0.70 (+/- 0.14)
____________________________________________________________________________________________________________________________________________________________________________________
1.0692048072814941


###lassolars

In [ ]:
sub='LassoLars';#R2: 0.86 (+/- 0.07) Excellent !!!!!
scores,_mdl,r2ss[prefix+sub]=modelisme(X,y,sklearn.linear_model.LassoLars,{'alpha':[0.0001,0.01,1,10]});

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


{'alpha': 0.0001}
0.7757794857025146


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


R2: 0.39 (+/- 0.88)
0.24616599082946777
____________________________________________________________________________________________________________________________________________________________________________________
0.6957651439011232
0.5660032489260058
0.8462372572540084
0.8453190089645225
0.5825046939155
0.5619096378669581
0.6055224267176251
0.7546553060692529
0.6245749883879327
0.789308789897073
R2: 0.69 (+/- 0.22)
____________________________________________________________________________________________________________________________________________________________________________________
0.958319902420044


###linearsvr

In [ ]:
sub='LinearSVR';#'random_state':[0,42],
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.svm.LinearSVR,{'tol':[0.0001],'C':[1,100,400],'epsilon':[0,1,10],'loss':['epsilon_insensitive'],'fit_intercept':[True],'intercept_scaling':[1],'dual':[True],'verbose':[0],'random_state':[None],'max_iter':[1000]});

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   14.8s finished


{'C': 400, 'dual': True, 'epsilon': 0, 'fit_intercept': True, 'intercept_scaling': 1, 'loss': 'epsilon_insensitive', 'max_iter': 1000, 'random_state': None, 'tol': 0.0001, 'verbose': 0}
15.85109543800354


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.1s finished


R2: 0.58 (+/- 0.29)
2.0830905437469482
____________________________________________________________________________________________________________________________________________________________________________________
0.21066372846172665
0.5008779322554794
0.6610117814418026
0.4120208996454765
0.2790626833482084
0.622776064124041
0.3199599518153038
0.5835801753799872
0.5591885791491857
0.6125237439510391
R2: 0.48 (+/- 0.30)
____________________________________________________________________________________________________________________________________________________________________________________
8.702344417572021


###svr

In [ ]:
sub='SVR';#stable mais faible !
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.svm.SVR,{'kernel':['rbf'],'degree':[3],'gamma':['scale'],'coef0':[0.0],'tol':[0.001],'C':[1.0],'epsilon':[0.1],'shrinking':[True],'cache_size':[200],'verbose':[False],'max_iter':[-1]});

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   17.4s finished


{'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
27.743587017059326


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   17.1s finished


R2: -0.06 (+/- 0.11)
17.096968173980713
____________________________________________________________________________________________________________________________________________________________________________________
-0.10017433172245283
-0.019512741751613216
-0.035016028174348346
-0.0187272222873045
-0.06392909658837764
-0.045414970280371
-0.08501279436489129
-0.05661859950019932
-0.023151312508999045
-0.04196549384451598
R2: -0.05 (+/- 0.05)
____________________________________________________________________________________________________________________________________________________________________________________
79.41922402381897


###SDGR

In [ ]:
%%script false 
#Stochastic Gradient Descent is sensitive to feature scaling, so it is highly recommended to scale your data.
#Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.
X_scaled=sklearn.preprocessing.StandardScaler()
y_scaled=sklearn.preprocessing.StandardScaler()
X_scaled.fit(X)
y_scaled.fit(y)

sub='SGDRegressor';
model=sklearn.linear_model.SGDRegressor
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X_scaled,y_scaled,model,
                                            {'alpha':[0.0001,0.2,1,2,10],'eta0':[0.000001,0.01],'max_iter':[1000],'verbose':[1],'loss':['squared_loss'],'learning_rate':['optimal','invscaling'],'power_t':[0.25]});

###StackingRegressor

In [ ]:
sub='StackingRegressor';#R2: 0.83 (+/- 0.33),
cvm=sklearn.ensemble.RandomForestRegressor(n_estimators=300);cv=5;cvm=None;#is ridgeCV
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,sklearn.ensemble.StackingRegressor([('Ridge',sklearn.linear_model.RidgeCV()),('BayesianRidge', sklearn.linear_model.BayesianRidge())], cvm,cv,n_jobs=-1),
                                       {'verbose':[1],'cv':[30,60]},
                                       nbRealTest=10,isObj=1);

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   42.1s finished


{'cv': 30, 'verbose': 1}
46.391765832901


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.2s finished


R2: -950449.92 (+/- 2582204.05)
3.1725335121154785
____________________________________________________________________________________________________________________________________________________________________________________
-461.99277463961494
-3675395.1656684843
-13087287.930629073
-21373.90810703905
-8709062.477864053
-757619.6335912847
-95557.18996347871
-967534.8793498505
-76254.59841875726
-1850422.709036986
R2: -2924097.05 (+/- 8487156.11)
____________________________________________________________________________________________________________________________________________________________________________________
8.021090507507324


###Stack2:Boosting
- Si l'on prenait 

In [ ]:
sub='Stack2';
cvm=sklearn.ensemble.RandomForestRegressor(n_estimators=300);cv=5;cvm=None;#is ridgeCV
scores,models[prefix+sub],r2ss[prefix+sub]=modelisme(X,y,
sklearn.ensemble.StackingRegressor([
            ('XGB',xgboost.XGBRegressor(learning_rate=1,objective='reg:squarederror')),
            ('GradBoost', sklearn.ensemble.GradientBoostingRegressor(learning_rate=0.1)),
            ('HistGradBoost',sklearn.ensemble.HistGradientBoostingRegressor(learning_rate=0.1)),
            ], cvm,cv,n_jobs=-1),
{'verbose':[1],'cv':[30,60]},#cela prend bien du temps
nbRealTest=10,isObj=1);

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 17.9min finished


{'cv': 30, 'verbose': 1}
1158.0451381206512


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.1min finished


R2: -53171028.03 (+/- 182104975.27)
64.9918212890625
____________________________________________________________________________________________________________________________________________________________________________________
-260765163.02512923
-11503.331083991705
-309.45179551845877
-797.0512571056587
-38.44917022398554
-23251463.885779154
-20730582.62937658
-442860.3065798532
-11174.397915908143
-62.34502891790291
R2: -30521395.49 (+/- 154471354.54)
____________________________________________________________________________________________________________________________________________________________________________________
163.13150715827942


###20 Real Test Shuffled Data on each Model

In [ ]:
#models
message('final countdown began a long time ago :'+prefix)
r2sPerModel={}
nbRealTest=20
#print(prefix)
#models.keys()
mk=dict.fromkeys(models.keys(),[])

for i in mk:
  if 'simple----Ops' in i:
    nk='simple-'+re.sub(r"simple-*",'', i);
    #nk='simple-'+i.replace('simple-*','')#
    print(i+' => '+nk)
    models[nk]=models[i];
    del(models[i]);

  if (not i.startswith('electricity-kbtu-gfa')) & (not i.startswith('simple-')):
    nk='simple-'+i.replace('kbtu-','kbtu-')    
    models[nk]=models[i];del(models[i]);print(i+'=>'+nk)  

prefixs='simple-electricity-kbtu,electricity-kbtu-gfa'.split(',');
for prefix in prefixs:
  print('prefix is:',prefix)
  for j in list(range(0,nbRealTest)):
    print('_'*180)    
    if('gfa' in prefix):
      y=df['gbfa'][1];print('gfa')
    else: 
      y=df['simple'][1]

    x_train,y_train,x_test,y_test=sklearn.model_selection.train_test_split(X,y,train_size=0.8)
    for i in models:
      if not i.startswith(prefix):
        #print('skippin:not same prefix:'+i,i.startswith(prefix));
        continue;      
      if i == prefix+'SGDRegressor':
        continue;
      if(prefix+'-'+i not in r2sPerModel.keys()):
        r2sPerModel[prefix+'-'+i]=[]      
      models[i].fit(x_train,x_test)
      predictions=models[i].predict(y_train)
      if False:#rapporter à Electricity-KBtu Total => tricher un peu, non ? car le r2 va être plus fort ( l'écart vis à vis de la métrique globale plus faible )
        if(toPredict=='Electricity-KBtu/Gfa'):
          predictions*=x_test['PropertyGFATotal']
          y_test*=x_test['PropertyGFATotal']

      r2=sklearn.metrics.r2_score(y_test,predictions)
      print(i+':'+str(round(r2,2)))#0.8663738878650655,0.33930533833303655
      r2sPerModel[prefix+'-'+i]+=[r2]

save(globals(),[],filename(toPredict),'r2sPerModel'.split(','))

prefix is: simple-electricity-kbtu
____________________________________________________________________________________________________________________________________________________________________________________
simple-electricity-kbtu-LinearSVR:0.53
simple-electricity-kbtu-SVR:-0.06
simple-electricity-kbtu-StackingRegressor:-145007.26
simple-electricity-kbtu-Stack2:-180741169.55
simple-electricity-kbtu-RandomForestRegressor:0.94
simple-electricity-kbtu-DecisionTreeRegressor:0.96
simple-electricity-kbtu-LinearRegression:0.78
simple-electricity-kbtu-HistGradientBoostingRegressor:0.86
simple-electricity-kbtu-Lasso:0.72
simple-electricity-kbtu-Ridge:0.72
simple-electricity-kbtu-XGBRegressor:0.97
simple-electricity-kbtu-GradientBoostingRegressor:0.98
simple-electricity-kbtu-PassiveAggressiveRegressor:0.58
simple-electricity-kbtu-OrthogonalMatchingPursuit:0.78
simple-electricity-kbtu-ElasticNet:0.78
simple-electricity-kbtu-MLPRegressor:0.58
simple-electricity-kbtu-AdaBoostRegressor:0.79

- simple-electricity-kbtu-simple-electricity-kbtu-GradientBoostingRegressor R2: 0.78 (+/- 0.51)
- simple-electricity-kbtu-simple-electricity-kbtu-ElasticNet R2: 0.71 (+/- 0.18))
----
- electricity-kbtu-gfa-electricity-kbtu-gfaLinearRegression R2: 1.00 (+/- 0.00)

In [ ]:
#save(globals(),[],filename(toPredict),'r2sPerModel'.split(','))
for prefix in prefixs:
  print('_'*180)
  print(prefix)
  for i in r2sPerModel.keys():
    if(not i.startswith(prefix)):
      continue
    _x=np.asarray(r2sPerModel[i]);
    print(i,"R2: %0.2f (+/- %0.2f)" % (_x.mean(), _x.std() * 2))#R2: 0.60 (+/- 0.53)

____________________________________________________________________________________________________________________________________________________________________________________
simple-electricity-kbtu
simple-electricity-kbtu-simple-electricity-kbtu-LinearSVR R2: 0.42 (+/- 1.13)
simple-electricity-kbtu-simple-electricity-kbtu-SVR R2: -0.05 (+/- 0.05)
simple-electricity-kbtu-simple-electricity-kbtu-StackingRegressor R2: -16997330.48 (+/- 98874392.07)
simple-electricity-kbtu-simple-electricity-kbtu-Stack2 R2: -21198336.07 (+/- 94086033.48)
simple-electricity-kbtu-simple-electricity-kbtu-RandomForestRegressor R2: 0.76 (+/- 0.49)
simple-electricity-kbtu-simple-electricity-kbtu-DecisionTreeRegressor R2: 0.78 (+/- 0.53)
simple-electricity-kbtu-simple-electricity-kbtu-LinearRegression R2: -11.37 (+/- 142.78)
simple-electricity-kbtu-simple-electricity-kbtu-HistGradientBoostingRegressor R2: 0.73 (+/- 0.38)
simple-electricity-kbtu-simple-electricity-kbtu-Lasso R2: 0.70 (+/- 0.19)
simple-elect

In [ ]:
#bagging = sklearn.ensemble.BaggingRegressor(sklearn.ensemble.ExtraTreesRegressor(),max_samples=0.5, max_features=0.5)
#voting = sklearn.ensemble.Voting(sklearn.ensemble.ExtraTreesRegressor())
assert(False)
globals().update(resume('checkpoint5'))#seattle
#-------------------------------------------------

from sklearn.pipeline import Pipeline
from sklearn.datasets import load_breast_cancer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

toPredict='Electricity(kBtu)';train=X=labelEncoded.drop([toPredict],axis=1);test=y=labelEncoded[toPredict]
nbRuns=1
train=X=labelEncoded.drop([toPredict],axis=1);test=y=labelEncoded[toPredict]
#sklearn pipeline regressor hyper parameters
for i in range(1,nbRuns+1):
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(labelEncoded.drop([toPredict],axis=1),labelEncoded[toPredict],test_size=0.2)#,random_state=100

#this is the classifier used for feature selection
  rfr_featr_sele = sklearn.ensemble.RandomForestRegressor(n_estimators=30, random_state=42) #,class_weight="balanced"
  rfecv = RFECV(estimator=rfr_featr_sele, step=1, cv=5, scoring = 'r2')

#you can have different classifier for your final classifier
  rfr = sklearn.ensemble.RandomForestRegressor(n_estimators=10, random_state=42)#,class_weight="balanced"
  CV_rfc = GridSearchCV(rfr, param_grid={'max_depth':[2,3]},cv= 5, scoring = 'r2')#roc_auc: pour classification

  pipeline  = Pipeline([('feature_sele',rfecv),('rfr_cv',CV_rfc)])
  pipeline.fit(X_train, y_train)
  pipeline.predict(X_test)

  nmse=sklearn.metrics.mean_squared_error(y_test,predictions)
  r2=sklearn.metrics.r2_score(y_test,predictions)
  print(nmse)
  print(r2)
  #pipeline._best_params
  #pipeline._best_estimator

from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)
extr=clf = sklearn.ensemble.ExtraTreesRegressor()
scores = cross_val_score(clf, X, y, cv=cv, scoring='r2')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))#Pas bon du tout !
#print(scores)#sur les n splits
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))#Pas bon du tout !

k_fold = sklearn.model_selection.KFold(n_splits=20)
scores = sklearn.model_selection.cross_val_score(clf,train,test, cv=k_fold, n_jobs=-1, scoring='r2')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))#Accuracy: 0.94 (+/- 0.34)


from sklearn.feature_selection import SelectKBest#number of best features passed to the model


X=train=labelEncoded.drop(energies,axis=1)#newKeys
y=test=labelEncoded[toPredict]


'''
num_pipeline = Pipeline([('imputer', SimpleImputer(strategy='median')),('drop_attributes', AttributeDeleter()),('std_scaler', StandardScaler()),])
'''


important_names = fn[ft > np.mean(ft)]

len(ft)# len(X.columns.values);117, len(X.columns.values)#105
plot_feature_importances(ft,'w',fn,fn='re.png')
save(globals(),[],'1','ft,search,X,y'.split(','))


pipe.steps[1][1].fit(train,test)
extr.fit(train,test)
extr.feature_importances_

#pipe.steps[1][1]

print(search.best_params_)
print(search.cv_results_['mean_test_score'],search.cv_results_['mean_train_score'],search.cv_results_['params'])#

for train_indices, test_indices in k_fold.split(X):
  clf.fit(train_indices,test_indices)
  #print('Train: %s | test: %s' % (train_indices, test_indices))